Libraries

In [1]:
import requests
from google.transit import gtfs_realtime_pb2
import geopandas as gpd
from shapely.geometry import Point, LineString
import pandas as pd
import leafmap
from IPython.display import display, clear_output
import csv

Dictionaries

In [2]:
# Line Dict
lines = {
    "A104": ["27", "Село Кътина - Метростанция Княгиня Мария Луиза - Село Кътина"],
    "TM37": ["3A", "Метростанция К. Величков - Гара Захарна фабрика - Метростанция К. Величков"],
    "A242": ["Y1", "БАЛКАНКАР АД - СУ СВ. КЛИМЕНТ ОХРИДСКИ - БАЛКАНКАР АД"],
    "A108": ["260", "Ж.К. КРАСНО СЕЛО - БУЛ. ПРАГА - Ж.К. КРАСНО СЕЛО"],
    "A256": ["805", "Метростанция Витоша - Студентски град - Метростанция Витоша"],
    "A27": ["81", "СЕВЕРНА ПРОМИШЛЕНА ЗОНА - КВ. ИВАНЯНЕ - СЕВЕРНА ПРОМИШЛЕНА ЗОНА"],
    "A28": ["98", "Метростанция Витоша - КМЕТСТВО ЖЕЛЕЗНИЦА - Метростанция Витоша"],
    "A246": ["109", "МЕТРОСТАНЦИЯ АКАД. А. Т.- БАЛАН - Ж.К. ЛОЗЕНЕЦ - МЕТРОСТАНЦИЯ АКАД. А. Т.- БАЛАН"],
    "TB34": ["20T", "ТЕАТЪР СОФИЯ - АВТОСТАНЦИЯ ГЕО МИЛЕВ - ТЕАТЪР СОФИЯ"],
    "A259": ["X43", "УЛ. ВАСИЛ КЪНЧЕВ - АВТОСТАНЦИЯ БАНКЯ - УЛ. ВАСИЛ КЪНЧЕВ"],
    "TB10": ["1", "Ж.К. ЛЕВСКИ Г - ПЕТА ГРАДСКА БОЛНИЦА - Ж.К. ЛЕВСКИ Г"],
    "A110": ["90", "АП Малашевци - Град Бухово - АП Малашевци"],
    "A251": ["X10", "УЛ. Т. КАБЛЕШКОВ - Ж.К. МЛАДОСТ-1 - УЛ. Т. КАБЛЕШКОВ"],
    "A48": ["26", "Ж.К. ОБЕЛЯ 1 - НАЧАЛОТО КВ. ГНИЛЯНЕ - Ж.К. ОБЕЛЯ 1"],
    "A12": ["102", "БАЛКАНКАР АД - Ж.К. ОВЧА КУПЕЛ-2 - БАЛКАНКАР АД"],
    "A61": ["61", "ЗЛАТНИ МОСТОВЕ - БУЛ. ЧЕРНИ ВРЪХ - ЗЛАТНИ МОСТОВЕ"],
    "TB21": ["9А", "АВТОБАЗА ИСКЪР - УЛ. ГЕН. ГУРКО - АВТОБАЗА ИСКЪР"],
    "A253": ["802", "МЕТРОСТАНЦИЯ ГОРНА БАНЯ - МЕТРОСТАНЦИЯ ГОРНА БАНЯ - МЕТРОСТАНЦИЯ ГОРНА БАНЯ"],
    "A79": ["70", "НСБАЛ ПО ОНКОЛОГИЯ - СЕЛО ПЛАНА - НСБАЛ ПО ОНКОЛОГИЯ"],
    "TB8": ["9", "Бул. Прага - ПЛ. СТОЧНА ГАРА - Бул. Прага"],
    "TM50": ["23", "Ж.К. ГЕО МИЛЕВ - 28-ми пощенски клон - Ж.К. ГЕО МИЛЕВ"],
    "A125": ["5TM", "Автостанция Княжево - Бул. Братя Бъкстон - Автостанция Княжево"],
    "TM51": ["4", "Ж.к. Бъкстон - Ул. Клокотница - Ж.к. Бъкстон"],
    "A174": ["42A", "Автостанция Банкя - Края на кв. Михайлово - Автостанция Банкя"],
    "TM47": ["22A", "Младежки театър - Ж.К. КРАСНА ПОЛЯНА - Младежки театър"],
    "A217": ["304", "Ж.К. ДРУЖБА-2 - НИМ - Ж.К. ДРУЖБА-2"],
    "A45": ["23", "ГАРА КУРИЛО - АВТОСТАНЦИЯ ОРЛАНДОВЦИ - ГАРА КУРИЛО"],
    "TB43": ["288", "АВТОСТАНЦИЯ ГЕО МИЛЕВ - ЗООПАРКА - АВТОСТАНЦИЯ ГЕО МИЛЕВ"],
    "TM29": ["11", "ХЮНДАЙ БЪЛГАРИЯ - Кв. Княжево - ХЮНДАЙ БЪЛГАРИЯ"],
    "A70": ["5", "СЕЛО ДОЛНИ ЛОЗЕН - АВТОСТАНЦИЯ ГЕО МИЛЕВ - СЕЛО ДОЛНИ ЛОЗЕН"],
    "A74": ["8", "СЕЛО КРИВИНА - ГАРА ИСКЪР - СЕЛО КРИВИНА"],
    "A13": ["31", "СЕЛО ГОЛЯНОВЦИ - МЕТРОСТАНЦИЯ СЛИВНИЦА - СЕЛО ГОЛЯНОВЦИ"],
    "A262": ["46", "АВТОСТАНЦИЯ БАНКЯ - МАНАСТИР СВ. ПЕТКА ПАРАСКЕВА - АВТОСТАНЦИЯ БАНКЯ"],
    "TB33": ["6", "село Герман - АВТОСТАНЦИЯ ГЕО МИЛЕВ - село Герман"],
    "A76": ["14", "АВТОСТАНЦИЯ ИЗТОК - ГАРА ИСКЪР - АВТОСТАНЦИЯ ИЗТОК"],
    "A119": ["83А", "БАЛКАНКАР АД - БУЛ. ЛОМСКО ШОСЕ - БАЛКАНКАР АД"],
    "A261": ["200", "ПЛ. СВ. АЛ. НЕВСКИ - ПЛ.СВ. АЛЕКСАНДЪР НЕВСКИ - ПЛ. СВ. АЛ. НЕВСКИ"],
    "A84": ["204", "Ж.К. ГОЦЕ ДЕЛЧЕВ - Ж.К. ДРУЖБА-2 - Ж.К. ГОЦЕ ДЕЛЧЕВ"],
    "A131": ["11TM", "МЕТРОСТАНЦИЯ ОВЧА КУПЕЛ - Автостанция Княжево - МЕТРОСТАНЦИЯ ОВЧА КУПЕЛ"],
    "A35": ["118", "Село Желява - Автостанция Изток - Село Желява"],
    "TB40": ["Д1", "Ж.к. Гоце Делчев - Бул. Братя Бъкстон - Ж.к. Гоце Делчев"],
    "TM27": ["1", "Ж.К. ИВАН ВАЗОВ - ПЕТА ГРАДСКА БОЛНИЦА - Ж.К. ИВАН ВАЗОВ"],
    "TB41": ["801", "Ж.К.ИВАН ВАЗОВ - Пл. Македония - Ж.К.ИВАН ВАЗОВ"],
    "A147": ["6TM", "Ж.К. ИВАН ВАЗОВ - ж.к. Лозенец - Ж.К. ИВАН ВАЗОВ"],
    "TB35": ["60", "БУЛ. КН. МАРИЯ ЛУИЗА - Ж.К. ОВЧА КУПЕЛ-2 - БУЛ. КН. МАРИЯ ЛУИЗА"],
    "TM49": ["6A", "Ж. к. Обеля-2 - МЕТРОСТАНЦИЯ ХАН КУБРАТ - Ж. к. Обеля-2"],
    "A43": ["22", "СЕЛО ПОДГУМЕР - ЦЕНТРАЛНА ГАРА - СЕЛО ПОДГУМЕР"],
    "TM30": ["7", "ПЕТА ГРАДСКА БОЛНИЦА - КВ. МАНАСТИРСКИ ЛИВАДИ-ЗАПАД - ПЕТА ГРАДСКА БОЛНИЦА"],
    "TM57": ["92", "ПЛ. ЖУРНАЛИСТ - ПЛ. ВЪЗРАЖДАНЕ - ПЛ. ЖУРНАЛИСТ"],
    "M1": ["M1", "Бизнес парк - Витоша - Бизнес парк"],
    "A243": ["Y2", "Ж.К. ЛЮЛИН-5 - 18 СУ УИЛЯМ ГЛАДСТОН - Ж.К. ЛЮЛИН-5"],
    "A3": ["111", "БУЛ. НИКОЛА ПЕТКОВ - Ж.К. МЛАДОСТ-1 - БУЛ. НИКОЛА ПЕТКОВ"],
    "A32": ["100", "БАЛКАНСТАР - ПЪТНОСТРОИТЕЛНА ТЕХНИКА-временна - БАЛКАНСТАР"],
    "A263": ["X1", "Ж.К. ГОЦЕ ДЕЛЧЕВ - Ж.К. ДРУЖБА-2 - Ж.К. ГОЦЕ ДЕЛЧЕВ"],
    "A257": ["49A", "АВТОСТАНЦИЯ БАНКЯ - МАНАСТИР СВ. ПЕТКА ПАРАСКЕВА - АВТОСТАНЦИЯ БАНКЯ"],
    "A2": ["9", "АВТОБАЗА ИСКЪР - Ул. Ген. Гурко - АВТОБАЗА ИСКЪР"],
    "TB24": ["123", "ЛИФТА СИМЕОНОВО - АВТОСТАНЦИЯ ГЕО МИЛЕВ - ЛИФТА СИМЕОНОВО"],
    "TB6": ["6", "Ж.к. Банишора - Ж.к. Гоце Делчев - Ж.к. Банишора"],
    "TB30": ["7А", "Ж.К. ЛЮЛИН-3 - УХО - Ж.К. ЛЮЛИН-3"],
    "A34": ["117", "АВТОСТАНЦИЯ ИЗТОК - ГРАД БУХОВО - АВТОСТАНЦИЯ ИЗТОК"],
    "A37": ["120", "Ж.К. ЛЕВСКИ-Г - ЗООПАРКА - Ж.К. ЛЕВСКИ-Г"],
    "TM23": ["8", "УЛ. ЦАР СИМЕОН - ж.к. Люлин-5 - УЛ. ЦАР СИМЕОН"],
    "TB44": ["6TM", "Ж.К.ИВАН ВАЗОВ - Пл. Македония - Ж.К.ИВАН ВАЗОВ"],
    "A96": ["90А", "Автостанция Изток - АП Малашевци - Автостанция Изток"],
    "A145": ["10ТМ", "МЕТРОСТАНЦИЯ ВИТОША - СУ СВ.КЛИМЕНТ ОХРИДСКИ - МЕТРОСТАНЦИЯ ВИТОША"],
    "A60": ["310", "Ж.К. ЛЮЛИН-5 - ПЛ. СТОЧНА ГАРА - Ж.К. ЛЮЛИН-5"],
    "M3": ["M3", "Хаджи Димитър - Горна баня - Хаджи Димитър"],
    "A52": ["78", "ЦЕНТРАЛНА ГАРА - ПЛОЩАДА КВ. ВРАЖДЕБНА - ЦЕНТРАЛНА ГАРА"],
    "TM40": ["15", "Ж.К. БЪКСТОН - ПЛ. ЖУРНАЛИСТ - Ж.К. БЪКСТОН"],
    "A40": ["20", "СЕЛО ЛОКОРСКО - ПЛ. ЛЪВОВ МОСТ - СЕЛО ЛОКОРСКО"],
    "A69": ["4", "КОКАЛЯНСКО ХАНЧЕ - Ж.К. МЛАДОСТ-1 - КОКАЛЯНСКО ХАНЧЕ"],
    "A101": ["42", "УЛ. ТЪРНОВО - КРАЯ НА КВ. МИХАЙЛОВО - УЛ. ТЪРНОВО"],
    "A58": ["280", "ВОЕННА АКАДЕМИЯ - СТУДЕНТСКИ ГРАД - ВОЕННА АКАДЕМИЯ"],
    "A8": ["77", "Ж.к. Западен парк - Централна гара - Ж.к. Западен парк"],
    "A98": ["54", "МЕТРОСТАНЦИЯ СЛИВНИЦА - ПЪТЕН ВЪЗЕЛ  КАЛОТИНА - МЕТРОСТАНЦИЯ СЛИВНИЦА"],
    "A167": ["294", "МЕТРОСТАНЦИЯ Г. М. ДИМИТРОВ - СТУДЕНТСКИ ГРАД - МЕТРОСТАНЦИЯ Г. М. ДИМИТРОВ"],
    "A49": ["28", "ЦЕНТЪР С. МРАМОР - СЕЛО ЛОКОРСКО - ЦЕНТЪР С. МРАМОР"],
    "TM24": ["10", "УЛ. ОПЪЛЧЕНСКА - Ж.К. ЗАПАДЕН ПАРК - УЛ. ОПЪЛЧЕНСКА"],
    "TB2": ["11", "ПЛОЩАД  НА АВИАЦИЯТА - Ж.К. ДРУЖБА -1 - ПЛОЩАД  НА АВИАЦИЯТА"],
    "A4": ["11", "УЛ. ДОБРОТИЧ - АВТОСТАНЦИЯ ГЕО МИЛЕВ - УЛ. ДОБРОТИЧ"],
    "A26": ["108", "Ж.К. ЛЮЛИН-5 - СТАДИОН ЛОКОМОТИВ - Ж.К. ЛЮЛИН-5"],
    "A220": ["184", "РАЗКЛОНА ЗА ЛЕТИЩЕТО - УЛ. ГЕН. ГУРКО - РАЗКЛОНА ЗА ЛЕТИЩЕТО"],
    "A91": ["604", "УМБАЛСМ ПИРОГОВ - ГАРА ИСКЪР - УМБАЛСМ ПИРОГОВ"],
    "A195": ["49", "МЕТРОСТАНЦИЯ СЛИВНИЦА - КВ. КЛИСУРА - МЕТРОСТАНЦИЯ СЛИВНИЦА"],
    "A208": ["11Tm", "КВ. ПАВЛОВО - Ж.К. ЗАПАДЕН ПАРК - КВ. ПАВЛОВО"],
    "A105": ["59", "С. МЪРЧАЕВО-ТОЛУМСКА МАХАЛА - МЕТРОСТАНЦИЯ МИЗИЯ - С. МЪРЧАЕВО-ТОЛУМСКА МАХАЛА"],
    "TB32": ["3", "ПЛОЩАД  НА АВИАЦИЯТА - Ж.К. ЛЕВСКИ Г - ПЛОЩАД  НА АВИАЦИЯТА"],
    "M2": ["M2", "Витоша - Бизнес парк - Витоша"],
    "A24": ["58", "ГАРА ВЛАДАЯ - АВТОСТАНЦИЯ КНЯЖЕВО - ГАРА ВЛАДАЯ"],
    "A6": ["64", "ЗООПАРКА - ЦЕНТЪР ПО ХИГИЕНА - ЗООПАРКА"],
    "TB7": ["7", "СУ СВ.КЛИМЕНТ ОХРИДСКИ - Ж.К. ЛЮЛИН-3 - СУ СВ.КЛИМЕНТ ОХРИДСКИ"],
    "A44": ["101", "ЦЕНТРАЛНА ГАРА - ФОНДОВИ ЖИЛИЩА - ЦЕНТРАЛНА ГАРА"],
    "TM28": ["3", "Кв. Орландовци - ЦЕНТРАЛНА ГАРА - Кв. Орландовци"],
    "A75": ["10", "КВ. Д. МИЛЕНКОВ - Кв.Бусманци - КВ. Д. МИЛЕНКОВ"],
    "A42": ["21", "СЕЛО ВОЙНЕГОВЦИ - ЦЕНТРАЛНА ГАРА - СЕЛО ВОЙНЕГОВЦИ"],
    "A225": ["N2", "СТУДЕНТСКИ ГРАД - КВ. ОБЕЛЯ - СТУДЕНТСКИ ГРАД"],
    "TM20": ["20", "Автостанция Изток - Депо Искър - Автостанция Изток"],
    "TM33": ["12", "ТРИЪГЪЛНИКА - Гара София север - ТРИЪГЪЛНИКА"],
    "A247": ["82A", "ИНЖСТРОЙ София АД - НАДЛЕЗ НАДЕЖДА - ИНЖСТРОЙ София АД"],
    "A9": ["83", "ХЮНДАЙ БЪЛГАРИЯ - ЗООПАРКА - ХЮНДАЙ БЪЛГАРИЯ"],
    "A59": ["309", "Ж.К. ЛЮЛИН-1,2 - БУЛ. ИЛИЯНЦИ-ПОДЛЕЗА - Ж.К. ЛЮЛИН-1,2"],
    "A72": ["7", "ГАРА ИСКЪР - ЖП ГАРА ВЕРИЛА - ГАРА ИСКЪР"],
    "A94": ["79А", "УЛ. ВАСИЛ КЪНЧЕВ - БУЛ. ИЛИЯНЦИ-ПОДЛЕЗА - УЛ. ВАСИЛ КЪНЧЕВ"],
    "A25": ["63", "Ж.к. Красно село - Бялата чешма - Ж.к. Красно село"],
    "A65": ["103", "ВЗ. ЛЮЛИН - АВТОСТАНЦИЯ ОВЧА КУПЕЛ - ВЗ. ЛЮЛИН"],
    "A56": ["86", "АП МАЛАШЕВЦИ - Ж.К. СВОБОДА - АП МАЛАШЕВЦИ"],
    "A93": ["18A", "ГРОБИЩЕН ПАРК МАЛАШЕВЦИ - АВТОСТАНЦИЯ ОРЛАНДОВЦИ - ГРОБИЩЕН ПАРК МАЛАШЕВЦИ"],
    "A54": ["82", "Ж.К. ЛЮЛИН-5 - ЦЕНТРАЛНА ГАРА - Ж.К. ЛЮЛИН-5"],
    "TB37": ["3TM", "БУЛ. КН. МАРИЯ ЛУИЗА - ПЛ. ВЪЗРАЖДАНЕ - БУЛ. КН. МАРИЯ ЛУИЗА"],
    "A53": ["79", "УЛ. ВАСИЛ КЪНЧЕВ - СТИНД - УЛ. ВАСИЛ КЪНЧЕВ"],
    "M4": ["M4", "Летище София - Витоша - Летище София"],
    "A168": ["7TM", "БУЛ. ЧЕРНИ ВРЪХ - НИМ - БУЛ. ЧЕРНИ ВРЪХ"],
    "TB27": ["1А", "ВМА - Кино Одеон - ВМА"],
    "A120": ["108A", "Ж.К. ЛЮЛИН -5 - БУЛ. ЛОМСКО ШОСЕ - Ж.К. ЛЮЛИН -5"],
    "A22": ["56", "Ж.К. ЗАПАДЕН ПАРК - ГАРА ЗАХАРНА ФАБРИКА - Ж.К. ЗАПАДЕН ПАРК"],
    "A106": ["68", "КВ. СИМЕОНОВО - ЗООПАРКА - КВ. СИМЕОНОВО"],
    "A254": ["803", "МЕТРОСТАНЦИЯ ГОРНА БАНЯ - КАНТОНА - МЕТРОСТАНЦИЯ ГОРНА БАНЯ"],
    "A240": ["6ТМ", "КВ. ОБЕЛЯ - ЦЕНТРАЛНА ГАРА - КВ. ОБЕЛЯ"],
    "TB26": ["184", "УЛ. ГЕН. ГУРКО - ДЪРЖАВНА ПЕЧАТНИЦА - УЛ. ГЕН. ГУРКО"],
    "A117": ["22ТМ", "БУЛ. ХРИСТО БОТЕВ - Ж.К. ЗАПАДЕН ПАРК - БУЛ. ХРИСТО БОТЕВ"],
    "TM25": ["5", "Ж.к. Бъкстон - Пл. Македония - Ж.к. Бъкстон"],
    "A224": ["N1", "Ж.К. МЛАДОСТ-4 - Ж.К. ЛЮЛИН-1,2 - Ж.К. МЛАДОСТ-4"],
    "TM55": ["27", "ЦЕНТРАЛНА ГАРА - Кв. Манастирски ливади-запад - ЦЕНТРАЛНА ГАРА"],
    "A215": ["110", "Ж.К. МЛАДОСТ-1 - Ж.К. БЪКСТОН - Ж.К. МЛАДОСТ-1"],
    "A63": ["66", "ЗООПАРКА - ХОТЕЛ МОРЕНИ - ЗООПАРКА"],
    "A55": ["85", "Ж.к.Хаджи Димитър - Ж.к. Връбница 2 - Ж.к.Хаджи Димитър"],
    "TM34": ["18", "пл. Гарибалди - пл. Журналист - пл. Гарибалди"],
    "A182": ["384", "Летище София Терминал 2 - Ж.к. Дружба 2 - Летище София Терминал 2"],
    "A227": ["N4", "Ж.К. ГОЦЕ ДЕЛЧЕВ - Ж.К. ДРУЖБА-2 - Ж.К. ГОЦЕ ДЕЛЧЕВ"],
    "A193": ["47", "МЕТРОСТАНЦИЯ СЛИВНИЦА - КВ. ГРАДОМАН - МЕТРОСТАНЦИЯ СЛИВНИЦА"],
    "A202": ["12Т", "Автостанция Илиянци - ГАРА СОФИЯ СЕВЕР - Автостанция Илиянци"],
    "A85": ["213", "БЛ. 480 Ж.К. МЛАДОСТ-4 - ЦЕНТРАЛНА ГАРА - БЛ. 480 Ж.К. МЛАДОСТ-4"],
    "A41": ["18", "АВТОСТАНЦИЯ ОРЛАНДОВЦИ - МЕЛНИЦАТА ЧЕПИНЦИ - АВТОСТАНЦИЯ ОРЛАНДОВЦИ"],
    "A205": ["505", "ПЛ. ОРЛОВ МОСТ - Парк музей Врана - ПЛ. ОРЛОВ МОСТ"],
    "A103": ["150", "Ж. К. ОБЕЛЯ 1 - ПЛОДОХРАНИЛИЩЕ - Ж. К. ОБЕЛЯ 1"],
    "A248": ["19", "Кв. Обеля - Ж.к. Обеля 1 - Кв. Обеля"],
    "TB3": ["5", "Надлез Надежда - ПЛОЩАД НА АВИАЦИЯТА - Надлез Надежда"],
    "A172": ["20TM", "МЕТРОСТАНЦИЯ ОПЪЛЧЕНСКА - ПЛ. ВАСИЛ ЛЕВСКИ - МЕТРОСТАНЦИЯ ОПЪЛЧЕНСКА"],
    "TB4": ["8", "Ж.к. Гоце Делчев - Кино Одеон - Ж.к. Гоце Делчев"],
    "A249": ["X50", "Парк музей Врана - Пл. Орлов мост - Парк музей Врана"],
    "A81": ["76", "УМБАЛ СВ. АННА  - Ж.К. ГОЦЕ ДЕЛЧЕВ - УМБАЛ СВ. АННА"],
    "A226": ["N3", "ПЛ. КНЯЗ АЛЕКСАНДЪР I - Ж.К. ОВЧА КУПЕЛ 2 - ПЛ. КНЯЗ АЛЕКСАНДЪР I"],
    "TB36": ["74", "БУЛ. КН. МАРИЯ ЛУИЗА - Ж.к. Гоце Делчев - БУЛ. КН. МАРИЯ ЛУИЗА"],
    "TM22": ["22", "Младежки театър - Автостанция Изток - Младежки театър"],
    "TM32": ["6", "НАДЛЕЗ НАДЕЖДА - Ж.к.Обеля 2 - НАДЛЕЗ НАДЕЖДА"],
    "TM35": ["7А", "КВ. ОРЛАНДОВЦИ - КВ. МАНАСТИРСКИ ЛИВАДИ-ЗАПАД - КВ. ОРЛАНДОВЦИ"],
    "A255": ["804", "МЕТРОСТАНЦИЯ ГОРНА БАНЯ - ПЛ. СРЕД СЕЛО - МЕТРОСТАНЦИЯ ГОРНА БАНЯ"],
    "A171": ["59A", "МЕТРОСТАНЦИЯ МИЗИЯ - Пл. Сред село - МЕТРОСТАНЦИЯ МИЗИЯ"],
    "A31": ["44", "КВ. ИВАНЯНЕ - АВТОСТАНЦИЯ БАНКЯ - КВ. ИВАНЯНЕ"],
    "A87": ["305", "ЦЕНТРАЛНА ГАРА - КУЛИНАРЕН К-Т ПЕЙФИЛ - ЦЕНТРАЛНА ГАРА"],
    "A95": ["86А", "УЛ. ОПЪЛЧЕНСКА - АП МАЛАШЕВЦИ - УЛ. ОПЪЛЧЕНСКА"],
    "A99": ["84", "ЛЕТИЩЕ СОФИЯ ТЕРМИНАЛ 2 - ВОЕННА АКАДЕМИЯ - ЛЕТИЩЕ СОФИЯ ТЕРМИНАЛ 2"],
    "A66": ["1", "СЕЛО КОКАЛЯНЕ - 83-ТО ОУ ПАНЧАРЕВО - СЕЛО КОКАЛЯНЕ"],
    "A38": ["12", "Автостанция Изток - Село Долни Богров - Автостанция Изток"],
    "TB9": ["2", "Кино Одеон - Ж.К. БЪКСТОН - Кино Одеон"],
    "A50": ["29", "Автостанция Илиянци - СЕЛО БАЛША - Автостанция Илиянци"],
    "A78": ["69", "ЦАРЕВА МАХАЛА - НСБАЛ ПО ОНКОЛОГИЯ - ЦАРЕВА МАХАЛА"],
    "A100": ["404", "ЦЕНТРАЛНА ГАРА - АП ДРУЖБА - ЦЕНТРАЛНА ГАРА"],
    "A90": ["413", "ПЛОЩАД ЦЕНТРАЛНА ГАРА - Технополис ж.к. Младост 4 - ПЛОЩАД ЦЕНТРАЛНА ГАРА"],
    "A252": ["X9", "МЕТРОСТАНЦИЯ АКАД. А. Т.- БАЛАН - Ж.К. ЛОЗЕНЕЦ - МЕТРОСТАНЦИЯ АКАД. А. Т.- БАЛАН"],
    "A36": ["119", "МК КРЕМИКОВЦИ - АВТОСТАНЦИЯ ИЗТОК - МК КРЕМИКОВЦИ"],
    "TB1": ["4", "ж.к. Хаджи Димитър - БУЛ. ПРОФ.ЦВ. ЛАЗАРОВ - ж.к. Хаджи Димитър"],
    "A200": ["314", "Село Бистрица - Ж.К. МЛАДОСТ 1 - Село Бистрица"],
    "TB42": ["22T", "ГАРА ПОДУЯНЕ - АВТОСТАНЦИЯ ИЗТОК - ГАРА ПОДУЯНЕ"],
    "A80": ["75", "АВТОСТАНЦИЯ ГЕО МИЛЕВ - ПЛ. ОРЛОВ МОСТ - АВТОСТАНЦИЯ ГЕО МИЛЕВ"],
    "A1": ["72", "Ж.к. Западен парк - Хотел Плиска - Ж.к. Западен парк"],
    "A77": ["67", "НСБАЛ ПО ОНКОЛОГИЯ - КВ. СИМЕОНОВО - НСБАЛ ПО ОНКОЛОГИЯ"],
    "A57": ["94", "Студентски град - СУ. СВ. КЛИМЕНТ ОХРИДСКИ - Студентски град"],
    "TB38": ["12T", "Бул. Княгиня Мария Луиза - АВТОСТАНЦИЯ ИЛИЯНЦИ - Бул. Княгиня Мария Луиза"],
    "A121": ["8TM", "Ж.К. ЗАПАДЕН ПАРК - Ж.К. ЛЮЛИН-5 - Ж.К. ЗАПАДЕН ПАРК"],
    "A68": ["3", "УЛ. ГЕН. ГУРКО - СЕЛО ДОЛНИ ПАСАРЕЛ - УЛ. ГЕН. ГУРКО"],
    "A29": ["107", "КВ. КАРПУЗИЦА - Боянска църква - КВ. КАРПУЗИЦА"],
    "TB39": ["73", "Ж.к. Младост 1 - БЛ.6, Ж.К. Младост 1 - Ж.к. Младост 1"],
    "A201": ["285", "Ж.К. ХАДЖИ ДИМИТЪР - Ж.К. ВРЪБНИЦА-2 - Ж.К. ХАДЖИ ДИМИТЪР"],
    "TM56": ["21", "Ж.К. ГЕО МИЛЕВ - Депо Искър - Ж.К. ГЕО МИЛЕВ"],
    "A82": ["88", "ЗООПАРКА - Ж.К. ДРУЖБА-2 - ЗООПАРКА"],
    "A46": ["24", "Автостанция Орландовци - Село Локорско - Автостанция Орландовци"],
    "TB25": ["84", "ЛЕТИЩЕ СОФИЯ ТЕРМИНАЛ 2 - Бл.3, ж.к. Дружба-1 - ЛЕТИЩЕ СОФИЯ ТЕРМИНАЛ 2"],
    "A109": ["30", "Ж.К. ОБЕЛЯ 1 - СЕЛО БАЛША - Ж.К. ОБЕЛЯ 1"]
}

# Obtain Bus Model Translate Dict
bus_models_ref = [
    "Karsan e-JEST",
    "Mercedes-Benz O345 Conecto I G",
    "MAN A23 Lion's City G NG313 CNG",
    "Yutong ZK6126HGA",
    "Mercedes-Benz Intouro II ME",
    "Mercedes-Benz Intouro II",
    "Mercedes-Benz Intouro II M",
    "BMC Procity II 12 CNG",
    "Higer KLQ6125GEV3 Chariot Ebus",
    "Mercedes-Benz Conecto II",
    "Mercedes-Benz O345 Conecto I C",
    "MAN A61 SG262 MANAŚ",
    "Ikarus 280.59",
    "Neoplan N4426/3 Centroliner",
    "MAN A39 Lion's City DD ND313",
    "BMC Belde 220 SLF",
    "Yutong E12",
    "Higer KLQ6832GEV",
    "MAN A60 SL232 MANAŚ"
]

# Obtain Bus Regs to Models Dict
bus_reg_to_model = {
    "1010": 1,
    "1011": 1,
    "1012": 1,
    "1013": 1,
    "1014": 1,
    "1015": 1,
    "1017": 1,
    "1020": 1,
    "1022": 1,
    "1023": 1,
    "1025": 1,
    "1028": 1,
    "1029": 1,
    "1030": 1,
    "1031": 1,
    "1033": 1,
    "1034": 1,
    "1035": 1,
    "1036": 1,
    "1038": 1,
    "1110": 2,
    "1114": 2,
    "1115": 2,
    "1117": 2,
    "1125": 2,
    "1126": 2,
    "1127": 2,
    "1130": 2,
    "1131": 2,
    "1136": 2,
    "1137": 2,
    "1138": 2,
    "1150": 3,
    "1151": 3,
    "1152": 3,
    "1153": 3,
    "1154": 3,
    "1155": 3,
    "1156": 3,
    "1157": 3,
    "1158": 3,
    "1159": 3,
    "1160": 3,
    "1161": 3,
    "1162": 3,
    "1163": 3,
    "1164": 3,
    "1165": 3,
    "1166": 3,
    "1167": 3,
    "1168": 3,
    "1169": 3,
    "1170": 3,
    "1171": 3,
    "1172": 3,
    "1173": 3,
    "1174": 3,
    "1175": 3,
    "1176": 3,
    "1178": 3,
    "1180": 3,
    "1181": 3,
    "1182": 3,
    "1183": 3,
    "1184": 3,
    "1185": 3,
    "1186": 3,
    "1187": 3,
    "1188": 3,
    "1189": 3,
    "1201": 4,
    "1202": 4,
    "1203": 4,
    "1204": 4,
    "1205": 4,
    "1206": 4,
    "1207": 4,
    "1208": 4,
    "1209": 4,
    "1212": 4,
    "1214": 4,
    "1215": 4,
    "1217": 4,
    "1220": 4,
    "1221": 4,
    "1222": 4,
    "1225": 4,
    "1226": 4,
    "1227": 4,
    "1228": 4,
    "1229": 4,
    "1230": 4,
    "1231": 4,
    "1301": 5,
    "1302": 5,
    "1304": 5,
    "1305": 5,
    "1306": 5,
    "1307": 5,
    "1308": 5,
    "1309": 5,
    "1310": 5,
    "1311": 6,
    "1312": 6,
    "1313": 6,
    "1314": 6,
    "1315": 6,
    "1316": 7,
    "1317": 7,
    "1318": 7,
    "1319": 7,
    "1320": 7,
    "1321": 7,
    "1401": 8,
    "1402": 8,
    "1403": 8,
    "1404": 8,
    "1405": 8,
    "1406": 8,
    "1407": 8,
    "1409": 8,
    "1410": 8,
    "1412": 8,
    "1413": 8,
    "1417": 8,
    "1420": 8,
    "1421": 8,
    "1422": 8,
    "1424": 8,
    "1426": 8,
    "1427": 8,
    "1601": 3,
    "1602": 3,
    "1603": 3,
    "1604": 3,
    "1605": 3,
    "1606": 3,
    "1607": 3,
    "1608": 3,
    "1609": 3,
    "1610": 3,
    "1611": 3,
    "1613": 3,
    "1614": 3,
    "1615": 3,
    "1616": 3,
    "1617": 3,
    "1618": 3,
    "1619": 3,
    "1620": 3,
    "1702": 9,
    "1703": 9,
    "1801": 10,
    "1803": 10,
    "1805": 10,
    "1807": 10,
    "1809": 10,
    "1819": 10,
    "1821": 10,
    "1823": 10,
    "1825": 10,
    "1829": 10,
    "1831": 10,
    "1833": 10,
    "1835": 10,
    "1837": 10,
    "1839": 10,
    "1841": 10,
    "1843": 10,
    "1845": 10,
    "1849": 10,
    "1851": 10,
    "1853": 10,
    "1855": 10,
    "1857": 10,
    "1859": 10,
    "1861": 10,
    "1863": 10,
    "1867": 10,
    "1869": 10,
    "1871": 10,
    "1921": 11,
    "1935": 11,
    "2000": 3,
    "2001": 3,
    "2002": 3,
    "2003": 3,
    "2005": 3,
    "2006": 3,
    "2007": 3,
    "2008": 3,
    "2010": 3,
    "2011": 3,
    "2013": 3,
    "2015": 3,
    "2016": 3,
    "2017": 3,
    "2018": 3,
    "2019": 3,
    "2020": 3,
    "2021": 3,
    "2022": 3,
    "2023": 3,
    "2024": 3,
    "2025": 3,
    "2027": 3,
    "2028": 3,
    "2029": 3,
    "2030": 3,
    "2031": 3,
    "2032": 3,
    "2033": 3,
    "2034": 3,
    "2035": 3,
    "2036": 3,
    "2037": 3,
    "2038": 3,
    "2039": 3,
    "2040": 3,
    "2041": 3,
    "2042": 3,
    "2043": 3,
    "2044": 3,
    "2045": 3,
    "2047": 4,
    "2048": 4,
    "2050": 4,
    "2051": 4,
    "2052": 4,
    "2053": 4,
    "2054": 4,
    "2055": 4,
    "2056": 4,
    "2057": 4,
    "2058": 4,
    "2059": 4,
    "2060": 4,
    "2061": 4,
    "2062": 4,
    "2063": 4,
    "2064": 4,
    "2065": 4,
    "2066": 4,
    "2067": 4,
    "2068": 4,
    "2069": 4,
    "2071": 4,
    "2072": 4,
    "2073": 4,
    "2074": 4,
    "2075": 4,
    "2076": 4,
    "2077": 4,
    "2078": 4,
    "2079": 4,
    "2081": 4,
    "2082": 4,
    "2083": 4,
    "2084": 4,
    "2085": 4,
    "2086": 4,
    "2087": 4,
    "2088": 4,
    "2089": 4,
    "2090": 4,
    "2140": 12,
    "2151": 12,
    "2153": 12,
    "2154": 12,
    "2155": 12,
    "2160": 12,
    "2161": 2,
    "2162": 2,
    "2163": 2,
    "2164": 2,
    "2165": 2,
    "2166": 2,
    "2167": 2,
    "2168": 2,
    "2171": 2,
    "2172": 2,
    "2300": 3,
    "2301": 3,
    "2302": 3,
    "2303": 3,
    "2304": 3,
    "2305": 3,
    "2306": 3,
    "2307": 3,
    "2308": 3,
    "2309": 3,
    "2310": 3,
    "2311": 3,
    "2312": 3,
    "2313": 3,
    "2314": 3,
    "2315": 3,
    "2316": 3,
    "2317": 3,
    "2318": 3,
    "2319": 3,
    "2500": 8,
    "2501": 8,
    "2501": 1,
    "2502": 8,
    "2502": 1,
    "2503": 8,
    "2503": 1,
    "2504": 8,
    "2504": 1,
    "2505": 8,
    "2505": 1,
    "2506": 8,
    "2507": 8,
    "2509": 8,
    "2510": 8,
    "2511": 8,
    "2512": 8,
    "2513": 8,
    "2514": 8,
    "2515": 8,
    "2516": 8,
    "2517": 8,
    "2518": 8,
    "2519": 8,
    "2600": 13,
    "2601": 14,
    "2602": 15,
    "2603": 15,
    "2604": 15,
    "2605": 15,
    "2720": 16,
    "2722": 16,
    "2723": 16,
    "2727": 16,
    "2728": 16,
    "2733": 16,
    "2736": 16,
    "2752": 16,
    "2800": 17,
    "2801": 17,
    "2802": 17,
    "2803": 17,
    "2804": 17,
    "2805": 17,
    "2806": 17,
    "2807": 17,
    "2808": 17,
    "2809": 17,
    "2810": 17,
    "2811": 18,
    "2812": 18,
    "2813": 18,
    "2814": 18,
    "2815": 18,
    "2816": 18,
    "2817": 18,
    "2818": 18,
    "2819": 18,
    "2820": 18,
    "2821": 18,
    "2822": 18,
    "2823": 18,
    "2824": 18,
    "2825": 18,
    "2826": 18,
    "2827": 18,
    "2828": 18,
    "2829": 18,
    "2830": 18,
    "2831": 18,
    "2832": 18,
    "3008": 19,
    "3011": 17,
    "3012": 17,
    "3013": 17,
    "3014": 17,
    "3015": 17,
    "3016": 17,
    "3017": 17,
    "3018": 17,
    "3019": 17,
    "3100": 3,
    "3102": 3,
    "3103": 3,
    "3104": 3,
    "3105": 3,
    "3106": 3,
    "3107": 3,
    "3108": 3,
    "3109": 3,
    "3110": 3,
    "3111": 3,
    "3112": 3,
    "3113": 3,
    "3114": 3,
    "3115": 3,
    "3116": 3,
    "3117": 3,
    "3118": 3,
    "3119": 3,
    "3120": 3,
    "3121": 3,
    "3122": 3,
    "3123": 3,
    "3124": 3,
    "3125": 3,
    "3126": 3,
    "3127": 3,
    "3128": 3,
    "3129": 3,
    "3130": 3,
    "3131": 3,
    "3132": 3,
    "3133": 3,
    "3134": 3,
    "3135": 3,
    "3136": 3,
    "3137": 3,
    "3138": 3,
    "3139": 3,
    "3140": 3,
    "3141": 3,
    "3142": 3,
    "3143": 3,
    "3144": 3,
    "3145": 3,
    "3146": 3,
    "3147": 3,
    "3148": 3,
    "3149": 3,
    "3150": 3,
    "3151": 3,
    "3152": 3,
    "3153": 3,
    "3154": 3,
    "3155": 3,
    "3156": 3,
    "3157": 3,
    "3158": 3,
    "3159": 3,
    "3301": 2,
    "3302": 2,
    "3304": 2,
    "3307": 2,
    "3309": 2,
    "3310": 2,
    "3311": 2,
    "3312": 2,
    "3313": 2,
    "3314": 2,
    "3315": 2,
    "3316": 2,
    "3317": 2,
    "3318": 2,
    "3319": 2,
    "3320": 2,
    "3321": 2,
    "3322": 2,
    "3400": 8,
    "3401": 8,
    "3402": 8,
    "3403": 8,
    "3404": 8,
    "3405": 8,
    "3406": 8,
    "3407": 8,
    "3408": 8,
    "3409": 8,
    "3410": 8,
    "3411": 8,
    "3600": 4,
    "3601": 4,
    "3603": 4,
    "3605": 4,
    "3606": 4,
    "3607": 4,
    "3608": 4,
    "3609": 4,
    "3610": 4,
    "3611": 4,
    "3612": 4,
    "3613": 4,
    "3614": 4,
    "3615": 4,
    "3616": 4,
    "3617": 4,
    "3618": 4,
    "3620": 4,
    "3621": 4,
    "3622": 4,
    "3623": 4,
    "3625": 4,
    "3626": 4,
    "3627": 4,
    "3628": 4,
    "3629": 4,
    "3630": 4,
    "3631": 4,
    "3632": 4,
    "3633": 4,
    "3634": 4,
    "3636": 4,
    "3637": 4,
    "3651": 4,
    "3653": 4,
    "3654": 4,
    "3655": 4,
    "3656": 4,
    "3657": 4,
    "3658": 4,
    "3659": 4,
    "3660": 4,
    "3661": 4,
    "3662": 4,
    "3663": 4,
    "3665": 4,
    "3666": 4,
    "3667": 4,
    "3668": 4,
    "3669": 4,
    "3670": 4,
    "3671": 4,
    "3700": 16,
    "3701": 16,
    "3704": 16,
    "3707": 16,
    "3715": 16,
    "3812": 16,
    "5001": 9,
    "5002": 9,
    "5003": 9,
    "5004": 9,
    "5005": 9,
    "5006": 9,
    "5007": 9,
    "5008": 9,
    "5009": 9,
    "5010": 9,
    "5011": 9,
    "5012": 9,
    "5013": 9,
    "5014": 9,
    "5015": 9,
    "5031": 9,
    "5032": 9,
    "5033": 9,
    "5034": 9,
    "5035": 9,
    "5036": 9,
    "5037": 9,
    "5038": 9,
    "5039": 9,
    "5040": 9,
    "5041": 9,
    "5042": 9,
    "5043": 9,
    "5044": 9,
    "5045": 9,
    "5046": 9,
    "5047": 9,
    "5048": 9,
    "5049": 9,
    "5050": 9,
    "5051": 9,
    "5052": 9,
    "5053": 9,
    "5054": 9,
    "5055": 9,
    "5056": 9,
    "5057": 9,
    "5058": 9,
    "5059": 9,
    "5060": 9,
    "5061": 9,
    "5062": 9,
    "5063": 9,
    "5064": 9,
    "А-8": 19
}

# Obtain Tram Model Translate Dict
tram_models_ref = [
    "PESA Swing 122NaSF",
    "PESA Swing 122NaSF2",
    "PESA Swing 122NaSF2-3",
    "Inekon T8M-700 IT",
    "Tatra T4D-M2",
    "Tatra T6B5B",
    "Tatra T6A2B",
    "Tatra T6A5",
    "Tatra T6A2SF",
    "Plasser & Theurer Beaver 4-73",
    "2-axle trailer car",
    "Graz/Siemens & Halske 2-axle motor car",
    "MAN/Siemens",
    "Duewag GT8",
    "T4M-50",
    "T4M-900",
    "Т4М-81",
    "Schindler/Siemens Be 4/6 S",
    "Kardalev",
    "Т8М-500 F",
    "T6M-700 F",
    "T8M-900 F"
]

# Obtain Tram Regs to Models Dict
tram_reg_to_model = {
    "2301": 1,
    "2321": 2,
    "2302": 1,
    "2322": 2,
    "2303": 1,
    "2323": 2,
    "2304": 1,
    "2324": 2,
    "2305": 1,
    "2325": 2,
    "2306": 1,
    "2326": 2,
    "2307": 1,
    "2327": 2,
    "2328": 2,
    "2309": 1,
    "2329": 2,
    "2310": 1,
    "2330": 2,
    "2311": 1,
    "2331": 2,
    "2312": 1,
    "2332": 2,
    "2313": 1,
    "2333": 2,
    "2314": 1,
    "2334": 2,
    "2315": 1,
    "2335": 2,
    "2316": 1,
    "2336": 2,
    "2317": 1,
    "2337": 2,
    "2318": 1,
    "2338": 2,
    "2319": 1,
    "2339": 3,
    "2320": 1,
    "2340": 3,
    "2341": 3,
    "2342": 3,
    "2343": 3,
    "2344": 3,
    "2345": 3,
    "2346": 3,
    "2347": 3,
    "2348": 3,
    "2349": 3,
    "2350": 3,
    "2351": 3,
    "2352": 3,
    "2353": 3,
    "2354": 3,
    "2355": 3,
    "3401": 4,
    "2356": 3,
    "2357": 3,
    "3403": 4,
    "2358": 3,
    "2359": 3,
    "3405": 4,
    "2360": 3,
    "3406": 4,
    "2361": 3,
    "2362": 3,
    "3408": 4,
    "2363": 3,
    "3409": 4,
    "2364": 3,
    "3410": 4,
    "2365": 3,
    "3411": 4,
    "2366": 3,
    "3412": 4,
    "2367": 3,
    "3413": 4,
    "3414": 4,
    "3415": 4,
    "3416": 4,
    "3417": 4,
    "3418": 4,
    "4080": 5,
    "4102": 6,
    "4101": 6,
    "4103": 6,
    "4104": 6,
    "4105": 6,
    "4107": 6,
    "4106": 6,
    "4108": 6,
    "4109": 6,
    "4110": 6,
    "4111": 6,
    "4112": 6,
    "4113": 6,
    "4114": 6,
    "4115": 6,
    "4116": 6,
    "4118": 6,
    "4119": 6,
    "4120": 6,
    "4122": 6,
    "4124": 6,
    "4125": 6,
    "4126": 6,
    "4127": 6,
    "4128": 6,
    "4129": 6,
    "4130": 6,
    "4131": 6,
    "4132": 6,
    "4133": 6,
    "4134": 6,
    "4135": 6,
    "4136": 6,
    "4137": 6,
    "3001": 7,
    "3002": 7,
    "3003": 7,
    "3004": 7,
    "3005": 7,
    "3006": 7,
    "3007": 7,
    "3008": 7,
    "3009": 7,
    "3010": 7,
    "3011": 7,
    "3012": 7,
    "3013": 7,
    "3014": 7,
    "3015": 7,
    "3016": 7,
    "3017": 7,
    "3018": 7,
    "3020": 7,
    "3021": 7,
    "3022": 7,
    "3023": 7,
    "3024": 7,
    "3025": 7,
    "3027": 7,
    "3028": 7,
    "3029": 7,
    "3030": 7,
    "3031": 7,
    "3032": 7,
    "2033": 7,
    "2034": 7,
    "3035": 7,
    "3036": 7,
    "3037": 7,
    "3038": 7,
    "3039": 7,
    "3040": 7,
    "4193": 8,
    "4163": 8,
    "4152": 8,
    "4146": 8,
    "4148": 8,
    "4160": 8,
    "4149": 8,
    "4164": 8,
    "4165": 8,
    "4143": 8,
    "4189": 8,
    "4184": 8,
    "4144": 8,
    "4140": 8,
    "4180": 8,
    "4147": 8,
    "4192": 8,
    "4191": 8,
    "4155": 8,
    "4151": 8,
    "4187": 8,
    "4188": 8,
    "4185": 8,
    "4145": 8,
    "4150": 8,
    "4142": 8,
    "4156": 8,
    "4153": 8,
    "4154": 8,
    "4157": 8,
    "4141": 8,
    "4181": 8,
    "4190": 8,
    "4161": 8,
    "4186": 8,
    "4162": 8,
    "4175": 8,
    "4166": 8,
    "4158": 8,
    "4167": 8,
    "4178": 8,
    "4159": 8,
    "4179": 8,
    "4170": 8,
    "4171": 8,
    "4172": 8,
    "4183": 8,
    "4182": 8,
    "4173": 8,
    "4174": 8,
    "2041": 9,
    "2042": 9,
    "2043": 9,
    "2044": 9,
    "2045": 9,
    "2046": 9,
    "2047": 9,
    "2048": 9,
    "2049": 9,
    "2050": 9,
    "2051": 9,
    "2052": 9,
    "2053": 9,
    "2054": 9,
    "2055": 9,
    "2056": 9,
    "2057": 9,
    "РТ": 10,
    "СЛ": 11,
    "ХД 1": 11,
    "РВ": 11,
    "35": 12,
    "92": 13,
    "4401": 14,
    "4410": 14,
    "4411": 14,
    "4415": 14,
    "80": 15,
    "88": 16,
    "81": 17,
    "82": 15,
    "84": 15,
    "85": 15,
    "681": 18,
    "660": 18,
    "662": 18,
    "665": 18,
    "664": 18,
    "667": 18,
    "668": 18,
    "669": 18,
    "670": 18,
    "671": 18,
    "672": 18,
    "674": 18,
    "675": 18,
    "677": 18,
    "679": 18,
    "684": 18,
    "683": 18,
    "501": 19,
    "503": 20,
    "504": 20,
    "505": 20,
    "802": 21,
    "506": 20,
    "507": 20,
    "508": 20,
    "933": 22,
    "916": 22,
    "509": 20,
    "708": 21,
    "510": 20,
    "511": 20,
    "932": 22,
    "934": 22,
    "929": 22,
    "2301": 1,
    "2321": 2,
    "2302": 1,
    "2322": 2,
    "2303": 1,
    "2323": 2,
    "2304": 1,
    "2324": 2,
    "2305": 1,
    "2325": 2,
    "2306": 1,
    "2326": 2,
    "2307": 1,
    "2327": 2,
    "2328": 2,
    "2309": 1,
    "2329": 2,
    "2310": 1,
    "2330": 2,
    "2311": 1,
    "2331": 2,
    "2312": 1,
    "2332": 2,
    "2313": 1,
    "2333": 2,
    "2314": 1,
    "2334": 2,
    "2315": 1,
    "2335": 2,
    "2316": 1,
    "2336": 2,
    "2317": 1,
    "2337": 2,
    "2318": 1,
    "2338": 2,
    "2319": 1,
    "2339": 3,
    "2320": 1,
    "2340": 3,
    "2341": 3,
    "2342": 3,
    "2343": 3,
    "2344": 3,
    "2345": 3,
    "2346": 3,
    "2347": 3,
    "2348": 3,
    "2349": 3,
    "2350": 3,
    "2351": 3,
    "2352": 3,
    "2353": 3,
    "2354": 3,
    "2355": 3,
    "3401": 4,
    "2356": 3,
    "2357": 3,
    "3403": 4,
    "2358": 3,
    "2359": 3,
    "3405": 4,
    "2360": 3,
    "3406": 4,
    "2361": 3,
    "2362": 3,
    "3408": 4,
    "2363": 3,
    "3409": 4,
    "2364": 3,
    "3410": 4,
    "2365": 3,
    "3411": 4,
    "2366": 3,
    "3412": 4,
    "2367": 3,
    "3413": 4,
    "3414": 4,
    "3415": 4,
    "3416": 4,
    "3417": 4,
    "3418": 4,
    "4080": 5,
    "4102": 6,
    "4101": 6,
    "4103": 6,
    "4104": 6,
    "4105": 6,
    "4107": 6,
    "4106": 6,
    "4108": 6,
    "4109": 6,
    "4110": 6,
    "4111": 6,
    "4112": 6,
    "4113": 6,
    "4114": 6,
    "4115": 6,
    "4116": 6,
    "4118": 6,
    "4119": 6,
    "4120": 6,
    "4122": 6,
    "4124": 6,
    "4125": 6,
    "4126": 6,
    "4127": 6,
    "4128": 6,
    "4129": 6,
    "4130": 6,
    "4131": 6,
    "4132": 6,
    "4133": 6,
    "4134": 6,
    "4135": 6,
    "4136": 6,
    "4137": 6,
    "3001": 7,
    "3002": 7,
    "3003": 7,
    "3004": 7,
    "3005": 7,
    "3006": 7,
    "3007": 7,
    "3008": 7,
    "3009": 7,
    "3010": 7,
    "3011": 7,
    "3012": 7,
    "3013": 7,
    "3014": 7,
    "3015": 7,
    "3016": 7,
    "3017": 7,
    "3018": 7,
    "3020": 7,
    "3021": 7,
    "3022": 7,
    "3023": 7,
    "3024": 7,
    "3025": 7,
    "3027": 7,
    "3028": 7,
    "3029": 7,
    "3030": 7,
    "3031": 7,
    "3032": 7,
    "2033": 7,
    "2034": 7,
    "3035": 7,
    "3036": 7,
    "3037": 7,
    "3038": 7,
    "3039": 7,
    "3040": 7,
    "4193": 8,
    "4163": 8,
    "4152": 8,
    "4146": 8,
    "4148": 8,
    "4160": 8,
    "4149": 8,
    "4164": 8,
    "4165": 8,
    "4143": 8,
    "4189": 8,
    "4184": 8,
    "4144": 8,
    "4140": 8,
    "4180": 8,
    "4147": 8,
    "4192": 8,
    "4191": 8,
    "4155": 8,
    "4151": 8,
    "4187": 8,
    "4188": 8,
    "4185": 8,
    "4145": 8,
    "4150": 8,
    "4142": 8,
    "4156": 8,
    "4153": 8,
    "4154": 8,
    "4157": 8,
    "4141": 8,
    "4181": 8,
    "4190": 8,
    "4161": 8,
    "4186": 8,
    "4162": 8,
    "4175": 8,
    "4166": 8,
    "4158": 8,
    "4167": 8,
    "4178": 8,
    "4159": 8,
    "4179": 8,
    "4170": 8,
    "4171": 8,
    "4172": 8,
    "4183": 8,
    "4182": 8,
    "4173": 8,
    "4174": 8,
    "2041": 9,
    "2042": 9,
    "2043": 9,
    "2044": 9,
    "2045": 9,
    "2046": 9,
    "2047": 9,
    "2048": 9,
    "2049": 9,
    "2050": 9,
    "2051": 9,
    "2052": 9,
    "2053": 9,
    "2054": 9,
    "2055": 9,
    "2056": 9,
    "2057": 9,
    "РТ": 10,
    "СЛ": 11,
    "ХД 1": 11,
    "РВ": 11,
    "35": 12,
    "92": 13,
    "4401": 14,
    "4410": 14,
    "4411": 14,
    "4415": 14,
    "80": 15,
    "88": 16,
    "81": 17,
    "82": 15,
    "84": 15,
    "85": 15,
    "681": 18,
    "660": 18,
    "662": 18,
    "665": 18,
    "664": 18,
    "667": 18,
    "668": 18,
    "669": 18,
    "670": 18,
    "671": 18,
    "672": 18,
    "674": 18,
    "675": 18,
    "677": 18,
    "679": 18,
    "684": 18,
    "683": 18,
    "501": 19,
    "503": 20,
    "504": 20,
    "505": 20,
    "802": 21,
    "506": 20,
    "507": 20,
    "508": 20,
    "933": 22,
    "916": 22,
    "509": 20,
    "708": 21,
    "510": 20,
    "511": 20,
    "932": 22,
    "934": 22,
    "929": 22
}

# Obtain Trolleybus Model Translate Dict 
trolley_models_ref = [
    "Ikarus 280.92",
    "Škoda 26Tr Solaris III",
    "Škoda 27Tr Solaris III",
    "Škoda 27Tr Solaris IV",
    "Higer KLQ6832GEV",
    "Yutong E12 (ZK6128BEVG)",
    "Karsan e-Jest",
    "Higer KLQ6125GEV3"
]

# Obtain Trolleybus Regs to Models Dict 
trolley_reg_to_model = {
    "2123": 1,
    "2913": 1,
    "1603": 2,
    "1604": 2,
    "1605": 2,
    "1606": 2,
    "1607": 2,
    "1608": 2,
    "1609": 2,
    "1610": 2,
    "1611": 2,
    "1612": 2,
    "1613": 2,
    "1614": 2,
    "1615": 2,
    "1616": 2,
    "1617": 2,
    "1618": 2,
    "1619": 2,
    "1620": 2,
    "1621": 2,
    "1622": 2,
    "1623": 2,
    "1624": 2,
    "1625": 2,
    "1626": 2,
    "1627": 2,
    "1628": 2,
    "1629": 2,
    "1630": 2,
    "1631": 2,
    "1632": 2,
    "1650": 3,
    "1651": 3,
    "1652": 3,
    "1653": 3,
    "1654": 3,
    "1655": 3,
    "1656": 3,
    "1657": 3,
    "1658": 3,
    "1667": 3,
    "1666": 3,
    "1663": 3,
    "1659": 3,
    "1661": 3,
    "1662": 3,
    "1660": 3,
    "1664": 3,
    "1665": 3,
    "1668": 3,
    "1673": 3,
    "1669": 3,
    "1670": 3,
    "1671": 3,
    "1674": 3,
    "1672": 3,
    "2677": 3,
    "2678": 3,
    "2679": 3,
    "2676": 3,
    "2680": 3,
    "2681": 3,
    "2687": 3,
    "2682": 3,
    "2686": 3,
    "2684": 3,
    "2685": 3,
    "2688": 3,
    "2698": 3,
    "2689": 3,
    "2691": 3,
    "2694": 3,
    "2697": 3,
    "2695": 3,
    "2683": 3,
    "2692": 3,
    "2696": 3,
    "2699": 3,
    "2693": 3,
    "2803": 4,
    "2825": 4,
    "2828": 4,
    "2703": 1,
    "2915": 1,
    "2702": 1,
    "2802": 4,
    "2805": 4,
    "2807": 4,
    "2809": 4,
    "2808": 4,
    "2804": 4,
    "2806": 4,
    "2810": 4,
    "2811": 4,
    "2814": 4,
    "2815": 4,
    "2816": 4,
    "2818": 4,
    "2813": 4,
    "2812": 4,
    "2817": 4,
    "2819": 4,
    "2823": 4,
    "2829": 4,
    "2821": 4,
    "2822": 4,
    "2824": 4,
    "2820": 4,
    "2827": 4,
    "2830": 4,
    "2811": 5,
    "2812": 5,
    "2813": 5,
    "2814": 5,
    "2815": 5,
    "2816": 5,
    "2817": 5,
    "2818": 5,
    "2819": 5,
    "2820": 5,
    "2821": 5,
    "2822": 5,
    "2823": 5,
    "2824": 5,
    "2825": 5,
    "2826": 5,
    "2827": 5,
    "2828": 5,
    "2829": 5,
    "2830": 5,
    "2831": 5,
    "2832": 5,
    "2810": 6,
    "2503": 7,
    "2502": 7,
    "2501": 7,
    "2505": 7,
    "2504": 7,
    "1010": 7,
    "1011": 7,
    "1012": 7,
    "1013": 7,
    "1014": 7,
    "1015": 7,
    "1018": 7,
    "1016": 7,
    "1017": 7,
    "1019": 7,
    "1026": 7,
    "1021": 7,
    "1022": 7,
    "1023": 7,
    "1028": 7,
    "1024": 7,
    "1025": 7,
    "1020": 7,
    "1029": 7,
    "1032": 7,
    "1033": 7,
    "1034": 7,
    "1027": 7,
    "1030": 7,
    "1031": 7,
    "1035": 7,
    "1036": 7,
    "1037": 7,
    "1038": 7,
    "1039": 7,
    "3011": 6,
    "3012": 6,
    "3013": 6,
    "3014": 6,
    "3015": 6,
    "3016": 6,
    "3017": 6,
    "3018": 6,
    "2800": 6,
    "2801": 6,
    "2802": 6,
    "2803": 6,
    "2804": 6,
    "2805": 6,
    "2806": 6,
    "2807": 6,
    "2808": 6,
    "2809": 6,
    "1702": 8,
    "1703": 8,
    "5001": 8,
    "5002": 8,
    "5003": 8,
    "5004": 8,
    "5005": 8,
    "5006": 8,
    "5007": 8,
    "5008": 8,
    "5009": 8,
    "5010": 8,
    "5011": 8,
    "5012": 8,
    "5013": 8,
    "5014": 8,
    "5015": 8,
    "5031": 8,
    "5032": 8,
    "5033": 8,
    "5034": 8,
    "5035": 8,
    "5036": 8,
    "5037": 8,
    "5038": 8,
    "5039": 8,
    "5040": 8,
    "5041": 8,
    "5042": 8,
    "5043": 8,
    "5044": 8,
    "5045": 8,
    "5046": 8,
    "5047": 8,
    "5049": 8,
    "5050": 8,
    "5051": 8,
    "5052": 8,
    "5053": 8,
    "5054": 8,
    "5055": 8,
    "5056": 8,
    "5057": 8,
    "5058": 8,
    "5059": 8,
    "5060": 8,
    "5061": 8,
    "5062": 8,
    "5063": 8,
    "5064": 8
}

# Combine Regs to Models
reg_to_model = {
    'A': bus_reg_to_model,
    'TB': trolley_reg_to_model,
    'TM': tram_reg_to_model
}

# Combine Models Translate Dicts
models_ref = {
    'A': bus_models_ref,
    'TB': trolley_models_ref,
    'TM': tram_models_ref
}

# Obtain stops Dict
stops = {
    "A0328": "БУЛ. К. ВЕЛИЧКОВ",
    "A2545": "УЛ. ЛЕСНОВСКА РЕКА",
    "A0778": "УЛ. ИНЖ. ГЕОРГИ БЕЛОВ",
    "A1351": "УЛ. БРЕЗИТЕ",
    "A2189": "УЛ. СТОИЛ ВОЙВОДА",
    "A0379": "ЖЕНСКИ ПАЗАР",
    "A1714": "МОСТА НА СУХАТА РЕКА",
    "A1166": "10-ТИ ДКЦ",
    "TB1874": "УЛ. ВИТИНЯ",
    "A2161": "УЛ. РИЛСКА ОБИТЕЛ",
    "TB2418": "БУЛ. ЦАРИЦА ЙОАНА",
    "A6772": "УЛ. ДЖЕРМАН - ВРЕМЕННА",
    "TM1116": "НАЦИОНАЛНА ОПЕРА",
    "A2301": "ХАНЧЕТО",
    "A6394": "ТЕХНИЧЕСКИ УНИВЕРСИТЕТ",
    "A0378": "ЖЕНСКИ ПАЗАР",
    "A2270": "129-ТО ОУ",
    "A1557": "СЕЛО СВЕТОВРАЧАНЕ",
    "A2280": "61-ВО ОУ",
    "A2516": "УЛ. БАБА ТОНКА",
    "A0464": "ГАРА ПИОНЕР",
    "A0830": "КВ. ГОРУБЛЯНЕ",
    "A0314": "БУЛ. ГОЦЕ ДЕЛЧЕВ",
    "A6473": "ХЛАДИЛЕН ЗАВОД",
    "A6469": "БЪЛГАРСКО НАЦИОНАЛНО РАДИО",
    "TB1157": "Ул. Гюешево",
    "TB1892": "УЛ. ГЕН. ВЛ. ДИНЧЕВ",
    "A0179": "БЛ. 28 Ж.К. ЛЕВСКИ-Г",
    "TB1964": "УЛ. ИСКЪР",
    "TB0311": "БУЛ. ГОЦЕ ДЕЛЧЕВ",
    "A6482": "ОКОЛОВРЪСТЕН ПЪТ",
    "TB6717": "УЛ. КАМЕН АНДРЕЕВ",
    "A6745": "ГАРА ПОДУЯНЕ",
    "A0523": "УЛ. ЛАГАДИНА",
    "TB1911": "УЛ. ГОЦЕ ДЕЛЧЕВ",
    "M18": "МЛАДОСТ 3",
    "A0770": "ИНСТИТУТ ПО ЧЕРНА МЕТАЛУРГИЯ - ПО ЖЕЛАНИЕ",
    "A2815": "ПАЗАРА КРАСНО СЕЛО",
    "TM0018": "24-ТИ ДКЦ",
    "A1824": "УЛ. АЛЕКСАНДЪР ЖЕНДОВ",
    "A1058": "МЕТРОСТАНЦИЯ ОПЪЛЧЕНСКА",
    "A0876": "КВ. ОВЧА КУПЕЛ",
    "A1125": "НАЧАЛО КВ. ГНИЛЯНЕ",
    "A6549": "УЛ.ИНДИРА ГАНДИ",
    "TB1302": "ПЛ. РУЧЕЙ",
    "TB1395": "БЪЛГАРСКА ТЕЛЕГРАФНА АГЕНЦИЯ",
    "TM0720": "НУ ЗА ТАНЦОВО ИЗКУСТВО",
    "A0834": "КВ. ДРАГАЛЕВЦИ",
    "TB2419": "БУЛ. ЦАРИЦА ЙОАНА",
    "A2231": "УЛ. ХАН КУБРАТ",
    "A2613": "МАНАСТИР СВ. ТЕОДОР СТРАТИЛАТ - ПО ЖЕЛАНИЕ",
    "A1102": "МОСТА ЧАВДАР",
    "A2659": "МЕТРОСТАНЦИЯ ЛОМСКО ШОСЕ",
    "TB0136": "БЛ. 135 Ж.К. ЛЮЛИН-10",
    "TM2085": "УЛ. ОПЪЛЧЕНСКА",
    "TB0138": "БЛ. 135 Ж.К. ЛЮЛИН-10",
    "A0918": "МЕСТНОСТ МАНИЛОВ ДОЛ",
    "A1496": "РУСКО ПОСОЛСТВО",
    "A0505": "ГРЕБНА БАЗА - ПО ЖЕЛАНИЕ",
    "A1760": "115-ТИ ПОЩЕНСКИ КЛОН",
    "TM2299": "ТЪРГОВСКИ К-С НАДЕЖДА",
    "A2198": "УЛ. СУХОДОЛСКИ ПЪТ - ПО ЖЕЛАНИЕ",
    "TM0647": "ПК СПАРТАК",
    "TB2849": "УЛ. ЧАВДАР МУТАФОВ",
    "A2857": "МЕТРОСТАНЦИЯ ВИТОША",
    "A2782": "УЛ. МОГИЛАТА",
    "A0018": "24-ТИ ДКЦ",
    "TB0912": "МЕТРОСТАНЦИЯ ВИТОША",
    "A6771": "УЛ. ХАЙДУТ СИДЕР",
    "A6753": "ЦЕНТРАЛНИ ХАЛИ",
    "TB1912": "УЛ. ГОЦЕ ДЕЛЧЕВ",
    "A2335": "ЦЕНТРАЛНИ ХАЛИ",
    "A0690": "Ж.К. СЛАТИНА",
    "A1897": "УЛ. ГЕН. НИКОЛА ГЕНЕВ",
    "A0020": "26-ТИ ДКЦ",
    "A2146": "УЛ. ПРОФ. ВАЙГАНД",
    "A0395": "БУЛ. ЧЕРНИ ВРЪХ",
    "A1827": "УЛ. ЛАЗАРОПОЛЕ",
    "A1437": "РАБОТИЛНИЦАТА",
    "A6247": "БЛ. 541 Ж.К. ЛЮЛИН 5",
    "TB2578": "102-РО ОУ",
    "TB0927": "КИНО ОДЕОН",
    "A0459": "ГАРА КУРИЛО - ПО ЖЕЛАНИЕ",
    "TM1717": "ОБЩИНА ЛЮЛИН",
    "TM1948": "УЛ. ЗОГРАФСКИ МАНАСТИР",
    "A0764": "НСБАЛ ПО ОНКОЛОГИЯ",
    "A6589": "СЕЛИМИЦА",
    "A6725": "БЛ. 480 Ж.К. МЛАДОСТ-4",
    "A1740": "ТЕХНИЧЕСКИ УНИВЕРСИТЕТ",
    "TB1986": "УЛ. КАП. Д. СПИСАРЕВСКИ",
    "A1959": "УЛ. ИЛИЯ БЕШКОВ",
    "A2483": "ПЪТНОСТРОИТЕЛНА ТЕХНИКА",
    "A1037": "СТАДИОНА ГЕРМАН",
    "A2034": "УЛ. ЛУИ АЙЕР",
    "A2374": "МЕТРОСТАНЦИЯ БИЗНЕС ПАРК",
    "A2620": "УЛ. КРУШАТА - ПО ЖЕЛАНИЕ",
    "A6392": "КОЛЕЖ ПО ТЕЛЕКОМУНИКАЦИИ",
    "TB0447": "УЛ. ЙЕРУСАЛИМ",
    "TB1256": "ПЕТА ГРАДСКА БОЛНИЦА",
    "A2723": "ПРОМИШЛЕНА ЗОНА УЛ. В. ЛЕВСКИ - ПО ЖЕЛАНИЕ",
    "A1965": "УЛ. ИСКЪР",
    "TM0299": "БУЛ. ВАСИЛ ЛЕВСКИ",
    "A2722": "ПРОМИШЛЕНА ЗОНА УЛ. В. ЛЕВСКИ - ПО ЖЕЛАНИЕ",
    "A0768": "ИНСТИТУТ ПО ЧЕРНА МЕТАЛУРГИЯ - ПО ЖЕЛАНИЕ",
    "A2517": "УЛ. РАЙНА КНЯГИНЯ",
    "A0722": "УЛ. 5007",
    "A1130": "НАЧАЛО С. МЪРЧАЕВО",
    "TB2827": "МЕТРОСТАНЦИЯ СУ СВ. КЛИМЕНТ ОХРИДСКИ",
    "TM0377": "БУЛ. СЛИВНИЦА",
    "M1": "СЛИВНИЦА",
    "TM1055": "МЕТРОСТАНЦИЯ ЛЮЛИН",
    "A1072": "ИНТЕР ЕКСПО ЦЕНТЪР",
    "A1323": "ХОТЕЛ ХЕМУС",
    "A1010": "ЛИФТА ДРАГАЛЕВЦИ",
    "A0456": "ГАРА КАЗИЧЕНЕ",
    "A1667": "УЛ. ВЕСКО ВЕСЕЛИНОВ",
    "A1157": "УЛ. ГЮЕШЕВО",
    "A1969": "УЛ. ЙОСИФ ЩРОСМАЙЕР",
    "A0298": "БУЛ. ВАСИЛ ЛЕВСКИ",
    "TM2234": "УЛ. ХЕМУС",
    "TM0298": "БУЛ. ВАСИЛ ЛЕВСКИ",
    "A6036": "АВТОСТАНЦИЯ КНЯЖЕВО",
    "A6462": "УЛ. ЛЕТОСТРУЙ",
    "TB0018": "24-ТИ ДКЦ",
    "A2199": "УЛ. СУХОДОЛСКИ ПЪТ - ПО ЖЕЛАНИЕ",
    "A1259": "ПЛОЩАД НА АВИАЦИЯТА",
    "A2086": "УЛ. ОПЪЛЧЕНСКА",
    "A1482": "УЛ. ДЕТЕЛИНА",
    "TB1567": "ПГСС",
    "TB2112": "УЛ. ПИРОТСКА",
    "TB0617": "ЛТУ",
    "A1470": "БЛ. 4 Ж.К. СУХАТА РЕКА",
    "A2166": "УЛ. ЛУИ ПАСТЬОР",
    "A2603": "ПЪТЕН ВЪЗЕЛ КАЛОТИНА",
    "A2533": "ПО ЖЕЛАНИЕ - ШОЛЦ БЪЛГАРИЯ",
    "A2616": "СПОРТНА ЗАЛА АРЕНА АРМЕЕЦ",
    "A1532": "СЕЛО ЖЕЛЯВА",
    "A6146": "ГАРА СОФИЯ СЕВЕР",
    "A1004": "ЛЕТИЩЕ ДОБРОСЛАВЦИ - ПО ЖЕЛАНИЕ",
    "A1924": "УЛ. ГЕОРГИ МИНЧЕВ",
    "A2156": "УЛ. РЕСЕНСКА ПОЛЯНА",
    "TB0623": "Ж.К. ИВАН ВАЗОВ",
    "TM1255": "ПЕТА ГРАДСКА БОЛНИЦА",
    "A2506": "ПЪТЕН ВЪЗЕЛ КАЛОТИНА",
    "A0850": "КВ. КЛИСУРА",
    "A0998": "ЛАГЕРА",
    "A2532": "ПО ЖЕЛАНИЕ - ШОЛЦ БЪЛГАРИЯ",
    "A0511": "ЧИТАЛИЩЕ СВ.СВ. КИРИЛ И МЕТОДИЙ",
    "TB1319": "ПЛ. СТОЧНА ГАРА",
    "A2602": "УЛ. ИСКЪР БОЖУРИЩЕ",
    "A2024": "АВТОСТАНЦИЯ ИЗТОК",
    "A6347": "УЛ. УЧЕНИЧЕСКА",
    "A0080": "АП ЗЕМЛЯНЕ",
    "A2570": "БЛ. 13 Ж.К. ЗАПАДЕН ПАРК",
    "A1870": "УЛ. ВИТИНЯ",
    "A2297": "СТАРАТА ФУРНА - ПО ЖЕЛАНИЕ",
    "A2031": "УЛ. ЛИЛЯЧЕ",
    "A2715": "КОШАРИТЕ",
    "A1502": "САТИРИЧЕН ТЕАТЪР",
    "A1730": "ТЕЛЕВИЗИОННА КУЛА",
    "M305": "ХАДЖИ ДИМИТЪР",
    "A2321": "ХОТЕЛ МАКСИ",
    "TM1969": "УЛ. ЙОСИФ ЩРОСМАЙЕР",
    "A1017": "МЕТРОСТАНЦИЯ ЦАРИГРАДСКО ШОСЕ",
    "A2114": "УЛ. ПИРОТСКА",
    "A6348": "УЛ. ЖАН ДЬО ЛАФОНТЕН",
    "A2871": "ХИЖА АЛЕКО",
    "TB2869": "Ловен парк",
    "A1343": "ПК ДИАНА",
    "A2381": "СТУДЕНТСКИ ГРАД",
    "A2869": "ЛОВЕН ПАРК",
    "A6744": "УЛ. ГЕОРГИ ВОЙТЕХ",
    "A1422": "ПЪРВА ГРАДСКА БОЛНИЦА",
    "A2624": "РЕЗИДЕНШЪЛ ПАРК СОФИЯ",
    "A0720": "НУ ЗА ТАНЦОВО ИЗКУСТВО",
    "A2870": "ХИЖА АЛЕКО",
    "A0289": "БУЛ. БРАТЯ БЪКСТОН",
    "A6471": "УЛ. МИТРОПОЛИТ КИРИЛ ВИДИНСКИ",
    "A2090": "УЛ. ОРИОН",
    "A2573": "УЛ. БУЛИНА ЛИВАДА",
    "TB0662": "Ж.К. МЛАДОСТ-1",
    "A6754": "УЛ.ОТЕЦ ГЕНАДИЙ - ВРЕМЕННА",
    "A0563": "МАХАЛА ЕЛАТА",
    "TB1821": "Ул. Алеко Константинов",
    "TB2859": "Ловен парк",
    "TM2336": "ЦЕНТРАЛНИ ХАЛИ",
    "A2042": "УЛ. МАКГАХАН",
    "TM2261": "УЛ. РАЙСКА ГРАДИНА",
    "A0726": "ЗАГ ДИНАМИК АД",
    "A2089": "УЛ. ОРЕХОВА ГОРА",
    "A6507": "МЕТРОСТАНЦИА К. ВЕЛИЧКОВ",
    "A1262": "ПЕТТЕ МОГИЛИ",
    "TB1728": "ТЕАТЪР СОФИЯ",
    "A6742": "Ж.К. ЗАПАДЕН ПАРК",
    "A6633": "РЕЗИДЕНЦИЯ БОЯНА - ВРЕМЕННА",
    "A0383": "БУЛ. ХРИСТО БОТЕВ",
    "A6535": "Ул.3-ти март-временна",
    "A1256": "ПЕТА ГРАДСКА БОЛНИЦА",
    "A1474": "РЕСТОРАНТ ЗЛАТНАТА РИБКА - ПО ЖЕЛАНИЕ",
    "A6743": "Ж.К. ЗАПАДЕН ПАРК",
    "A0649": "Ж.К. ЛЮЛИН-1,2",
    "TM0411": "Бл. 4 ж.к. Сухата река",
    "A0205": "МБАЛ ТОКУДА БОЛНИЦА",
    "A1264": "ПЛ. АЛ.НЕВСКИ",
    "A1501": "САТИРИЧЕН ТЕАТЪР",
    "A1265": "ПЛ. АЛ.НЕВСКИ",
    "A1423": "ПЪРВА ГРАДСКА БОЛНИЦА",
    "A0767": "ИНСТИТУТ ПО ЧЕРНА МЕТАЛУРГИЯ - ПО ЖЕЛАНИЕ",
    "A6303": "УЛ. СЪЗНАНИЕ",
    "A6548": "ПЛ.ЛЪВОВ МОСТ",
    "TM1778": "КВ. ТРИЪГЪЛНИКА",
    "A0750": "ЗООПАРКА",
    "TM2260": "УЛ. РАЙСКА ГРАДИНА",
    "TM1495": "РУСКО ПОСОЛСТВО",
    "A0882": "КВ. ОВЧА КУПЕЛ",
    "A6693": "СЕЛО НЕГОВАН - ВРЕМЕННА",
    "TM0381": "УЛ. ХАН КУБРАТ",
    "TM2595": "Ж.К. ГЕО МИЛЕВ",
    "A0544": "РИТЕКС АД",
    "TM0305": "БУЛ. ВЪЗКРЕСЕНИЕ",
    "A0637": "Ж.К. КРАСНО СЕЛО",
    "A2054": "УЛ. НАРОДНО ХОРО",
    "A1754": "УЛ. ВИТИНЯ",
    "A6474": "ЦЕНТРАЛЕН ГРОБИЩЕН ПАРК",
    "A6532": "УЛ.БЕЛИ БРОД - ВРЕМЕННА",
    "TB0014": "163-ТО ОУ",
    "A6705": "ул.Крушва градина - временна",
    "A1333": "ЦЕНТРАЛНА ГАРА",
    "TB2105": "УЛ. ОХРИД",
    "A0177": "ПРОМИШЛЕНА ЗОНА Ж.К. ДРУЖБА 1",
    "A0558": "ЕЗЕРОТО",
    "A0907": "ПЪТЕН ВЪЗЕЛ ФИЛИПОВЦИ",
    "A1352": "ПО ЖЕЛАНИЕ",
    "A0032": "36-ТО СОУ",
    "A1289": "ПЛ. ОРЛОВ МОСТ",
    "A1321": "ПЛ. СТОЧНА ГАРА",
    "A2456": "СТОКОВ БАЗАР ИЛИЯНЦИ",
    "A6140": "УЛ.БУРЯ",
    "A0041": "66-ТО СОУ",
    "A0460": "ГАРА ОБЕЛЯ",
    "TM0744": "СОФАРМА АД",
    "A2258": "УЛ. ЧЕРКОВНА",
    "A6508": "УЛ. ЦАР СИМЕОН",
    "A6487": "МЕТРОСТАНЦИЯ СУ СВ. КЛИМЕНТ ОХРИДСКИ",
    "A0853": "АВТОСТАНЦИЯ КНЯЖЕВО",
    "A6441": "КВ.ОВЧА КУПЕЛ",
    "A2421": "УЛ. ПЛОВДИВ",
    "A1449": "РАЗКЛОНА ЗА АП ДРУЖБА",
    "A1677": "УЛ. 19",
    "A6307": "УЛ. МАКГАХАН",
    "A0085": "КОМПЛЕКС НА БАН",
    "A0261": "БОЯНСКА ЦЪРКВА",
    "A1649": "МАХАЛА БАТАРЕЯТА",
    "OM_N12": "",
    "TB2036": "УЛ. ЛУИ АЙЕР",
    "TB2689": "Ж.К. БОКАР",
    "TM1718": "Община Люлин",
    "A6502": "МОСТА КВ. КРИВИНА",
    "A2373": "МЕТРОСТАНЦИЯ БИЗНЕС ПАРК",
    "TM0881": "Кв. Овча купел",
    "A6768": "ХЮНДАЙ БЪЛГАРИЯ",
    "A6769": "СТАДИОН ЛОКОМОТИВ - ВРЕМЕННА",
    "A6767": "ХЮНДАЙ БЪЛГАРИЯ",
    "A6645": "УЛ. 2 - ВРЕМЕННА",
    "TM1823": "УЛ. АЛЕКСАНДЪР ЖЕНДОВ",
    "A1076": "МОСТА КВ. ВИТОША",
    "A6425": "УЛ. Д-Р ИВАН СЕЛИМИНСКИ",
    "A0290": "БУЛ. БЪЛГАРИЯ",
    "A1156": "УЛ. ГЮЕШЕВО",
    "A6421": "УЛ.10",
    "A2458": "НИМ",
    "A6090": "БОЯНСКА ЦЪРКВА",
    "A1783": "ТРИЪГЪЛНИКА Ж.К. ЛЮЛИН",
    "A2333": "ЦЕНТРАЛЕН ГРОБИЩЕН ПАРК",
    "A0533": "ДЕТСКИ ЯСЛИ",
    "A0597": "Ж.К. ГЕО МИЛЕВ",
    "A6757": "УЛ. ХИСАРЯ",
    "A6824": "МЕТРОСТАНЦИЯ К. ВЕЛИЧКОВ",
    "A1204": "ОПИТНА СТАНЦИЯ",
    "A6756": "УЛ. ХИСАРЯ",
    "A1964": "УЛ. ИСКЪР",
    "A1287": "ПЛ. ОРЛОВ МОСТ",
    "A1701": "ПРОМИШЛЕНА ЗОНА КАЗИЧЕНЕ",
    "TB1985": "УЛ. КАП. Д. СПИСАРЕВСКИ",
    "A0305": "БУЛ. ВЪЗКРЕСЕНИЕ",
    "A6758": "Ж.К. МЛАДОСТ-4",
    "M306": "ТЕAТРАЛНА",
    "A1466": "РЕЗИДЕНЦИЯ ВРАНА",
    "A6604": "МС МУСАГЕНИЦА",
    "TB1810": "ХРАМ СВ. КЛИМЕНТ ОХРИДСКИ",
    "A1810": "ХРАМ СВ. КЛИМЕНТ ОХРИДСКИ",
    "A2708": "УЛ. ОБЗОР",
    "M10": "ЖОЛИО КЮРИ",
    "TB2040": "УЛ. МАКГАХАН",
    "A1461": "РАЗСАДНИК МАЛО БУЧИНО",
    "A2444": "МЕТРОСТАНЦИЯ ОБЕЛЯ",
    "A1035": "УМБАЛ АЛЕКСАНДРОВСКА",
    "A6480": "УЛ. МАКГАХАН",
    "A0660": "Ж.К. ЛЕВСКИ-Г",
    "A2606": "УЛ. ЛЮБЛЯНА",
    "TB1739": "ПГ ПО ДИЗАЙН ЕЛИСАВЕТА ВАЗОВА",
    "TM2023": "АВТОСТАНЦИЯ ИЗТОК",
    "A2471": "УЛ. ЙЕРУСАЛИМ",
    "TB1226": "29-ТИ ДКЦ",
    "A0871": "КВ. ОБЕЛЯ",
    "A0078": "АП ДРУЖБА",
    "A6770": "ПК СПАРТАК - ВРЕМЕННА",
    "A1772": "ТРД",
    "A2294": "ФОХАР АД",
    "A2662": "МЕТРОСТАНЦИЯ ХАН КУБРАТ",
    "TB1469": "БЛ. 4  Ж.К. СУХАТА РЕКА",
    "TM1279": "Пл. Македония",
    "A2017": "УЛ. КУБАДИН",
    "TB0371": "БУЛ. ПРОФ.ЦВ. ЛАЗАРОВ",
    "TB2062": "УЛ. НИШАВА",
    "TM0354": "БУЛ. ПАТРИАРХ ЕВТИМИЙ",
    "A2274": "153-ТА ГИМНАЗИЯ",
    "TB0301": "БУЛ. ВИТОША",
    "TM2825": "БУЛ. ВИТОША",
    "A1172": "ОКОЛОВРЪСТЕН ПЪТ",
    "A0486": "86-ТИ ПОЩЕНСКИ КЛОН",
    "A1626": "СТАРАТА ЧЕШМА - ПО ЖЕЛАНИЕ",
    "A0890": "КВ. ПАВЛОВО",
    "A1077": "УЛ. ГРАДИНА",
    "A6397": "СТУДЕНТСКА ПОЛИКЛИНИКА",
    "A6037": "МЕТРОСТАНЦИЯ ОВЧА КУПЕЛ",
    "A0485": "СТОКОВ БАЗАР ИЛИЯНЦИ",
    "A2704": "ХОТЕЛ МОРЕНИ",
    "A6594": "Ж.К. ИВАН ВАЗОВ",
    "TB0314": "БУЛ. ГОЦЕ ДЕЛЧЕВ",
    "TM0633": "Ж.К. КРАСНА ПОЛЯНА",
    "A0296": "БУЛ. ВАРДАР",
    "A6687": "УЛ. РАВНЕЦ - ВРЕМЕННА",
    "A2107": "МЕТРОСТАНЦИЯ ХАДЖИ ДИМИТЪР",
    "A0332": "БУЛ. ЛОМСКО ШОСЕ",
    "A1842": "УЛ. БЕСАРАБИЯ",
    "A1020": "МАГАЗИН МЕТРО 2",
    "A1787": "ТУРМАЧКА МАХАЛА",
    "A6067": "МУЗЕЙ НА МЕЧКАТА - ПО ЖЕЛАНИЕ",
    "A6118": "СЕМИНАРИЯТА",
    "A0216": "БЛ. 43 Ж.К. ГОРУБЛЯНЕ",
    "A0525": "ДЕТСКИ ДОМ",
    "A0432": "В.З. БАНКЯ",
    "A0110": "БЕТОНОВ ВЪЗЕЛ",
    "A0235": "МЕТРОСТАНЦИЯ ДРУЖБА",
    "M0": "ОБЕЛЯ",
    "A2344": "ЦЕНТЪР С. ПОДГУМЕР",
    "A0334": "БУЛ. ЛОМСКО ШОСЕ",
    "A6186": "ВРЕМЕННА УЧИЛИЩЕТО",
    "A1759": "115-ТИ ПОЩЕНСКИ КЛОН",
    "A0534": "ДЕТСКИ ЯСЛИ",
    "A6475": "ТЕХНОТЕСТ",
    "A1930": "УЛ. ДИМИТЪР ПЕШЕВ",
    "A2285": "64-ТО ОУ",
    "TB0588": "Ж.К. БЪКСТОН",
    "TM0477": "ГАРА СОФИЯ СЕВЕР",
    "A0397": "ХОТЕЛ ХИЛТЪН",
    "A1792": "УЛ. 2",
    "A0808": "КВ. БОЯНА",
    "A0971": "КОРАБЧЕТО",
    "A0229": "БЛ. 519 Ж.К. ОВЧА КУПЕЛ-1",
    "TB2677": "БЛ. 245 Ж.К. МЛАДОСТ 2",
    "A6119": "УЛ.АКАД.СТ.МЛАДЕНОВ",
    "M318": "ГОРНА БАНЯ",
    "A1403": "УЛ. ХИСАРЯ",
    "TB0484": "УЛ. Т. КАБЛЕШКОВ",
    "TM1631": "БУЛ. КРЪСТЬО ПАСТУХОВ",
    "A0344": "БУЛ. НИКОЛА ВАПЦАРОВ",
    "A0903": "КВ. СЛАВОВЦИ - ПО ЖЕЛАНИЕ",
    "A1528": "СЕЛО ДОЛНИ ПАСАРЕЛ",
    "A2400": "ЦЕНТЪР ДОЛНИ ЛОЗЕН",
    "A0901": "КВ. СИМЕОНОВО",
    "A1275": "ПЛ. ЛЪВОВ МОСТ",
    "A1704": "МАГАЗИН ВИТОША",
    "A0515": "ДЕПО ИСКЪР",
    "A6089": "МЕТРОСТАНЦИЯ МИЗИЯ",
    "TB1856": "УЛ. БЯЛО МОРЕ",
    "A0015": "163-ТО ОУ",
    "A1370": "ПО ЖЕЛАНИЕ",
    "TM2115": "НАДЛЕЗ НАДЕЖДА",
    "A0483": "ГАРА ГОРНА БАНЯ",
    "A6411": "БУЛ. ПРАГА",
    "A0366": "БУЛ. ПРАГА",
    "A1168": "ОВЧАРСКИ КОШАРИ",
    "A1290": "ПЛ. ОРЛОВ МОСТ",
    "A1606": "СК НА НСА",
    "A2788": "66-ТО ОДЗ ПАНЧАРЕВО",
    "A1401": "ПОСОЛСТВАТА",
    "A2232": "УЛ. ХАН КУБРАТ",
    "TB6168": "ПЕТА ГРАДСКА БОЛНИЦА - ВРЕМЕННА",
    "A6346": "ГАРА ИВАНЯНЕ",
    "TB1151": "СК ЦСКА",
    "A2221": "УЛ. ТЪРНОВО",
    "M5": "КОНСТАНТИН ВЕЛИЧКОВ",
    "A1859": "УЛ. БЯЛО МОРЕ",
    "A2213": "УЛ. Т. КАБЛЕШКОВ",
    "A0164": "БЛ. 214 Ж.К. БОРОВО",
    "A0959": "КМЕТСТВО ИВАНЯНЕ",
    "A2065": "ХИТ МАРКЕТ ЛЮЛИН",
    "A2298": "ТЪРГОВСКИ К-С НАДЕЖДА",
    "TB1253": "ЗАПАДЕН ПАРК",
    "A1064": "УЛ. 503",
    "A2284": "152-РО ОУ",
    "M14": "АЛЕКСАНДЪР МАЛИНОВ",
    "OM_N13": "",
    "TB0865": "УЛ. БИСТРИШКО ШОСЕ",
    "A1685": "ПРОМИШЛЕНА ЗОНА - ПО ЖЕЛАНИЕ",
    "A0262": "ГРОБИЩЕН ПАРК БОЯНА - ПО ЖЕЛАНИЕ",
    "A6324": "Ул. Нешо Бончев",
    "A0238": "БЛ. 6 Ж.К. МЛАДОСТ-1",
    "TM0671": "Ж.К. МОТОПИСТА",
    "A1243": "КВ. ПАНЧАРЕВО - ПО ЖЕЛАНИЕ",
    "A0562": "МАХАЛА ЕЛАТА",
    "A0935": "КМЕТСТВО БОЖУРИЩЕ",
    "A2296": "СТАРАТА ФУРНА - ПО ЖЕЛАНИЕ",
    "A7004": "СКАЙ СИТИ",
    "TB1613": "СТАДИОН Г. АСПАРУХОВ",
    "TB2660": "МЕТРОСТАНЦИЯ ЛОМСКО ШОСЕ",
    "TM1732": "ТЕЛЕВИЗИОННА КУЛА",
    "A0548": "ПРОМИШЛЕНА ЗОНА КРИВИНА",
    "A2818": "УЛ. ВРЪПЛИНА",
    "A0939": "ЦЕНТЪРА КВ. ИЛИЯНЦИ - ПО ЖЕЛАНИЕ",
    "TB0976": "Ул. Сребърна",
    "TB1820": "УЛ. АЛЕКСАНДЪР ЖЕНДОВ",
    "A6357": "УЛ.БЪЛГАРСКИ ВОИН",
    "TM2332": "ЦЕНТРАЛЕН ГРОБИЩЕН ПАРК",
    "A1175": "ОКОЛОВРЪСТЕН ПЪТ",
    "A1942": "УЛ. СТАРА ПЛАНИНА",
    "A6572": "БУЛ. ПРАГА",
    "TB6418": "Пл.Руски паметник",
    "TM0754": "СТОКОВ БАЗАР ИЛИЯНЦИ",
    "A1218": "ОЦ ОВЧА КУПЕЛ",
    "A1336": "ПЛОДОХРАНИЛИЩЕ",
    "A1642": "УЛ. РУЧЕЙ",
    "A1691": "УНСС",
    "A2709": "КВ. ГОРНА БАНЯ",
    "A6720": "БЛ. 20 Ж.К. ЛЕВСКИ В - временна",
    "TM0727": "ЗАГ ДИНАМИК АД",
    "A0537": "ПРОИЗВОДСТВЕН К-С МОСКВА",
    "A2567": "Ж.К. ЗАПАДЕН ПАРК",
    "A1393": "ПОЛЕТО",
    "A2003": "УЛ. КОЗЛОДУЙ",
    "A6131": "ПЛ. РУСКИ ПАМЕТНИК",
    "A6701": "Ул. Народен герой-временна",
    "A0028": "28-МИ ДКЦ",
    "TB0164": "БЛ. 214 Ж.К. БОРОВО",
    "A1282": "ПЛ. МАКЕДОНИЯ",
    "A6155": "ВРЕМЕННА 8ТМ",
    "TB2343": "ЦЕНТЪР ПО ХИГИЕНА",
    "A2087": "УЛ. ОПЪЛЧЕНСКА",
    "A1152": "СК ЦСКА",
    "A6700": "Ул. Народен герой-временна",
    "A0024": "28-МИ ДКЦ",
    "A0337": "БУЛ. МАДРИД",
    "A1610": "СПОРТНА ЗАЛА ХР. БОТЕВ",
    "A2215": "УЛ. ТОДОРИНИ КУКЛИ",
    "A0113": "БЕТОНОВ ВЪЗЕЛ - ПО ЖЕЛАНИЕ",
    "A1114": "НАДЛЕЗ НАДЕЖДА",
    "A2138": "УЛ. ПРОБУДА",
    "OM_N14": "",
    "A2378": "ШУМАКО - ПО ЖЕЛАНИЕ",
    "A1217": "РАЗКЛОНА ЗА ЛИФТ ДРАГАЛЕВЦИ - ПО ЖЕЛАНИЕ",
    "TB1224": "29-ТИ ДКЦ",
    "A0203": "БЛ. 39 Ж.К. ГОРУБЛЯНЕ",
    "A0900": "УЛ. ЛЮБЛЯНА",
    "A1856": "УЛ. БЯЛО МОРЕ",
    "A2548": "АВТОБАЗА ИСКЪР",
    "TB6718": "ДАНОН СЕРДИКА АД",
    "A6021": "Ж.К. ГЕО МИЛЕВ",
    "A0741": "ЗИМЕН ДВОРЕЦ НА СПОРТА",
    "A0979": "НАЧАЛО С. ЛОКОРСКО",
    "A1975": "УЛ. КАЛИМАНЦИ",
    "TM0365": "БУЛ. ПРАГА",
    "A6075": "Ж.К. ЛЕВСКИ Г",
    "A1225": "УЛ. ПАТРИАРХ ДАМЯН",
    "A1315": "ПЛ. СТОЧНА ГАРА",
    "A6164": "БУЛ. П.ЕВТИМИЙ ВР.",
    "A1808": "УЛ. 6-ТИ СЕПТЕМВРИ",
    "A2724": "ПРОМИШЛЕНА ЗОНА Ж.К. ДРУЖБА 1",
    "TB0396": "ХОТЕЛ ХИЛТЪН",
    "A1871": "УЛ. ВИТИНЯ",
    "TM0410": "БЛ. 4 Ж.К. СУХАТА РЕКА",
    "A1694": "СТУДЕНТСКИ ГРАД",
    "A1797": "УЛ. ДЕМОКРАЦИЯ",
    "A0916": "ВОЕННОРЕМОНТЕН ЗАВОД",
    "A2257": "УЛ. ЦВЕТНА ГРАДИНА",
    "A1634": "ХЮНДАЙ БЪЛГАРИЯ",
    "A1727": "ТЕАТЪР СОФИЯ",
    "A6517": "БУЛ. БОТЕВГРАДСКО ШОСЕ",
    "A0999": "ЛАГЕРА",
    "A2660": "МЕТРОСТАНЦИЯ ЛОМСКО ШОСЕ",
    "A1276": "ПЛ. ЛЪВОВ МОСТ",
    "A6583": "УЛ. АТ. МАНЧЕВ",
    "A6399": "УЛ. ОДРИН",
    "A1174": "ОКОЛОВРЪСТЕН ПЪТ",
    "A6120": "ТЕХНИЧЕСКИ УНИВЕРСИТЕТ",
    "TB0868": "КВ. МАНАСТИРСКИ ЛИВАДИ",
    "TB1696": "Ж.К. ИЗТОК",
    "TB1998": "УЛ. КНЯЗ БОРИС I",
    "A0614": "ЛТУ",
    "A0978": "НАЧАЛО С. ЛОКОРСКО",
    "A0451": "ГАРА ЗАХАРНА ФАБРИКА",
    "A0046": "УЛ. ПОЩЕНСКА",
    "A0276": "БУЛ. АЛ. ПУШКИН",
    "A0331": "УЛ. ХРИСТО СИЛЯНОВ",
    "OM_N15": "",
    "A0377": "БУЛ. СЛИВНИЦА",
    "TB1799": "КУЛТУРЕН ЦЕНТЪР ЛЮЛИН",
    "TM0879": "Кв. Овча купел",
    "A2705": "АВТОСТАНЦИЯ ОВЧА КУПЕЛ",
    "A1216": "РАЗКЛОНА ЗА ЛИФТ ДРАГАЛЕВЦИ - ПО ЖЕЛАНИЕ",
    "TB0023": "ОНКОЛОГИЧЕН ДИСПАНСЕР",
    "A0324": "БУЛ. КНЯГИНЯ МАРИЯ ЛУИЗА",
    "A1584": "СЕРВИЗ ПОЛМО",
    "A2498": "УЛ. КАЛО",
    "A2503": "УЛ. ХАН ТЕРВЕЛ КВ. МАКСИМ ГОРКИ",
    "A0961": "КМЕТСТВО ЛОЗЕН",
    "A0196": "БЛ. 35 Ж.К. ЗАХАРНА ФАБРИКА",
    "A1241": "ПЛ. ВАСИЛ ЛЕВСКИ",
    "A0991": "КУЛИНАРЕН К-Т ПЕЙФИЛ",
    "TB2463": "Ж.К. МЛАДОСТ-1",
    "TB6107": "НДК",
    "A1003": "ЛЕТИЩЕ ДОБРОСЛАВЦИ - ПО ЖЕЛАНИЕ",
    "TM2112": "УЛ. ПИРОТСКА",
    "A1358": "КАРИЕРНИ МАТЕРИАЛИ АД",
    "A6506": "МЕТРОСТАНЦИЯ К. ВЕЛИЧКОВ",
    "A2653": "Ж.К. СОФИЯ ПАРК",
    "A6685": "УЛ. ВАСИЛ ЛЕВСКИ - ВРЕМЕННА",
    "A0345": "БУЛ. НИКОЛА ПЕТКОВ",
    "A1316": "ПЛ. СТОЧНА ГАРА",
    "A1813": "УЛ. 640",
    "A2790": "МЕТРОСТАНЦИЯ СОФИЙСКА СВЕТА ГОРА",
    "TB1965": "УЛ. ИСКЪР",
    "A0277": "БУЛ. АЛ. ПУШКИН",
    "TB2136": "БУЛ. ГОЦЕ ДЕЛЧЕВ",
    "TB2596": "ПЕТА ГРАДСКА БОЛНИЦА",
    "A0190": "ТЕХНО ПАРК СОФИЯ",
    "TB1306": "ПЛ. РУЧЕЙ",
    "A0030": "36-ТИ ПОЩЕНСКИ КЛОН",
    "A2095": "УЛ. ОРФЕЙ",
    "A1404": "11-ТИ ПОЩЕНСКИ КЛОН",
    "A6183": "СУ СВ. КЛ. ОХРИДСКИ",
    "A1899": "УЛ. ГЕН. НИКОЛА ЖЕКОВ",
    "A2101": "УЛ. ОХРИД",
    "A2202": "УЛ. ЦАРИБРОДСКА",
    "A2223": "УЛ. УОШБЪРН",
    "A1705": "МАГАЗИН ВИТОША",
    "OM_N16": "",
    "A1135": "НДК",
    "A6707": "МЕТРОСТАНЦИЯ БЕЛИ ДУНАВ",
    "TM1872": "Ул. Витиня",
    "TM2585": "УЛ. МЮНХЕН",
    "A1129": "НАЧАЛО С. КЪТИНА - ПО ЖЕЛАНИЕ",
    "A2162": "Ж.К. КРАСНА ПОЛЯНА",
    "M314": "КРАСНО СЕЛО",
    "A1022": "МАХАЛА ДЖЮЛНИЦИ",
    "TB1313": "ПЛ. СТОЧНА ГАРА",
    "A1865": "УЛ. ВЕЛЧО АТАНАСОВ",
    "A2406": "ШКОЛА БЧК",
    "A2661": "МЕТРОСТАНЦИЯ ХАН КУБРАТ",
    "A2768": "УЛ. ЦВЕТНА ГРАДИНА",
    "M7": "СЕРДИКА",
    "A1492": "РУМЪНСКО ПОСОЛСТВО",
    "A1326": "УЛ. КЛОКОТНИЦА",
    "A0936": "КМЕТСТВО ЖЕЛЕЗНИЦА",
    "A0989": "КУЛИНАРЕН К-Т ПЕЙФИЛ",
    "A1112": "СТЦ ИНТЕРПРЕД",
    "A0543": "РИТЕКС АД",
    "A0695": "ГАРА ЧЕПИНЦИ",
    "TB0021": "26-ТИ ДКЦ",
    "TM1987": "УЛ. КИРИЛ БЛАГОЕВ",
    "A2134": "УЛ. ПРОФ. Д. АТАНАСОВ",
    "A0116": "БИОМЕТ ЕООД - ПО ЖЕЛАНИЕ",
    "A0023": "ОНКОЛОГИЧЕН ДИСПАНСЕР",
    "A1819": "УЛ. АЛЕКСАНДЪР ЖЕНДОВ",
    "A0956": "ОБЩИНА НОВИ ИСКЪР - ПО ЖЕЛАНИЕ",
    "A1739": "ПГ ПО ДИЗАЙН ЕЛИСАВЕТА ВАЗОВА",
    "A2186": "УЛ. СТАРА ПЛАНИНА",
    "A1002": "ЛЕТИЩЕ ДОБРОСЛАВЦИ",
    "TB2740": "МЕТРОСТАНЦИЯ ДРУЖБА",
    "TM2253": "УЛ. ЦАРЕВЕЦ",
    "M203": "НАДЕЖДА",
    "A0500": "ГОРУБЛЯНСКО ХАНЧЕ",
    "A0643": "Ж.К. КРЕМИКОВЦИ",
    "A2123": "УЛ. ПОП ГРУЙО",
    "A2369": "ДОМ НА КУЛТУРАТА ИСКЪР",
    "A7007": "Бул.Шипченски проход",
    "A6703": "Ул. Папрат - временна",
    "TB6697": "ПК Спартак-временна",
    "A2720": "ВЗ. ЛЮЛИН",
    "A6629": "УЛ. УРАЛ",
    "A0933": "КМЕТСТВО БАЛША",
    "A2346": "ЦЕНТЪР КВ.ФИЛИПОВЦИ",
    "TM1410": "28-МИ ПОЩЕНСКИ КЛОН",
    "A0732": "ЗАСЛОН СИНАНИЦА",
    "A6404": "МЛАДЕЖКИ ТЕАТЪР",
    "A0532": "ДЕТСКИ ДОМ КИНОЦЕНТЪР",
    "A2732": "ПАЗАР МАЛАШЕВЦИ",
    "A1831": "УЛ. БАБА ТОНКА - ПО ЖЕЛАНИЕ",
    "A0899": "КВ. СИМЕОНОВО",
    "A1518": "СЕЛО ГОЛЯНОВЦИ",
    "A0326": "БУЛ. К. ВЕЛИЧКОВ",
    "A6028": "УЛ. ГЕН. ГУРКО",
    "M23": "ЛЕТИЩЕ СОФИЯ",
    "A1862": "УЛ. ВАСИЛ КЪНЧЕВ",
    "A6637": "ВАКЛИНОВИ КОШАРИ - ПО ЖЕЛАНИЕ",
    "TB2278": "61-ВО ОУ",
    "A0017": "20-ТИ ДКЦ",
    "A0488": "ПРОМИШЛЕНА ЗОНА",
    "A1971": "УЛ. ЙОСИФ ЩРОСМАЙЕР",
    "A0284": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "A2137": "УЛ. ПРОБУДА",
    "A6428": "БУЛ. ПЕЙО ЯВОРОВ",
    "TM1058": "МЕТРОСТАНЦИЯ ОПЪЛЧЕНСКА",
    "A2158": "УЛ. РИЛСКА ОБИТЕЛ",
    "A1364": "УЛ. ЦАР ИВАН АСЕН ІІ",
    "TM0567": "БАЛКАНКАР АД",
    "TM2826": "БУЛ. ВИТОША",
    "A1616": "НАЧАЛО С. ДОБРОСЛАВЦИ",
    "A2509": "ХОТЕЛ МАКСИ",
    "TM0797": "КАТОЛИЧЕСКИ ГРОБИЩЕН ПАРК",
    "A0347": "БУЛ. НИКОЛА ПЕТКОВ",
    "A6529": "УЛ.БЕЛИ БРОД - ВРЕМЕННА",
    "OM_N17": "",
    "A6456": "УЛ. 6-ТИ СЕПТЕМВРИ",
    "A2425": "БОЛНИЦАТА КВ. ДРАГАЛЕВЦИ",
    "TM0010": "14-ТИ ДКЦ",
    "A1228": "КВ. ФАКУЛТЕТА",
    "A1545": "СЕЛО КЪТИНА",
    "A0471": "ГАРА ПОДУЯНЕ",
    "TM0035": "КВ. ГАРА ИСКЪР",
    "M4": "ВАРДАР",
    "A0757": "ИНЖСТРОЙ СОФИЯ АД",
    "A2088": "УЛ. ОРЕХОВА ГОРА",
    "A1703": "МАГАЗИН ВИТОША",
    "A2510": "ПО ЖЕЛАНИЕ",
    "A2127": "УЛ. ПОП ГРУЙО",
    "A6500": "СУ 130 СТ. КАРАДЖА",
    "A1509": "СЕЛИМИЦА",
    "A1814": "УЛ. 640",
    "A1869": "УЛ. ВИТИНЯ",
    "A2203": "УЛ. ТЕМЕНУГА - ПО ЖЕЛАНИЕ",
    "A2811": "УЛ. САВА ФИЛАРЕТОВ",
    "A1188": "ОКОЛОВРЪСТЕН ПЪТ",
    "TB1754": "УЛ. ВИТИНЯ",
    "A2504": "УЛ. ХАН КРУМ КВ. МАКСИМ ГОРКИ",
    "A6248": "БЛ. 541 Ж.К. ЛЮЛИН 5",
    "A0507": "ГРОБИЩЕН ПАРК НЕГОВАН",
    "TB0681": "Ж.К. ОБЕЛЯ-2",
    "A1832": "УЛ. БАБА ТОНКА - ПО ЖЕЛАНИЕ",
    "A1205": "ОПИТНА СТАНЦИЯ",
    "TM0320": "БУЛ. ДЖАВАХАРЛАЛ НЕРУ",
    "A2164": "УЛ. РОЯК",
    "A2214": "УЛ. ТОДОРИНИ КУКЛИ",
    "A1715": "ОБЩИНА ЛЮЛИН",
    "A0818": "КВ. ВЕРДИКАЛ",
    "A1395": "БЪЛГАРСКА ТЕЛЕГРАФНА АГЕНЦИЯ",
    "A1005": "ЛЕТИЩЕ ДОБРОСЛАВЦИ",
    "A1467": "РЕЗИДЕНЦИЯ ВРАНА",
    "A6602": "БУЛ. ВАРНА",
    "A0093": "31-ВО СУЧЕМ",
    "A2652": "Ж.К. СОФИЯ ПАРК",
    "A0468": "ГАРА ПОДУЯНЕ",
    "A1233": "УЛ. КРАЛИ МАРКО",
    "A1878": "УЛ. ВИХРЕН",
    "TM1979": "УЛ. КАП. Д. СПИСАРЕВСКИ",
    "A2313": "ХЛЕБОЗАВОДА",
    "A2075": "УЛ. ОДРИН",
    "TB0653": "Ж.К. ЛЮЛИН-3",
    "TB2779": "Аджибадем сити клиник",
    "A2259": "УЛ. ЧЕРКОВНА",
    "A1095": "МОСТА НА ВЛАДАЙСКА РЕКА",
    "A1534": "СЕЛО ЖИТЕН",
    "A2698": "ЗАСЛОН БАЙ КРЪСТЬО - ПО ЖЕЛАНИЕ",
    "A0452": "ГАРА ЗАХАРНА ФАБРИКА",
    "TB1824": "УЛ. АЛЕКСАНДЪР ЖЕНДОВ",
    "A1650": "НАЧАЛО С. ГОЛЯНОВЦИ",
    "A1900": "УЛ. ГЕН. НИКОЛА ЖЕКОВ",
    "A1943": "УЛ. СТАРА ПЛАНИНА",
    "A6243": "БУЛ. БЪЛГАРИЯ",
    "A0453": "ГАРА ИВАНЯНЕ",
    "A6132": "УМБАЛСМ ПИРОГОВ",
    "TM0910": "МЕТРОСТАНЦИЯ ВИТОША",
    "A1353": "ПО ЖЕЛАНИЕ",
    "A0969": "ПАЗАРА МЛАДОСТ 1",
    "A2190": "УЛ. СТОЯН МИХАЙЛОВСКИ",
    "TB0517": "ДЕПО ИСКЪР",
    "A0807": "КВ. БОЯНА",
    "M20": "ДРУЖБА",
    "A1521": "СЕЛО ГОРНИ БОГРОВ",
    "TB6411": "БУЛ. ПРАГА",
    "A0858": "КВ. КУМАРИЦА",
    "A1552": "КМЕТСТВО МЪРЧАЕВО",
    "A1226": "29-ТИ ДКЦ",
    "A2249": "УЛ. ЦАР СИМЕОН",
    "A1826": "УЛ. АНГЕЛ ВОЙВОДА",
    "A1214": "РАЗКЛОНА ЗА ВОЛУЯК",
    "TB2080": "УЛ. ОПЪЛЧЕНСКА",
    "TM2098": "УЛ. ОСВОБОЖДЕНИЕ",
    "TB0325": "БУЛ. К. ВЕЛИЧКОВ",
    "A1193": "УМБАЛ СВ. АННА",
    "A2729": "РАЗКЛОНА ЗА КВ. ВИДНИТЕ",
    "A0579": "СОФИЯ ТЕХ ПАРК",
    "A6512": "БУЛ. АКАД. ИВ. ГЕШОВ",
    "M208": "СЕРДИКА 2",
    "A0037": "53-ТО ОУ",
    "A6166": "СЕЛО ЧЕПИНЦИ",
    "A6429": "БИСТРИШКО ШОСЕ",
    "TM1727": "ТЕАТЪР СОФИЯ",
    "A1306": "ПЛ. РУЧЕЙ",
    "A0373": "БУЛ. ХРИСТОФОР КОЛУМБ",
    "A6565": "УЛ. БЯЛАТА ЧЕШМА - ПО ЖЕЛАНИЕ",
    "A2491": "МЕТРОСТАНЦИЯ БИЗНЕС ПАРК",
    "A2814": "УЛ. ВАСИЛ ЛЕВСКИ",
    "A2238": "УЛ. ХРИСТО БОТЕВ",
    "TM1594": "СОФИЯ МЕД АД",
    "TB2124": "УЛ. ПОП ГРУЙО",
    "A2097": "УЛ. ОРЯХОВО",
    "A6320": "УЛ. Н. КОПЕРНИК",
    "A1396": "ВИСШЕ УЧИЛИЩЕ ПО ТЕЛЕКОМУНИКАЦИИ И ПОЩИ",
    "A0932": "КМЕТСТВО БАЛША",
    "A0982": "КРАЯ НА КВ. МИХАЙЛОВО",
    "A0908": "ПЪТЕН ВЪЗЕЛ ФИЛИПОВЦИ",
    "A1858": "УЛ. БЯЛО МОРЕ",
    "A0169": "МЕТРОСТАНЦИЯ АЛ. МАЛИНОВ",
    "A0844": "КВ. ИЗГРЕВ",
    "A1220": "УЛ. ЗОРНИЦА",
    "A1278": "ПЛ. ЛЪВОВ МОСТ",
    "A1015": "Ж.К. ДЪРВЕНИЦА",
    "TM0596": "Ж.К. ГЕО МИЛЕВ",
    "A1223": "УЛ. НАДЕЖДА",
    "A2331": "ЦАРЕВА МАХАЛА",
    "A6245": "УЛ. РЕЗБАРСКА - ВРЕМЕННА",
    "A0307": "БУЛ. ВЪЗКРЕСЕНИЕ",
    "A6920": "УЛ. ТИРАНА",
    "TB0355": "БУЛ. ПЕНЧО СЛАВЕЙКОВ",
    "TB0647": "ПК СПАРТАК",
    "A2187": "УЛ. СТАРА ПЛАНИНА",
    "TM0386": "УАСГ",
    "A0866": "КВ. МАНАСТИРСКИ ЛИВАДИ",
    "TB0443": "УМБАЛ СВ. ЕКАТЕРИНА",
    "TM0379": "ЖЕНСКИ ПАЗАР",
    "A0798": "КАУЧУКОВ ЗАВОД ЗЕБРА - ПО ЖЕЛАНИЕ",
    "A1202": "ОПИТНА СТАНЦИЯ",
    "TB6604": "МС МУСАГЕНИЦА",
    "TM1932": "УЛ. ДОБРОТИЧ",
    "A0527": "ДЕТСКИ ДОМ",
    "A1075": "ДАНОН СЕРДИКА АД",
    "A1379": "ПО ЖЕЛАНИЕ",
    "A1998": "УЛ. КНЯЗ БОРИС I",
    "A2460": "ГАРА КУРИЛО",
    "A2276": "144-ТО СОУ",
    "TM1933": "УЛ. ДОБРОТИЧ",
    "A1806": "УЛ. 6-ТИ СЕПТЕМВРИ",
    "A2226": "УЛ. ХАЙДУТ СИДЕР",
    "TB1134": "НДК",
    "A0043": "8-МИ ДКЦ",
    "A0094": "НЦ ПО РЕХАБИЛИТАЦИЯ",
    "TM1737": "ПГД ЕЛИСАВЕТА ВАЗОВА",
    "A0461": "ГАРА ОБЕЛЯ",
    "TB1875": "УЛ. ВИХРЕН",
    "A1207": "ОПИТНА СТАНЦИЯ",
    "TB0406": "БУЛ. ПАНЧО ВЛАДИГЕРОВ",
    "TM0582": "КВ. МАНАСТИРСКИ ЛИВАДИ - ЗАПАД",
    "A1309": "ПЛ. СРЕД СЕЛО",
    "A2117": "УЛ. ПОДУЕНСКА",
    "A1948": "УЛ. ЗОГРАФСКИ МАНАСТИР",
    "TM1996": "ПЛ. ГАРИБАЛДИ",
    "A6251": "БУЛ.ЧЕРНИ ВРЪХ",
    "A2029": "УЛ. ЛИЛЯЧЕ",
    "A2043": "УЛ. МАЛОМИР",
    "A0925": "БАЛКАНКЕРАМИК АД - ПО ЖЕЛАНИЕ",
    "A1985": "УЛ. КАП. Д. СПИСАРЕВСКИ",
    "A1477": "УЛ. ДЕТЕЛИНА",
    "A2414": "ЧИТАЛИЩЕ СВЕТЛИНА",
    "TB0646": "Ж.К. ЛЕВСКИ Г",
    "A6134": "Ж.К. ЛАГЕРА",
    "TM1046": "МЕТРОСТАНЦИЯ ВАРДАР",
    "A0495": "ГОРСКО СТОПАНСТВО",
    "A1342": "ПК ДИАНА",
    "A2140": "УЛ. ПРОМИШЛЕНА",
    "M312": "БЪЛГАРИЯ",
    "A1301": "ПЛ. РУЧЕЙ",
    "A1938": "УЛ. ДРАВА",
    "A6686": "УЛ. ВАСИЛ ЛЕВСКИ - ВРЕМЕННА",
    "A6139": "УЛ.ОМАЯ",
    "TB1894": "УЛ. ГЕН. НИКОЛА ГЕНЕВ",
    "A1563": "СЕЛО ЧЕПИНЦИ",
    "A1652": "ПРОМИШЛЕНА ЗОНА ГОРНИ ЛОЗЕН",
    "TB2380": "ЮЖЕН ПАРК",
    "A0776": "УЛ. КАЛОЯНОВО",
    "A0930": "КИНОЦЕНТЪР БОЯНА",
    "M315": "ОВЧА КУПЕЛ",
    "A1967": "КВ. КИНОЦЕНТЪР",
    "A6099": "КВ. ГРАДОМАН",
    "A1465": "НИМ",
    "A1912": "УЛ. ГОЦЕ ДЕЛЧЕВ",
    "A2470": "ТОЛУМСКА МАХАЛА - ПО ЖЕЛАНИЕ",
    "A0467": "ГАРА ПОДУЯНЕ",
    "A1578": "ТЮФ",
    "A1633": "ХЮНДАЙ БЪЛГАРИЯ",
    "A6681": "УЛ. КРУШАТА - ВРЕМЕННА",
    "A1268": "ПЛ. БИРИМИРЦИ",
    "A1666": "УЛ. ЦАР ИВАН ШИШМАН",
    "TM0476": "Гара София север",
    "TM0013": "140-ТО СОУ",
    "A2315": "ХЛЕБОЗАВОД БОТУНЕЦ АД",
    "TB1941": "УЛ. ДУНАВ",
    "A1684": "НАЧАЛО КВ. ТРЕБИЧ - ПО ЖЕЛАНИЕ",
    "TM0851": "КВ. КНЯЖЕВО",
    "A6033": "УЛ. ВИТИНЯ",
    "TB1930": "УЛ. ДИМИТЪР ПЕШЕВ",
    "A1199": "УЛ. ВРЪХ МАНЧО",
    "A2157": "УЛ. РЕСЕНСКА ПОЛЯНА",
    "A6403": "МЛАДЕЖКИ ТЕАТЪР",
    "A0904": "КВ. СЛАВОВЦИ - ПО ЖЕЛАНИЕ",
    "A1539": "СЕЛО КОКАЛЯНЕ",
    "A0282": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "A2283": "152-РО ОУ",
    "A2350": "НАЧАЛО С. ЖИТЕН",
    "TM1165": "10-ТИ ДКЦ",
    "A0070": "РЦ ПО ПРОТЕЗИРАНЕ",
    "A1683": "НАЧАЛО КВ. ТРЕБИЧ - ПО ЖЕЛАНИЕ",
    "A2604": "УЛ. ИСКЪР БОЖУРИЩЕ",
    "A2592": "КОКАЛЯНСКО ХАНЧЕ",
    "A6484": "БЛ.207 Ж.К.ОБЕЛЯ 2",
    "A1419": "ПРОСТОР",
    "A1222": "УЛ. НАДЕЖДА",
    "A0457": "ГАРА КУРИЛО - ПО ЖЕЛАНИЕ",
    "M3": "ЗАПАДЕН ПАРК",
    "TM0353": "БУЛ. ПАТРИАРХ ЕВТИМИЙ",
    "A1514": "КМЕТСТВО ВЛАДАЯ",
    "A2363": "ПЛ. ВИТОША",
    "A1849": "УЛ. БУРЯ",
    "A6519": "УЛ. ТИРАНА",
    "A6678": "УЛ. КАЛИМАНЦИ - ВРЕМЕННА",
    "A1523": "СЕЛО ГОРНИ ЛОЗЕН",
    "A1958": "УЛ. ИЛИЯ БЕШКОВ",
    "A2224": "УЛ. УРАЛ",
    "A2735": "УЛ. 746 - ПО ЖЕЛАНИЕ",
    "A0617": "ЛТУ",
    "A0738": "ЗЕЛЕНЧУКОВА ГРАДИНА",
    "A0746": "СОФАРМА АД",
    "TB2107": "МЕТРОСТАНЦИЯ ХАДЖИ ДИМИТЪР",
    "A0975": "СОФИЯ МЕД АД",
    "A1749": "ТЕЦ ИЗТОК",
    "TB0866": "КВ. МАНАСТИРСКИ ЛИВАДИ",
    "A1830": "УЛ. АТАНАС УЗУНОВ",
    "A2618": "УЛ. ЕДЕЛВАЙС - ПО ЖЕЛАНИЕ",
    "A2071": "УЛ. ОБОРИЩЕ",
    "A2412": "ЦЕНТЪР ДОЛНИ ЛОЗЕН",
    "A1499": "НАЧАЛО С. ВЛАДАЯ - ПО ЖЕЛАНИЕ",
    "A6018": "кв. Бузема-временна",
    "TB6233": "Ж.К. МОТОПИСТА",
    "A1546": "СЕЛО ЛОКОРСКО",
    "A1335": "ПЛАЖА ПАНЧАРЕВО - ПО ЖЕЛАНИЕ",
    "TB1034": "УМБАЛ АЛЕКСАНДРОВСКА",
    "A1295": "ПЛ. РУСКИ ПАМЕТНИК",
    "A1421": "ПТИЦЕФЕРМАТА",
    "A1531": "СЕЛО ЖЕЛЕЗНИЦА",
    "A0394": "БУЛ. ЦАР БОРИС ІІІ",
    "A1089": "МОСТА ДОЛНИ ПАСАРЕЛ",
    "TM1993": "УЛ. КЛОКОТНИЦА",
    "A6122": "УЛ.РОСАРИО",
    "A6550": "УЛ.ИНДИРА ГАНДИ",
    "A2229": "УЛ. ХАЙДУТ СИДЕР",
    "A6665": "УЛ. ОВЧЕ ПОЛЕ - ВРЕМЕННА",
    "TB0303": "МЕТРОСТАНЦИЯ ХАДЖИ ДИМИТЪР",
    "A2250": "УЛ. ЦАР СИМЕОН",
    "A2780": "МЕТРОСТАНЦИЯ ВИТОША",
    "TB0803": "Ж.К. БАНИШОРА",
    "A1548": "СЕЛО МАЛО БУЧИНО",
    "A1660": "СТРАННОПРИЕМНИЦАТА",
    "A0823": "КВ. ВРАЖДЕБНА",
    "A0960": "КМЕТСТВО ИВАНЯНЕ",
    "A1550": "СЕЛО МРАМОР",
    "A1208": "ОРАНЖЕРИЯТА - ПО ЖЕЛАНИЕ",
    "A1378": "ПО ЖЕЛАНИЕ",
    "TM0847": "КВ. ИЛИЯНЦИ",
    "TM1570": "СЕМИНАРИЯТА",
    "A1252": "ЗАПАДЕН ПАРК",
    "A2737": "В.З. ПОДГУМЕР - ПО ЖЕЛАНИЕ",
    "A0905": "КВ. ТРЕБИЧ - ПО ЖЕЛАНИЕ",
    "A0988": "КУЛИНАРЕН К-Т ПЕЙФИЛ",
    "A1234": "УЛ. КРАЛИ МАРКО",
    "A6442": "КВ.ОВЧА КУПЕЛ",
    "A0620": "Ж.К. ЗАПАДЕН ПАРК",
    "A2813": "ВИТОШКО ЛАЛЕ",
    "A0025": "28-МИ ДКЦ",
    "A2801": "СЕЛО ДОЛНИ ЛОЗЕН",
    "M201": "ЛОМСКО ШОСЕ",
    "A1468": "РЕМОНТНА БАЗА",
    "A0526": "УЛ. ЛАГАДИНА",
    "TB2326": "ХОТЕЛ ПЛИСКА",
    "OM_N18": "",
    "A1161": "117 - ТО СОУ",
    "A2810": "УЛ. ХАН КУБРАТ",
    "TM0892": "Кв. Павлово",
    "A2781": "ЗООПАРКА",
    "A0126": "БЛ. 10 Ж.К. ЛЕВСКИ-Г",
    "A0443": "УМБАЛ СВ. ЕКАТЕРИНА",
    "TB1241": "ПЛ. ВАСИЛ ЛЕВСКИ",
    "A0965": "УЛ. СТРАНДЖА",
    "A0784": "УЛ. ДРУЖБА",
    "A2696": "РАЗКЛОНА ЗА ДРАГАЛЕВСКИ МАНАСТИР - ПО ЖЕЛАНИЕ",
    "TB2122": "УЛ. ПОП БОГОМИЛ",
    "TB2210": "УЛ. Т. КАБЛЕШКОВ",
    "A2457": "СТОКОВ БАЗАР ИЛИЯНЦИ",
    "A0051": "АВТОСТАНЦИЯ БАНКЯ",
    "A0449": "ГАРА ВОЛУЯК",
    "A2508": "БЛ. 60 СТУДЕНТСКИ ГРАД",
    "A2765": "КВ. РЕПУБЛИКА",
    "TM1777": "КВ. ТРИЪГЪЛНИКА",
    "A2154": "УЛ. РЕЗБАРСКА",
    "A2415": "УЛ. МЛАДОСТ - ПО ЖЕЛАНИЕ",
    "A2569": "БЛ. 13 Ж.К. ЗАПАДЕН ПАРК",
    "A0269": "БУЛ. АКАД. ИВ. ГЕШОВ",
    "A1420": "ПТИЦЕФЕРМАТА",
    "A2133": "УЛ. ПОРУЧИК ГР. ЗАГОРСКИ",
    "A1504": "СБА - ПО ЖЕЛАНИЕ",
    "A0659": "ХОТЕЛ АМБАСАДОР",
    "A1780": "ТРИЪГЪЛНИКА КВ. ГОРУБЛЯНЕ",
    "TM1045": "МЕТРОСТАНЦИЯ ВАРДАР",
    "A0288": "БУЛ. БРАТЯ БЪКСТОН",
    "A0427": "УЛ. ЗЛАТЕН ВЕК",
    "A2362": "ПЛ. ВИТОША",
    "A6115": "ГОРУБЛЯНСКО ХАНЧЕ",
    "TM1908": "ПЛ. СЛАВЕЙКОВ",
    "A1785": "ТРЪНСКА МАХАЛА",
    "A1898": "УЛ. ГЕН. НИКОЛА ЖЕКОВ",
    "A1412": "ПРЕЧИСТВАТЕЛНА СТАНЦИЯ",
    "A1679": "НАЧАЛО С. ПОДГУМЕР - ПО ЖЕЛАНИЕ",
    "A0510": "ГРОБИЩЕН ПАРК ДРАГАЛЕВЦИ",
    "A1189": "ПРОМИШЛЕНА ЗОНА - ЛОМСКО ШОСЕ",
    "TM2588": "БУЛ. ПРОФ. ЦВ. ЛАЗАРОВ",
    "A0338": "БУЛ. МАДРИД",
    "A0934": "КМЕТСТВО БОЖУРИЩЕ",
    "A0385": "БУЛ. ХРИСТО БОТЕВ",
    "A1981": "УЛ. КАП. В. ДАНАДЖИЕВ",
    "A2330": "ХОТЕЛ ХЕМУС",
    "A1765": "УЛ. ТРАКИЙСКА МОГИЛА",
    "TM2086": "УЛ. ОПЪЛЧЕНСКА",
    "A6253": "МОСТА НА СУХАТА РЕКА - ВРЕМЕННА",
    "A1524": "СЕЛО ГОРНИ ЛОЗЕН",
    "A0498": "ГОРУБЛЯНСКО ХАНЧЕ",
    "A1450": "РАЗКЛОНА ЗА АП ДРУЖБА",
    "TB1287": "ПЛ. ОРЛОВ МОСТ",
    "A0724": "УЛ. ПОРУЧИК ХР.ТОПРАКЧИЕВ",
    "TB2491": "МЕТРОСТАНЦИЯ БИЗНЕС ПАРК",
    "TM0656": "ж.к. Люлин-5",
    "A0119": "109-ТО ОУ",
    "A1162": "117 - ТО СОУ",
    "TB2342": "ЦЕНТЪР ПО ХИГИЕНА",
    "A0074": "БУЛ. ПЕЙО ЯВОРОВ",
    "A0760": "УМБАЛСМ ПИРОГОВ",
    "A2074": "УЛ. ОБОРИЩЕ",
    "A2739": "МЕТРОСТАНЦИЯ ДРУЖБА",
    "A0715": "СПИИДО ЕООД",
    "A1311": "ПЛ. СТОЧНА ГАРА",
    "A0301": "БУЛ. ВИТОША",
    "A1911": "УЛ. ГОЦЕ ДЕЛЧЕВ",
    "A6698": "БУЛ. РОЖЕН - ВРЕМЕННА",
    "A2687": "ОКОЛОВРЪСТЕН ПЪТ",
    "A1621": "НАЧАЛО С. ДОБРОСЛАВЦИ",
    "A6724": "ж.к. Хаджи Димитър - временна",
    "A2511": "ПО ЖЕЛАНИЕ",
    "TM6400": "БУЛ. ПРАГА",
    "TB6525": "ВМА - ВРЕМЕННА",
    "A1579": "РАЗКЛОНА ЗА КВ. ВРАЖДЕБНА",
    "TB0074": "БУЛ. ПЕЙО ЯВОРОВ",
    "TB1858": "УЛ. БЯЛО МОРЕ",
    "A1142": "УЛ. ЗАД СЕЛО",
    "A6516": "БУЛ. БОТЕВГРАДСКО ШОСЕ",
    "A6135": "Ж.К. ХИПОДРУМА",
    "A0433": "В.З. СУХОДОЛ",
    "A1090": "МОСТА ЗА РЕЗИДЕНЦИЯ ВРАНА",
    "A0893": "КВ. ПАВЛОВО",
    "A2082": "УЛ. ОПЪЛЧЕНСКА",
    "A1059": "МЕТРОСТАНЦИЯ СЛИВНИЦА",
    "TM1873": "УЛ. ВИТИНЯ",
    "A0805": "КВ. БЕНКОВСКИ",
    "A2064": "ХИТ МАРКЕТ ЛЮЛИН",
    "A1586": "СЛАТИНА БУЛГАРПЛОД",
    "A0045": "УЛ. ПОЩЕНСКА",
    "A1489": "РУМЪНСКО ПОСОЛСТВО",
    "TM0326": "БУЛ. К. ВЕЛИЧКОВ",
    "A1039": "СТАДИОНА ГЕРМАН",
    "TM0547": "БУЛ. РОЖЕН",
    "A6162": "УЛ. ПЕРНИК",
    "A0924": "БАЛКАНКЕРАМИК АД - ПО ЖЕЛАНИЕ",
    "A2359": "УЛ. РАЛИЦА",
    "A1418": "ПРОСТОР",
    "A2339": "ЦЕНТЪР КВ. АБДОВИЦА",
    "A0737": "ЗЕЛЕНЧУКОВА ГРАДИНА",
    "A2342": "ЦЕНТЪР ПО ХИГИЕНА",
    "A1802": "УЛ. 5",
    "A0928": "ЧИТАЛИЩЕ Д-Р П. БЕРОН",
    "TB0238": "БЛ. 6 Ж.К. МЛАДОСТ-1",
    "A0664": "Ж.К. МЛАДОСТ-2",
    "A2416": "УЛ. МЛАДОСТ - ПО ЖЕЛАНИЕ",
    "A2505": "УЛ. ХАН КРУМ КВ. МАКСИМ ГОРКИ",
    "A0836": "КВ. ДРАГАЛЕВЦИ",
    "A0826": "ГАРА ИСКЪР",
    "A1947": "УЛ. ЗЛАТИЦА",
    "A0884": "КВ. ОРЛАНДОВЦИ",
    "A1030": "УЛ. МОНТЕВИДЕО",
    "A0565": "БАЛКАНКАР АД",
    "A2328": "ХОТЕЛ ХЕМУС",
    "TM0911": "МЕТРОСТАНЦИЯ ВИТОША",
    "TM2073": "УЛ. ОБОРИЩЕ",
    "A2390": "БИЗНЕС ПАРК СОФИЯ",
    "TB0162": "БЛ. 207 Ж.К. ОБЕЛЯ-2",
    "A0484": "УЛ. Т. КАБЛЕШКОВ",
    "A1196": "УМБАЛ СВ. АННА",
    "TM1988": "УЛ. КИРИЛ БЛАГОЕВ",
    "TM1915": "УЛ. ГРОЗДЕН",
    "A2688": "Ж.К. БОКАР",
    "TM1051": "МЕТРОСТАНЦИЯ К. ВЕЛИЧКОВ",
    "A2473": "НОВА ТЕЛЕВИЗИЯ",
    "A0856": "КВ. КУМАРИЦА - ПО ЖЕЛАНИЕ",
    "A0021": "26-ТИ ДКЦ",
    "A0859": "КВ. КУМАРИЦА - ПО ЖЕЛАНИЕ",
    "A2366": "ЧИТАЛИЩЕ Д. ЧИНТУЛОВ",
    "A6492": "УЛ. САМОКОВСКО ШОСЕ",
    "A0297": "БУЛ. ВАСИЛ ЛЕВСКИ",
    "A1294": "ПЛ. ПИРДОП",
    "A2680": "МЕТРОСТАНЦИЯ ЦАРИГРАДСКО ШОСЕ",
    "A2716": "ПО ЖЕЛАНИЕ",
    "A0578": "БУЛ. ХРИСТОФОР КОЛУМБ",
    "A0972": "КОРИЯТА",
    "A2692": "УЛ. СТОЯН МИХАЙЛОВСКИ",
    "A1368": "ПО ЖЕЛАНИЕ",
    "A1512": "СЕЛО БИСТРИЦА",
    "A2311": "ХЛАДИЛЕН ЗАВОД",
    "A2821": "МЕТРОСТАНЦИЯ ХАДЖИ ДИМИТЪР",
    "TB1716": "ОБЩИНА ЛЮЛИН",
    "A2372": "ДОМ НА КУЛТУРАТА ИСКЪР",
    "TB0334": "БУЛ. ЛОМСКО ШОСЕ",
    "TB6616": "Ж.К. МЛАДОСТ-1",
    "A2079": "УЛ. ОПЪЛЧЕНСКА",
    "TB2452": "РВД",
    "TM2298": "ТЪРГОВСКИ К-С НАДЕЖДА",
    "A6323": "Ул. Ком",
    "A1150": "СТРАННОПРИЕМНИЦАТА",
    "A0833": "КВ. Д. МИЛЕНКОВ",
    "A1693": "СТУДЕНТСКИ ГРАД",
    "A0481": "УЛ. СИНЧЕЦ",
    "A2742": "МЕТРОСТАНЦИЯ АЛ. МАЛИНОВ",
    "A1670": "УЛ. ЛИПА",
    "A1536": "СЕЛО ЖИТЕН",
    "A0749": "ЗООПАРКА",
    "A0889": "КВ. ПАВЛОВО",
    "A2538": "УЛ. МАЛЬОВИЦА - ПО ЖЕЛАНИЕ",
    "TB2111": "УЛ. ПИРОТСКА",
    "A0955": "ОБЩИНА НОВИ ИСКЪР - ПО ЖЕЛАНИЕ",
    "A1186": "ОКОЛОВРЪСТЕН ПЪТ - ПО ЖЕЛАНИЕ",
    "A1516": "СЕЛО ВОЙНЕГОВЦИ",
    "A2326": "ХОТЕЛ ПЛИСКА",
    "A2357": "УЛ. ПРОФ. АЛ. СТАНИШЕВ",
    "TM2249": "УЛ. ЦАР СИМЕОН",
    "A2689": "Ж.К. БОКАР",
    "TB2676": "БЛ. 245 Ж.К. ОБЕЛЯ-2",
    "TB0626": "Ж.К. ИВАН ВАЗОВ",
    "A6612": "МЕТРОСТАНЦИЯ ВАРДАР",
    "TB1152": "СК ЦСКА",
    "A2410": "УЛ. ПРОЛЕТ",
    "A6606": "ОКОЛОВРЪСТЕН ПЪТ",
    "TM1254": "ПЕТА ГРАДСКА БОЛНИЦА",
    "TM0383": "БУЛ. ХРИСТО БОТЕВ",
    "A0841": "КВ. ИЗГРЕВ - ПО ЖЕЛАНИЕ",
    "TM0300": "БУЛ. ВАСИЛ ЛЕВСКИ",
    "A1517": "СЕЛО ГЕРМАН",
    "A2622": "АВТОСТАНЦИЯ КНЯЖЕВО",
    "A1903": "УЛ. ГЕН. ГУРКО",
    "A2286": "УЛ. УЧИЛИЩНА",
    "A1515": "КМЕТСТВО ВЛАДАЯ",
    "TM0295": "БУЛ. ВАРДАР",
    "TM1731": "ТЕЛЕВИЗИОННА КУЛА",
    "A2306": "ХИПЕРМАРКЕТ БИЛА",
    "A6027": "УЛ. ЗАРНИЧЕ",
    "TB0382": "БУЛ. ХРИСТО БОТЕВ",
    "TB2012": "УЛ. КРАКРА",
    "A0165": "БЛ. 214 Ж.К. БОРОВО",
    "A0462": "ГАРА ПИОНЕР",
    "A0857": "КВ. КУМАРИЦА",
    "A1866": "УЛ. ВЕЛЧО АТАНАСОВ",
    "A2050": "УЛ. МОНТЕВИДЕО",
    "A6074": "ОКОЛОВРЪСТЕН ПЪТ-временна",
    "TM1135": "НДК",
    "TB2024": "АВТОСТАНЦИЯ ИЗТОК",
    "TB0516": "Депо Искър",
    "A6520": "СУПЕРМАРКЕТ",
    "A0283": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "A2305": "ХИПЕРМАРКЕТ БИЛА",
    "TB2780": "Метростанция Витоша",
    "A0636": "Ж.К. КРАСНО СЕЛО",
    "A0788": "КАНТОНА",
    "A6651": "УЛ. 7 - ВРЕМЕННА",
    "A2562": "УЛ. АНДРЕЙ ГЕРМАНОВ",
    "A0814": "КВ. БУСМАНЦИ",
    "A6325": "УЛ. МЕЛОДИЯ",
    "A1983": "УЛ. КАП. Д. СПИСАРЕВСКИ",
    "A6147": "БУЛ. РОЖЕН",
    "A1190": "ПРОМИШЛЕНА ЗОНА - ЛОМСКО ШОСЕ",
    "A2124": "УЛ. ПОП ГРУЙО",
    "A0450": "ГАРА ВОЛУЯК",
    "A2062": "УЛ. НИШАВА",
    "A2547": "БИСТРИШКИ ТЕРАСИ - ПО ЖЕЛАНИЕ",
    "TM2003": "УЛ. КОЗЛОДУЙ",
    "A0627": "Ж.К. ДИАНАБАД",
    "A1511": "СЕЛО БАЛША",
    "TB6169": "УЛ. ПОДП. КАЛИТИН - ВРЕМЕННА",
    "A6682": "УЛ. КРУШАТА - ВРЕМЕННА",
    "A2132": "УЛ. ПОРУЧИК ГР. ЗАГОРСКИ",
    "A2507": "БЛ. 60 СТУДЕНТСКИ ГРАД",
    "A6699": "СТАДИОН ЛОКОМОТИВ - ВРЕМЕННА",
    "A0970": "КОРАБЧЕТО",
    "TM2067": "МЕТРОСТАНЦИЯ ОВЧА КУПЕЛ",
    "A1599": "БАЛКАН СТАР",
    "TB0372": "БУЛ. ХРИСТОФОР КОЛУМБ",
    "A2753": "БЛ. 458 Ж.К. МЛАДОСТ 4",
    "A0325": "БУЛ. К. ВЕЛИЧКОВ",
    "A2069": "УЛ. ОБИКОЛНА",
    "A2091": "УЛ. ОРИОН",
    "A2405": "ШКОЛА БЧК",
    "A1695": "Ж.К. ИЗТОК",
    "A2320": "ХОТЕЛ МАКСИ",
    "A6726": "Ж.К. МЛАДОСТ 4- временна",
    "TB1300": "ПЛ. РУСКИ ПАМЕТНИК",
    "A1559": "СЕЛО СВЕТОВРАЧАНЕ",
    "A1146": "ГРОБИЩЕН ПАРК МАЛАШЕВЦИ",
    "A1609": "СК НА НСА",
    "A2452": "РВД",
    "A1127": "НАЧАЛО КВ. КЛИСУРА",
    "TM0886": "КВ. ОРЛАНДОВЦИ",
    "A2482": "УЛ. ВИТИНЯ",
    "M212": "ВИТОША",
    "TM0364": "БУЛ. ПРАГА",
    "A0418": "ВМА",
    "TM1939": "УЛ. ДУНАВ",
    "A1219": "ОЦ ОВЧА КУПЕЛ",
    "A6477": "БУЛ.БРЮКСЕЛ",
    "A1519": "СЕЛО ГОЛЯНОВЦИ",
    "A6136": "УЛ. НИШАВА",
    "TM2665": "ЦЕНТРАЛНА АВТОГАРА",
    "A6470": "БЪЛГАРСКО НАЦИОНАЛНО РАДИО",
    "TB2087": "УЛ. ОПЪЛЧЕНСКА",
    "A1671": "УЛ. СПОРТИСТ - ПО ЖЕЛАНИЕ",
    "A2418": "БУЛ. ЦАРИЦА ЙОАНА",
    "TB2082": "УЛ. ОПЪЛЧЕНСКА",
    "A1876": "ЯЗ. МАЛО БУЧИНО",
    "A1672": "УЛ. СПОРТИСТ - ПО ЖЕЛАНИЕ",
    "A6220": "ЦЕНТРАЛНА ГАРА",
    "A6287": "ОКОЛОВР. ПЪТ - ВРЕМЕННА",
    "TB1819": "УЛ. АЛЕКСАНДЪР ЖЕНДОВ",
    "A2593": "ХЕРМЕС ПАРК",
    "A2833": "РАЗКЛОНА ЗА С. ГЕРМАН",
    "A2136": "БУЛ. ГОЦЕ ДЕЛЧЕВ",
    "A0687": "Ж.К. СЛАТИНА",
    "A1990": "УЛ. КИРИЛ ХРИСТОВ",
    "A0725": "ЗАГ ДИНАМИК АД",
    "A0895": "КВ. РЕПУБЛИКА",
    "A0832": "КВ. ГРАДОМАН",
    "TB2354": "НЦ ПО РАДИОБИОЛОГИЯ - ПО ЖЕЛАНИЕ",
    "A1678": "УЛ. 19",
    "TM0036": "КВ. ГАРА ИСКЪР",
    "A0358": "БУЛ. ПЕНЧО СЛАВЕЙКОВ",
    "A6362": "УЛ. СТАРА ПЛАНИНА - ВРЕМЕННА",
    "TM1573": "СЕМИНАРИЯТА",
    "A0957": "ОБЩИНА БАНКЯ",
    "A2453": "РВД",
    "A6391": "КОЛЕЖ ПО ТЕЛЕКОМУНИКАЦИИ",
    "TM0097": "НЦ ПО РЕХАБИЛИТАЦИЯ",
    "A0431": "В.З. БАНКЯ",
    "A1209": "ОРАНЖЕРИЯТА - ПО ЖЕЛАНИЕ",
    "A2312": "ХЛАДИЛЕН ЗАВОД",
    "A6345": "ГАРА ИВАНЯНЕ",
    "A0846": "КВ. ИЗГРЕВ - ПО ЖЕЛАНИЕ",
    "A0943": "ЧИТАЛИЩЕ ПРОСВЕТА",
    "A1367": "ПО ЖЕЛАНИЕ",
    "A1655": "ПРОМИШЛЕНА ЗОНА",
    "A6282": "ЦЕНТРАЛНА ГАРА - ВРЕМЕННА",
    "TM2311": "ХЛАДИЛЕН ЗАВОД",
    "A0224": "БЛ. 465 Ж.К. МЛАДОСТ-4",
    "A0157": "БЛ. 205 Ж.К. МЛАДОСТ-2",
    "A0405": "БУЛАВТО",
    "A2391": "БИЗНЕС ПАРК СОФИЯ",
    "TB0171": "БЛ. 241 Ж.К. ОБЕЛЯ-2",
    "A6491": "УЛ. САМОКОВСКО ШОСЕ",
    "A0862": "КВ. МАЛАШЕВЦИ",
    "A6494": "БУЛ. ХРИСТО БОТЕВ",
    "A1613": "СТАДИОН Г. АСПАРУХОВ",
    "A6683": "УЛ. ПОБЕДА - ВРЕМЕННА",
    "A1574": "СЕМИНАРИЯТА",
    "A2368": "ЧИТАЛИЩЕ ОТЕЦ ПАИСИЙ",
    "A2673": "ГАРА СОФИЯ СЕВЕР",
    "A6369": "УЛ. АЛЕКО ТУРАНДЖА",
    "A1347": "ПО ЖЕЛАНИЕ",
    "A1527": "ЦЕНТЪР ДОЛНИ ЛОЗЕН",
    "A1857": "УЛ. БЯЛО МОРЕ",
    "A2163": "Ж.К. КРАСНА ПОЛЯНА",
    "A2764": "ЗООПАРКА",
    "A2799": "МАНАСТИР СВ. АПОСТОЛИ ПЕТЪР И ПАВЕЛ",
    "A1369": "ПО ЖЕЛАНИЕ",
    "A1728": "ТЕАТЪР СОФИЯ",
    "A2168": "УЛ. САН СТЕФАНО",
    "A2455": "ЛЕТИЩЕ СОФИЯ ТЕРМИНАЛ 2",
    "A0430": "В.З. ГЕРМАН",
    "A1581": "РАЗКЛОНА ЗА КВ. ВРАЖДЕБНА",
    "TB2221": "УЛ. ТЪРНОВО",
    "A6632": "КВ. БОЯНА - ВРЕМЕННА",
    "TB1344": "ПК СПАРТАК",
    "A0545": "ПРОМИШЛЕНА ЗОНА КРИВИНА",
    "A0765": "ДИПЛОМАТИЧЕСКИ КЛУБ",
    "TM0384": "БУЛ. ХРИСТО БОТЕВ",
    "A2422": "СПЗ МОДЕРНО ПРЕДГРАДИЕ",
    "TM0707": "БУЛ. ВАРДАР",
    "A1131": "НАЧАЛО С. МЪРЧАЕВО",
    "A2682": "КОМПЛЕКС НА БАН",
    "A1288": "ПЛ. ОРЛОВ МОСТ",
    "A0264": "БОЯНСКО ХАНЧЕ",
    "TM0706": "БУЛ. ВАРДАР",
    "A1099": "МОСТА НА Р. ИСКЪР",
    "A2096": "УЛ. ОРЯХОВО",
    "A2248": "УЛ. ЦАР СИМЕОН",
    "A2590": "УЛ. ИНЖ. ГЕОРГИ БЕЛОВ",
    "A0428": "УЛ. ГЕРЕНА",
    "A0514": "АВТОСЕРВИЗ",
    "A0676": "ОБЩИНА ВРЪБНИЦА",
    "TM2084": "УЛ. ОПЪЛЧЕНСКА",
    "A6980": "109-ТО ОУ - ВРЕМЕННА",
    "A2519": "УЛ. ПРОСВЕТА",
    "TB0363": "БУЛ. ПРАГА",
    "A0109": "БЛ. 33 Ж.К. СЛАТИНА",
    "A1349": "НАЧАЛО КВ Д. МИЛЕНКОВ",
    "TB1763": "ПГХМБТ ПРОФ. П. РАЙКОВ",
    "A0865": "УЛ. БИСТРИШКО ШОСЕ",
    "A6603": "Бул. Варна",
    "A0630": "Ж.К. ДИАНАБАД",
    "A1394": "БЪЛГАРСКА ТЕЛЕГРАФНА АГЕНЦИЯ",
    "A2717": "ПО ЖЕЛАНИЕ",
    "TB0032": "36-ТО СОУ",
    "A2784": "МБАЛ ВИТА",
    "A2351": "ТЕХНИЧЕСКА БИБЛИОТЕКА",
    "A6356": "КВ. ИЗГРЕВ",
    "A2303": "РЕСТОРАНТ ЧЕРНАТА КОТКА",
    "A2066": "УЛ. ОБИКОЛНА",
    "A1097": "МОСТА НА Р. ИСКЪР",
    "A0974": "СОФИЯ МЕД АД",
    "A1359": "КАРИЕРНИ МАТЕРИАЛИ АД",
    "A1405": "11-ТИ ПОЩЕНСКИ КЛОН",
    "TB6410": "БУЛ. ПРАГА",
    "A1203": "ОПИТНА СТАНЦИЯ",
    "TM1925": "УЛ. ДИМИТЪР ПЕТКОВ",
    "A1648": "МАХАЛА БАТАРЕЯТА",
    "TM1272": "ПЛ. ВЪЗРАЖДАНЕ",
    "A1637": "ХЮНДАЙ БЪЛГАРИЯ",
    "A2655": "КВ. ХЛАДИЛНИКА",
    "TM1329": "ЦЕНТРАЛНА ГАРА",
    "A0530": "ДЕТСКИ ДОМ",
    "A2327": "ХОТЕЛ ПЛИСКА",
    "A2733": "ПАЗАР МАЛАШЕВЦИ",
    "M9": "СТАДИОН ВАСИЛ ЛЕВСКИ",
    "TB2158": "УЛ. РИЛСКА ОБИТЕЛ",
    "A2324": "ХОТЕЛ ПЛИСКА",
    "TB2659": "МЕТРОСТАНЦИЯ ЛОМСКО ШОСЕ",
    "TB6478": "НДК",
    "TB2678": "БЛ. 245 Ж.К. МЛАДОСТ 2",
    "A0302": "БУЛ. ВИТОША",
    "A0868": "КВ. МАНАСТИРСКИ ЛИВАДИ",
    "A0350": "БУЛ. НИКОЛА ПЕТКОВ",
    "A6723": "ж.к. Хаджи Димитър - временна",
    "M206": "ЦЕНТРАЛНА ГАРА",
    "A2049": "УЛ. МОНТЕВИДЕО",
    "M211": "ДЖЕЙМС БАУЧЕР",
    "A2318": "ХМС",
    "A1982": "УЛ. КАП. В. ДАНАДЖИЕВ",
    "A6600": "Ж.К. ЛЕВСКИ Г",
    "A2364": "УЛ. ПРОФ. АЛ. СТАНИШЕВ",
    "A6483": "БЛ.207 Ж.К.ОБЕЛЯ 2",
    "TB0324": "БУЛ. КНЯГИНЯ МАРИЯ ЛУИЗА",
    "TB1103": "МОСТА ЧАВДАР",
    "A0739": "ЗЕЛЕНЧУКОВА ГРАДИНА",
    "A1974": "УЛ. КАЛИМАНЦИ",
    "TB2483": "Пътностроителна техника",
    "A1920": "УЛ. ДАМЕ ГРУЕВ - ПО ЖЕЛАНИЕ",
    "TM1274": "ПЛ. ЖУРНАЛИСТ",
    "A6026": "УЛ. ЗАРНИЧЕ",
    "A2681": "ХИПЕРМАРКЕТ БИЛА",
    "TB1295": "ПЛ. РУСКИ ПАМЕТНИК",
    "A1363": "ПО ЖЕЛАНИЕ",
    "A2145": "УЛ. ПРОФ. ВАЙГАНД",
    "A0454": "ГАРА ИВАНЯНЕ",
    "A2273": "144-ТО СОУ",
    "A2725": "РАЗКЛОНА ЗА КВ. ВИДНИТЕ",
    "A6317": "УЛ. КОСТА ЛУЛЧЕВ",
    "A0555": "ДЪРЖАВНА ПЕЧАТНИЦА",
    "A6087": "УЛ. МАКГАХАН",
    "A0964": "УЛ. СТРАНДЖА",
    "TB1401": "ПОСОЛСТВАТА",
    "A2141": "УЛ. ПРОМИШЛЕНА",
    "A6098": "УМБАЛСМ ПИРОГОВ",
    "A2539": "УЛ. МАЛЬОВИЦА - ПО ЖЕЛАНИЕ",
    "A2020": "УЛ. ЛАТИНКА",
    "A0048": "АВТОРЕМОНТНА БАЗА",
    "A1191": "ОКОЛОВРЪСТЕН ПЪТ",
    "A1038": "МЕЛНИЦАТА ЧЕПИНЦИ",
    "TB6605": "МС МУСАГЕНИЦА",
    "M316": "МИЗИЯ / НБУ",
    "A2469": "ТОЛУМСКА МАХАЛА - ПО ЖЕЛАНИЕ",
    "A2630": "МИРОВСКО ШОСЕ - ПО ЖЕЛАНИЕ",
    "TB0177": "ПРОМИШЛЕНА ЗОНА Ж.К. ДРУЖБА 1",
    "A2367": "ЧИТАЛИЩЕ ОТЕЦ ПАИСИЙ",
    "TB2121": "УЛ. ПОП БОГОМИЛ",
    "A1840": "УЛ. БЕСАРАБИЯ",
    "A1184": "ОКОЛОВРЪСТЕН ПЪТ",
    "A2011": "УЛ. КОСТЕНСКИ ВОДОПАД",
    "TM2421": "УЛ. ПЛОВДИВ",
    "TB2069": "УЛ. ОБИКОЛНА",
    "A0564": "БАЛКАНКАР АД",
    "A1251": "ЗАПАДЕН ПАРК",
    "A1415": "ПРЕЧИСТВАТЕЛНА СТАНЦИЯ",
    "A0111": "БЕТОНОВ ВЪЗЕЛ КАЗИЧЕНЕ",
    "A0265": "БОЯНСКО ХАНЧЕ",
    "A6182": "СУ СВ. КЛ. ОХРИДСКИ",
    "M317": "ОВЧА КУПЕЛ 2",
    "A1815": "УЛ. 9",
    "A2356": "УЛ. ОДЕСА",
    "A2150": "УЛ. РАВНЕЦ - ПО ЖЕЛАНИЕ",
    "A6598": "БИЗНЕС ПАРК СОФИЯ",
    "A1905": "УЛ. ГЕО МИЛЕВ",
    "A6642": "КМЕТСТВО КАЗИЧЕНЕ",
    "A6652": "ГАРА ОБЕЛЯ",
    "TB0638": "Ж.к. Красно село",
    "A0309": "БУЛ. ГОЦЕ ДЕЛЧЕВ",
    "TB0141": "БЛ. 155 Ж.К. ДРУЖБА-1",
    "A0912": "МЕТРОСТАНЦИЯ ВИТОША",
    "A6631": "КВ. БОЯНА - ВРЕМЕННА",
    "A6514": "УЛ. РАЧО ПЕТКОВ - КАЗАНДЖИЯТА",
    "A6445": "УЛ. ГЕН ГУРКО",
    "A6326": "УЛ. КИРИЛ ПЪРЛИЧЕВ",
    "A0962": "КОКАЛЯНСКО ХАНЧЕ - ПО ЖЕЛАНИЕ",
    "TM2663": "МЕТРОСТАНЦИЯ ХАН КУБРАТ",
    "A2337": "ЦЕНТРАЛНИ ХАЛИ",
    "A6706": "ул.Крушва градина - временна",
    "A0529": "ДЕТСКИ ДОМ - ПО ЖЕЛАНИЕ",
    "A1923": "УЛ. ДАМЯНИЦА",
    "TB0579": "СОФИЯ ТЕХ ПАРК",
    "A0591": "Ж.К. БЪКСТОН",
    "A1034": "УМБАЛ АЛЕКСАНДРОВСКА",
    "TM0356": "БУЛ. ПЕНЧО СЛАВЕЙКОВ",
    "M16": "БИЗНЕС ПАРК СОФИЯ",
    "A1260": "ПЛОЩАД НА АВИАЦИЯТА",
    "A0355": "БУЛ. ПЕНЧО СЛАВЕЙКОВ",
    "TM0546": "БУЛ. РОЖЕН",
    "TM0728": "ЗАГ ДИНАМИК АД",
    "TM2310": "ХЛАДИЛЕН ЗАВОД",
    "A0927": "КИНО ОДЕОН",
    "A0077": "АП ДРУЖБА",
    "A2193": "УЛ. СТЕФАН ПЕШЕВ",
    "A6680": "ХИПЕРМАРКЕТ ХИТ- ВРЕМЕННА",
    "TB2008": "УЛ. КОСТЕНСКИ ВОДОПАД",
    "TB0163": "БЛ. 207 Ж.К. ОБЕЛЯ-2",
    "A6488": "МЕТРОСТАНЦИЯ СЕРДИКА",
    "A1281": "ПЛ. МАКЕДОНИЯ",
    "A2502": "УЛ. ХАН ТЕРВЕЛ КВ. МАКСИМ ГОРКИ",
    "A1696": "Ж.К. ИЗТОК",
    "A6468": "УЛ. ГРАФ ИГНАТИЕВ",
    "A6584": "УЛ. ПРОФ. АТАНАС ИШИРКОВ",
    "A6481": "УЛ. ПАНАЙОТ ПИПКОВ",
    "TB1903": "УЛ. ГЕН. ГУРКО",
    "TB1999": "УЛ. КНЯЗ БОРИС I",
    "TM2248": "УЛ. ЦАР СИМЕОН",
    "TM0334": "БУЛ. ЛОМСКО ШОСЕ",
    "A6360": "БУЛ.ХРИСТО БОТЕВ",
    "TB2471": "УЛ. ЙЕРУСАЛИМ",
    "A2611": "УЛ. МОГИЛКА",
    "A2217": "УЛ. ТОДОРИНИ КУКЛИ",
    "A2371": "ЦЪРКВАТА СВ. ГЕОРГИ",
    "OM_N19": "",
    "A1121": "НАЧАЛО С.ДОЛНИ ЛОЗЕН",
    "TM1994": "УЛ. КЛОКОТНИЦА",
    "A0004": "БЛ. 411 Ж.К. МЛАДОСТ-4",
    "A0415": "ОКОЛОВРЪСТЕН ПЪТ",
    "A6727": "Ул. Каймакчалан-временна",
    "TB2222": "УЛ. УОШБЪРН",
    "TB1719": "ОБЩИНА ЛЮЛИН",
    "TM0885": "КВ. ОРЛАНДОВЦИ",
    "A0599": "Ж.К. ГОЦЕ ДЕЛЧЕВ",
    "M209": "НДК",
    "A2360": "УЛ. ЛЮЛЯК",
    "A2135": "УЛ. ПРОФ. Д. АТАНАСОВ",
    "A1537": "СЕЛО КАЗИЧЕНЕ",
    "A1825": "УЛ. АНГЕЛ ВОЙВОДА",
    "A6316": "УЛ. Н. КОПЕРНИК",
    "A1817": "УЛ. 9",
    "M311": "МЕДИЦИНСКИ УНИВЕРСИТЕТ",
    "TM0312": "БУЛ. ГОЦЕ ДЕЛЧЕВ",
    "TM0375": "МОДНА КЪЩА АГРЕСИЯ",
    "A0573": "МЕСТНОСТ ЕЛИТЕ",
    "A1549": "СЕЛО МИРОВЯНЕ - ПО ЖЕЛАНИЕ",
    "TB0015": "163-ТО ОУ",
    "A2275": "153-ТА ГИМНАЗИЯ",
    "A2192": "УЛ. СТЕФАН ПЕШЕВ",
    "A0419": "ВМА",
    "A6143": "ПЛ.ПИРДОП",
    "TM0597": "Ж.К. ГЕО МИЛЕВ",
    "TM1281": "ПЛ. МАКЕДОНИЯ",
    "A0602": "Ж.К. ДИАНАБАД",
    "A2786": "УЛ. СОЧИ",
    "A2030": "УЛ. ЛИЛЯЧЕ",
    "A1063": "МЕТРОСТАНЦИЯ СЛИВНИЦА",
    "A1344": "ПК СПАРТАК",
    "TB1321": "ПЛ. СТОЧНА ГАРА",
    "A0744": "СОФАРМА АД",
    "A1314": "ПЛ. СТОЧНА ГАРА",
    "A1151": "СК ЦСКА",
    "TB0016": "20-ТИ ДКЦ",
    "A6377": "Кв.Вердикал-временна",
    "A1171": "ОКОЛОВРЪСТЕН ПЪТ",
    "A6309": "Кемпингски хотел Зографски",
    "A0669": "Ж.К. МЛАДОСТ-4",
    "A0651": "Ж.К. ЛЮЛИН-3",
    "A2300": "ХАНЧЕТО - ПО ЖЕЛАНИЕ",
    "A2600": "УЛ. КОМ",
    "TM2583": "УЛ. ИСКЪРСКО ШОСЕ",
    "TM2586": "УЛ. АМСТЕРДАМ",
    "A6233": "Ж.К. МОТОПИСТА",
    "A2271": "129-ТО ОУ",
    "TB1035": "УМБАЛ АЛЕКСАНДРОВСКА",
    "A0504": "ГРАД БУХОВО",
    "A6185": "ВРЕМЕННА УЧИЛИЩЕТО",
    "TB6407": "МЕТРОСТАНЦИЯ ОПЪЛЧЕНСКА",
    "A0921": "УЛ. ЧЕПИНСКО ШОСЕ",
    "A2758": "УЛ. ДИМИТЪР АШОВ",
    "A2629": "МИРОВСКО ШОСЕ - ПО ЖЕЛАНИЕ",
    "A1312": "ПЛ. СТОЧНА ГАРА",
    "A2568": "Ж.К. ЗАПАДЕН ПАРК",
    "TM0327": "БУЛ. К. ВЕЛИЧКОВ",
    "A1187": "ОКОЛОВРЪСТЕН ПЪТ",
    "A0948": "КМЕТСТВО СВЕТОВРАЧАНЕ",
    "A1569": "СЕМИНАРИЯТА",
    "A0931": "КИНОЦЕНТЪР БОЯНА",
    "TM1635": "ХЮНДАЙ БЪЛГАРИЯ",
    "A2741": "МЕТРОСТАНЦИЯ СОФИЙСКА СВЕТА ГОРА",
    "A6513": "УЛ. 5006",
    "A0480": "ГАРА ГОРНА БАНЯ",
    "A1554": "СЕЛО НЕГОВАН",
    "A2770": "СТАДИОНА КВ. ГОРУБЛЯНЕ",
    "A6605": "МС МУСАГЕНИЦА",
    "TB1695": "Ж.К. ИЗТОК",
    "A2115": "НАДЛЕЗ НАДЕЖДА",
    "A6281": "КАТОЛИЧЕСКИ ГРОБИЩЕН ПАРК",
    "A6318": "НУ ЗА ТАНЦОВО ИЗКУСТВО",
    "A0086": "КОМПЛЕКС НА БАН",
    "A1966": "КВ. КИНОЦЕНТЪР",
    "A1143": "УЛ. ЗАД СЕЛО",
    "A1680": "НАЧАЛО С. ПОДГУМЕР - ПО ЖЕЛАНИЕ",
    "A0304": "БУЛ. ВЪЗКРЕСЕНИЕ",
    "A0329": "БУЛ. ЛОМСКО ШОСЕ",
    "A2572": "МЕТРОСТАНЦИЯ ВАРДАР",
    "A1950": "УЛ. ИВАН ЙОСИФОВ",
    "M310": "НДК 2",
    "A0560": "ЕЛ. КАНТОН",
    "A1481": "УЛ. КИТЕН",
    "A1180": "ОКОЛОВРЪСТЕН ПЪТ",
    "A6684": "УЛ. ПОБЕДА - ВРЕМЕННА",
    "TB0418": "ВМА",
    "A1061": "МЕТРОСТАНЦИЯ СЛИВНИЦА",
    "A0825": "ГАРА ИСКЪР",
    "A2501": "НАЧАЛО ТОЛЕВА МАХАЛА",
    "A1743": "ТЕХНИЧЕСКИ УНИВЕРСИТЕТ",
    "A0158": "БЛ. 206 Ж.К. МЛАДОСТ-2",
    "A0249": "БЛ. 7 Ж.К. ХР. СМИРНЕНСКИ",
    "TM2589": "БУЛ. ПРОФ. ЦВ. ЛАЗАРОВ",
    "A1069": "МИНАТА - ПО ЖЕЛАНИЕ",
    "A2643": "К-С ЦАРИГРАДСКИ",
    "A2678": "БЛ. 245 Ж.К. МЛАДОСТ 2",
    "A0727": "ЗАГ ДИНАМИК АД",
    "A2700": "ГОРСКИ ДОМ",
    "TB1736": "ПГ ПО ДИЗАЙН ЕЛИСАВЕТА ВАЗОВА",
    "A0292": "БУЛ. ВАРДАР",
    "A1480": "УЛ. КИТЕН",
    "A2051": "УЛ. НАДЕЖДА",
    "A2793": "БЛ. 648 Ж.К. ЛЮЛИН 6",
    "A6296": "УЛ. НАЙДЕН ГЕРОВ",
    "A6465": "ЦЕНТРАЛЕН ГРОБИЩЕН ПАРК",
    "TB2010": "УЛ. КОСТЕНСКИ ВОДОПАД",
    "A0552": "ДЪРЖАВНА ПЕЧАТНИЦА",
    "A7000": "БУЛ. ПРАГА",
    "TB0020": "26-ТИ ДКЦ",
    "A0987": "МК КРЕМИКОВЦИ",
    "A6148": "ВРЕМЕННА",
    "A6505": "ЦЕНТРАЛЕН ГРОБИЩЕН ПАРК",
    "TB0268": "БУЛ. АКАД. ИВ. ГЕШОВ",
    "A0346": "БУЛ. НИКОЛА ПЕТКОВ",
    "A1713": "МОСТА НА СУХАТА РЕКА",
    "A0730": "ЗАЛА ФЕСТИВАЛНА",
    "A0822": "КВ. ВИТОША",
    "A1675": "НАЧАЛО С. МРАМОР - ПО ЖЕЛАНИЕ",
    "A1181": "ОКОЛОВРЪСТЕН ПЪТ",
    "A0736": "ЗЕЛЕНЧУКОВА ГРАДИНА",
    "A0531": "ДЕТСКИ ДОМ КИНОЦЕНТЪР",
    "A0786": "УЛ. АСЕН ЙОРДАНОВ",
    "TM2077": "УЛ. ОДРИН",
    "A1533": "СЕЛО ЖИТЕН",
    "A2210": "УЛ. Т. КАБЛЕШКОВ",
    "TM0472": "ГАРА ПОДУЯНЕ",
    "A0308": "БУЛ. ГОЦЕ ДЕЛЧЕВ",
    "A1829": "УЛ. АТАНАС УЗУНОВ",
    "A2349": "НАЧАЛО С. ЖИТЕН",
    "A6279": "УЛ. Г. ИЗМИРЛИЕВ",
    "A2526": "ПРЕЧИСТВАТЕЛНА СТАНЦИЯ",
    "A6094": "Ж.К. БЪКСТОН",
    "A1867": "УЛ. ВЕЛЧО АТАНАСОВ",
    "A1896": "УЛ. ГЕН. НИКОЛА ГЕНЕВ",
    "A2685": "БОТАНИЧЕСКА ГРАДИНА",
    "TM2242": "УЛ. ХРИСТО СИЛЯНОВ",
    "A0233": "МЕТРОСТАНЦИЯ АЛ. МАЛИНОВ",
    "A1611": "СПОРТНА ЗАЛА ХР. БОТЕВ",
    "A6322": "УЛ. ЖЕЛЕЗОПЪТНА",
    "A1297": "ПЛ. РУСКИ ПАМЕТНИК",
    "A1000": "УЛ. ЛОКОМОТИВ",
    "A6156": "БУЛ. ЛУИ ПАСТЬОР",
    "A0670": "Ж.К. МЛАДОСТ-4",
    "A2309": "ХЛАДИЛЕН ЗАВОД",
    "A2170": "УЛ. СВ.СВ. КИРИЛ И МЕТОДИЙ",
    "TM1630": "БУЛ. КРЪСТЬО ПАСТУХОВ",
    "A1091": "МОСТА ЗА РЕЗИДЕНЦИЯ ВРАНА",
    "A1337": "ПЛОДОХРАНИЛИЩЕ",
    "TB1984": "УЛ. КАП. Д. СПИСАРЕВСКИ",
    "A0434": "В.З. СУХОДОЛ",
    "TB0221": "БЛ. 458 Ж.К. НАДЕЖДА-4",
    "TB6170": "УЛ. ШУМЕН - ВРЕМЕННА",
    "A6396": "УЛ. 8-МИ ДЕКЕМВРИ",
    "A6499": "СУ 130 СТ. КАРАДЖА",
    "A0559": "ЕЗЕРОТО",
    "A6230": "БУЛ. ПЕНЧО СЛАВЕЙКОВ",
    "TB1303": "ПЛ. РУЧЕЙ",
    "A0840": "КВ. ИВАНЯНЕ",
    "A1973": "УЛ. КАЛИМАНЦИ",
    "TB2615": "СПОРТНА ЗАЛА АРЕНА АРМЕЕЦ",
    "A0112": "БЕТОНОВ ВЪЗЕЛ КАЗИЧЕНЕ",
    "A0528": "ДЕТСКИ ДОМ - ПО ЖЕЛАНИЕ",
    "TB1318": "ПЛ. СТОЧНА ГАРА",
    "A0465": "ГАРА ПИОНЕР",
    "TB0600": "Ж.К. ГОЦЕ ДЕЛЧЕВ",
    "A1937": "УЛ. ДРАВА",
    "TM2477": "МЕТРОСТАНЦИЯ ХАН КУБРАТ",
    "A6241": "МЕТРОСТАНЦИЯ ОБЕЛЯ",
    "A0493": "ИНСТИТУТ ПО ОРТОПЕДИЯ",
    "TM1293": "ПЛ. ПИРДОП",
    "A1132": "НАЧАЛО С. ПЛАНА",
    "A6175": "БУЛ. СЛИВНИЦА - ВРЕМЕННА",
    "A1016": "МЕТРОСТАНЦИЯ ЦАРИГРАДСКО ШОСЕ",
    "A0492": "РЕПУБЛИКАНСКИ НЕВРОДИСПАНСЕР",
    "A1757": "ТОЛЕВА МАХАЛА",
    "A0008": "131-ВО СОУ",
    "A1922": "УЛ. ДАМЯНИЦА",
    "TB1902": "УЛ. ГЕН. ГУРКО",
    "TM1282": "ПЛ. МАКЕДОНИЯ",
    "A0306": "БУЛ. ВЪЗКРЕСЕНИЕ",
    "A0566": "БАЛКАНКАР АД",
    "TB0310": "БУЛ. ГОЦЕ ДЕЛЧЕВ",
    "A0941": "КМЕТСТВО КАЗИЧЕНЕ",
    "A6024": "ПРОМИШЛЕНА ЗОНА БОЖУРИЩЕ",
    "TB2097": "УЛ. ОРЯХОВО",
    "TM0387": "УАСГ",
    "A6409": "БУЛ. АЛЕКСАНДЪР СТАМБОЛИЙСКИ",
    "A6510": "УЛ. ДИМИТЪР ПЕШЕВ",
    "A1724": "ТЕАТЪР СОФИЯ",
    "A0406": "БУЛ. ПАНЧО ВЛАДИГЕРОВ",
    "A0992": "КУЛТУРЕН ДОМ",
    "TB0874": "КВ. ОБЕЛЯ",
    "A0654": "Ж.К. ЛЮЛИН-3",
    "TM0342": "Бул. Никола Вапцаров",
    "A0372": "БУЛ. ХРИСТОФОР КОЛУМБ",
    "M11": "Г. М. ДИМИТРОВ",
    "TB6430": "МЕТРОСТАНЦИЯ КН. МАРИЯ ЛУИЗА",
    "A1308": "ПЛ. СВ. НЕДЕЛЯ",
    "A2612": "УЛ. МОГИЛКА",
    "A1801": "УЛ. ИВАН АНДОНОВ",
    "A2233": "УЛ. ХАН КУБРАТ",
    "A2255": "УЛ. ЦВЕТНА ГРАДИНА",
    "A1647": "НАЧАЛО С. ВОЙНЕГОВЦИ - ПО ЖЕЛАНИЕ",
    "A1719": "ОБЩИНА ЛЮЛИН",
    "A2355": "УЛ. ОДЕСА",
    "A2619": "УЛ. ЕДЕЛВАЙС - ПО ЖЕЛАНИЕ",
    "TM0519": "ДЕПО ИСКЪР",
    "A0793": "КАНТОНА",
    "A1291": "ПЛ. ПИРДОП",
    "TM0796": "КАТОЛИЧЕСКИ ГРОБИЩЕН ПАРК",
    "A1750": "ТЕЦ ИЗТОК",
    "TB0578": "БУЛ. ХРИСТОФОР КОЛУМБ",
    "A1366": "ПО ЖЕЛАНИЕ",
    "A0278": "БУЛ. АЛ. ПУШКИН",
    "A6300": "УМБАЛСМ ПИРОГОВ",
    "A0014": "163-ТО ОУ",
    "M22": "СОФИЙСКА СВЕТА ГОРА",
    "A1665": "УЛ. ЦАР ИВАН ШИШМАН",
    "TB0356": "БУЛ. ПЕНЧО СЛАВЕЙКОВ",
    "A1503": "СБА - ПО ЖЕЛАНИЕ",
    "A2012": "УЛ. КРАКРА",
    "A2749": "УЛ. АЛ. СТАМБОЛИЙСКИ",
    "TM1494": "РУСКО ПОСОЛСТВО",
    "A2615": "СПОРТНА ЗАЛА АРЕНА АРМЕЕЦ",
    "M207": "ЛЪВОВ МОСТ",
    "M202": "БЕЛИ ДУНАВ",
    "A1547": "СЕЛО МАЛО БУЧИНО",
    "A1261": "РЕСТОРАНТ ЧЕРНАТА КОТКА",
    "A2800": "СЕЛО ДОЛНИ ЛОЗЕН",
    "A0388": "БУЛ. ЦАР БОРИС ІІІ",
    "A1221": "УЛ. ЗОРНИЦА",
    "A2151": "УЛ. РАВНЕЦ - ПО ЖЕЛАНИЕ",
    "A6145": "МОДНА КЪЩА АГРЕСИЯ",
    "A0835": "КВ. ДРАГАЛЕВЦИ",
    "TB6415": "КИНО ОДЕОН",
    "A0330": "БУЛ. ЛОМСКО ШОСЕ",
    "A1057": "МЕТРОСТАНЦИЯ ЛЮЛИН",
    "A0909": "МЕТРОСТАНЦИЯ ВИТОША",
    "A6093": "ПЛ. РУСКИ ПАМЕТНИК",
    "A0009": "131-ВО СОУ",
    "A0236": "МЕТРОСТАНЦИЯ ДРУЖБА",
    "A1318": "ПЛ. СТОЧНА ГАРА",
    "A2408": "ХИПЕРМАРКЕТ ПРАКТИКЕР",
    "OM_N20": "",
    "A6591": "РУМЪНСКО ПОСОЛСТВО",
    "A2546": "БИСТРИШКИ ТЕРАСИ - ПО ЖЕЛАНИЕ",
    "A6580": "УЛ. КНЯЖЕВСКА",
    "A2785": "МБАЛ ВИТА",
    "A2336": "ЦЕНТРАЛНИ ХАЛИ",
    "TM2587": "УЛ. АМСТЕРДАМ",
    "A0917": "МЕСТНОСТ МАНИЛОВ ДОЛ",
    "TM2584": "УЛ. МЮНХЕН",
    "A2358": "УЛ. РАЛИЦА",
    "A1362": "ПО ЖЕЛАНИЕ",
    "A1529": "СЕЛО ДОЛНИ ПАСАРЕЛ",
    "TM0880": "КВ. ОВЧА КУПЕЛ",
    "TB1102": "МОСТА ЧАВДАР",
    "A0234": "МЕТРОСТАНЦИЯ АЛ. МАЛИНОВ",
    "TB0270": "БУЛ. АКАД. ИВ. ГЕШОВ",
    "A1060": "МЕТРОСТАНЦИЯ СЛИВНИЦА",
    "A2380": "ЮЖЕН ПАРК",
    "A6607": "ЕСТАКАДАТА",
    "TM0294": "БУЛ. ВАРДАР",
    "A1565": "СЕЛО ЧЕПИНЦИ",
    "A1729": "ТЕЛЕВИЗИОННА КУЛА",
    "TB2688": "Ж.К. БОКАР",
    "A0806": "КВ. БЕНКОВСКИ",
    "A2518": "УЛ. РАЙНА КНЯГИНЯ",
    "A2329": "ХОТЕЛ ХЕМУС",
    "A6030": "ВОЕННА АКАДЕМИЯ",
    "TB2277": "90-ТО СОУ",
    "A0815": "КВ. БУСМАНЦИ",
    "A1577": "ТЮФ",
    "A6582": "УНСС - ВРЕМЕННА",
    "A0491": "СТОКОВ БАЗАР ИЛИЯНЦИ",
    "A1564": "СЕЛО ЧЕПИНЦИ",
    "A7003": "СКАЙ СИТИ",
    "A2769": "СТАДИОНА КВ. ГОРУБЛЯНЕ",
    "A1123": "УЛ. МИНЗУХАР",
    "TB1260": "ПЛОЩАД НА АВИАЦИЯТА",
    "A0034": "КВ. ГАРА ИСКЪР",
    "A2108": "МЕТРОСТАНЦИЯ ХАДЖИ ДИМИТЪР",
    "TB1394": "БЪЛГАРСКА ТЕЛЕГРАФНА АГЕНЦИЯ",
    "A1725": "ТЕАТЪР СОФИЯ",
    "A1853": "УЛ. БЪЛГАРСКИ ВОИН",
    "A6722": "ул. Ст. Доспевски - временна",
    "A2314": "ХЛЕБОЗАВОДА",
    "A2008": "УЛ. КОСТЕНСКИ ВОДОПАД",
    "A0095": "НЦ ПО РЕХАБИЛИТАЦИЯ",
    "A1024": "МАХАЛА ИВАНЯНЕ - ПО ЖЕЛАНИЕ",
    "A6222": "УЛ. ОДРИН - ВРЕМЕННА",
    "TB6171": "УЛ. ДИМИТЪР ПЕТКОВ - ВРЕМЕННА",
    "TM0271": "БУЛ. АКАД. ИВ. ГЕШОВ",
    "A0068": "РЦ ПО ПРОТЕЗИРАНЕ",
    "A6304": "УЛ. СЪЗНАНИЕ",
    "TM1721": "СЪДЕБНА ПАЛАТА",
    "A0033": "КВ. ГАРА ИСКЪР",
    "A0773": "УЛ. КАЛОЯНОВО",
    "A0816": "КВ. БУСМАНЦИ",
    "A1365": "УЛ. ЦАР ИВАН АСЕН ІІ",
    "A2441": "КАНТОНА",
    "A0716": "СПИИДО ЕООД",
    "A1173": "КВ. СУХОДОЛ",
    "TM2594": "УЛ. ОБИКОЛНА",
    "A0824": "КВ. ВРАЖДЕБНА",
    "A6427": "ВОЕННА АКАДЕМИЯ",
    "TB0073": "БУЛ. ПЕЙО ЯВОРОВ",
    "A0215": "БЛ. 43 Ж.К. ГОРУБЛЯНЕ",
    "A2822": "УЛ. БРЕЗНИК",
    "A1277": "ПЛ. ЛЪВОВ МОСТ",
    "A2671": "МОДНА КЪЩА АГРЕСИЯ",
    "A6511": "КИНО АРЕНА II",
    "TB1806": "УЛ. 6-ТИ СЕПТЕМВРИ",
    "A1082": "МОСТА НА СЕЛСКА РЕКА",
    "A0863": "КВ. МАЛАШЕВЦИ",
    "A6688": "УЛ. РАВНЕЦ - ВРЕМЕННА",
    "M19": "ЦАРИГРАДСКО ШОСЕ",
    "A1176": "ОКОЛОВРЪСТЕН ПЪТ",
    "A2057": "УЛ. НАРОДНО ХОРО",
    "A0984": "УЛ. ВРАЧАНСКА",
    "A6664": "УЛ. ОВЧЕ ПОЛЕ - ВРЕМЕННА",
    "A0073": "БУЛ. ПЕЙО ЯВОРОВ",
    "TB0283": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "A0441": "ВОЕННА АКАДЕМИЯ",
    "A0828": "КВ. ГНИЛЯНЕ",
    "A1133": "НАЧАЛО С. ПЛАНА",
    "A1224": "29-ТИ ДКЦ",
    "TM0357": "БУЛ. ПЕНЧО СЛАВЕЙКОВ",
    "A2478": "УЛ. 5010",
    "A2515": "УЛ. БАБА ТОНКА",
    "A6443": "БЛ. 35 Ж.К. ЗАПАДЕН ПАРК",
    "A2683": "УЛ. ЗАХАРИ ПОПОВ",
    "A1800": "УЛ. ИВАН АНДОНОВ",
    "A6551": "ТЕЛЕВИЗИОННА КУЛА",
    "A2752": "МЕТРОСТАНЦИЯ АКАД. А. Т.- БАЛАН",
    "A2485": "СОФГЕОПРОУЧВАНЕ",
    "A0980": "УЛ. МЕЧА ПОЛЯНА - ПО ЖЕЛАНИЕ",
    "TB2601": "УЛ. ГЕН. ГУРКО",
    "A2269": "КАЛФИН ДОЛ - ПО ЖЕЛАНИЕ",
    "A1390": "УЛ. ВЕНЕРА",
    "TM2055": "УЛ. НАРОДНО ХОРО",
    "A0482": "УЛ. СИНЧЕЦ",
    "A0874": "КВ. ОБЕЛЯ",
    "A1796": "УЛ. ДЕМОКРАЦИЯ",
    "M309": "ПАТРИАРХ ЕВТИМИЙ",
    "A0396": "ХОТЕЛ ХИЛТЪН",
    "A0827": "КВ. ГНИЛЯНЕ",
    "A1746": "ТЕЦ ЛЮЛИН",
    "M15": "АЛЕКСАНДЪР ТЕОДОРОВ-БАЛАН",
    "TB0926": "КИНО ОДЕОН",
    "A6433": "МЕТРОСТАНЦИЯ ВИТОША",
    "A0011": "14-ТИ ДКЦ",
    "A6015": "УЛ. УОШБЪРН",
    "A0122": "БЛ. 1 Ж.К. ГОРУБЛЯНЕ",
    "TB1983": "УЛ. КАП. Д. СПИСАРЕВСКИ",
    "A6034": "УЛ. ЛЕВСКИ ВЕКОВЕН",
    "A6372": "УЛ.АРБАНАСИ",
    "A1968": "УЛ. ЙОСИФ ЩРОСМАЙЕР",
    "A2427": "АВТОСТАНЦИЯ ИЗТОК",
    "A6613": "БУЛ. ВАРДАР",
    "TM2168": "УЛ. САН СТЕФАНО",
    "A0817": "КВ. ВЕРДИКАЛ",
    "A2098": "УЛ. ОСВОБОЖДЕНИЕ",
    "A2340": "ЦЕНТЪР С. ВОЙНЕГОВЦИ",
    "A0977": "УЛ. СРЕБЪРНА",
    "A1784": "ТРЪНСКА МАХАЛА",
    "A1884": "УЛ. ВЪЗРОЖДЕНСКА",
    "A0849": "КВ. КАРПУЗИЦА",
    "TB0587": "Ж.К. БЪКСТОН",
    "A2411": "УЛ. ПРОЛЕТ",
    "A2435": "АВТОСТАНЦИЯ ОБЕЛЯ",
    "TB2093": "УЛ. ОРИОН",
    "A0831": "КВ. ГОРУБЛЯНЕ",
    "A1562": "СЕЛО ЧЕПИНЦИ",
    "A2677": "БЛ. 245 Ж.К. МЛАДОСТ 2",
    "A0883": "КВ. ОРЛАНДОВЦИ",
    "M2": "ЛЮЛИН",
    "A0512": "ЧИТАЛИЩЕ СВ.СВ. КИРИЛ И МЕТОДИЙ",
    "OM_N21": "",
    "A2564": "УЛ. ГЕОРГИ КАРАСЛАВОВ",
    "A1486": "РУДНИКА - ПО ЖЕЛАНИЕ",
    "TB0144": "БЛ. 162 Ж.К. ДРУЖБА-1",
    "A6395": "УЛ. 8-МИ ДЕКЕМВРИ",
    "A2092": "УЛ. ОРИОН",
    "A2423": "УЛ. ФЕЛИКС КАНИЦ",
    "TB2234": "Ул. Хемус",
    "TB1139": "НАЦИОНАЛЕН ДВОРЕЦ НА КУЛТУРАТА",
    "A1653": "ПРОМИШЛЕНА ЗОНА ГОРНИ ЛОЗЕН",
    "A2019": "УЛ. ЛАТИНКА",
    "TB1893": "УЛ. ГЕН. ВЛ. ДИНЧЕВ",
    "TB2131": "УЛ. ПОП ГРУЙО",
    "TM0745": "СОФАРМА АД",
    "A0756": "ИНЖСТРОЙ СОФИЯ АД",
    "A1460": "РАЗСАДНИК МАЛО БУЧИНО",
    "A2399": "ЗАПАДЕН ПАРК",
    "A2757": "УЛ. ДИМИТЪР АШОВ",
    "TB2724": "ПРОМИШЛЕНА ЗОНА Ж.К. ДРУЖБА 1",
    "A0031": "36-ТО СОУ",
    "A1215": "РАЗКЛОНА ЗА ВОЛУЯК",
    "TM1711": "МАГАЗИН ВИТОША",
    "TB0678": "Ж.К. ОБЕЛЯ-2",
    "A1659": "ЕЗЕРОТО",
    "A2382": "СТУДЕНТСКИ ГРАД",
    "A2591": "УЛ. ИНЖ. ГЕОРГИ БЕЛОВ",
    "A6497": "ПЛОЩАД НЕЗАВИСИМОСТ",
    "A0520": "ДЕПО ИСКЪР",
    "A0675": "ОБЩИНА ВРЪБНИЦА",
    "A2052": "УЛ. НАДЕЖДА",
    "TB6029": "УЛ. ОБОРИЩЕ",
    "A2143": "УЛ. ПРОСВЕТА",
    "TB2458": "НИМ",
    "TB1929": "УЛ. ДИМИТЪР ПЕШЕВ",
    "M12": "МУСАГЕНИЦА",
    "A0089": "ТЕХНОТЕСТ",
    "A1956": "УЛ. ЛЮЛЯК",
    "A1758": "ТОЛЕВА МАХАЛА",
    "TM1331": "ЦЕНТРАЛНА ГАРА",
    "A0949": "КМЕТСТВО СВЕТОВРАЧАНЕ",
    "A1285": "ПЛ. МЕГДАНА",
    "A1269": "ПЛ. ВЪЗРАЖДАНЕ",
    "A6240": "МЕТРОСТАНЦИЯ ОБЕЛЯ",
    "A1669": "УЛ. ЛИПА",
    "A2560": "КОКАЛЯНСКО ХАНЧЕ",
    "M21": "ИСКЪРСКО ШОСЕ",
    "TB0576": "СОФИЯ ТЕХ ПАРК",
    "A1646": "НАЧАЛО С. ВОЙНЕГОВЦИ - ПО ЖЕЛАНИЕ",
    "TM1916": "УЛ. ГРОЗДЕН",
    "TB1863": "УЛ. ВАСИЛ КЪНЧЕВ",
    "TB0052": "АВТОСТАНЦИЯ ГЕО МИЛЕВ",
    "TB1913": "УЛ. ГРАФ ИГНАТИЕВ",
    "A2282": "177-МО ОУ",
    "A2228": "УЛ. ХАЙДУТ СИДЕР",
    "TM1776": "ТРИЪГЪЛНИКА",
    "TM2211": "УЛ. Т. КАБЛЕШКОВ",
    "A2756": "МЕТРОСТАНЦИЯ ВИТОША",
    "TB0172": "БЛ. 241 Ж.К. ОБЕЛЯ-2",
    "A2126": "УЛ. ПОП ГРУЙО",
    "TM0470": "ГАРА ПОДУЯНЕ",
    "A0091": "ТЕХНОТЕСТ",
    "A2490": "МЕТРОСТАНЦИЯ БИЗНЕС ПАРК",
    "TB1288": "ПЛ. ОРЛОВ МОСТ",
    "TB2272": "90-ТО СОУ",
    "A1828": "УЛ. ЛАЗАРОПОЛЕ",
    "A1101": "МОСТА НА Р. ИСКЪР КВ. АБДОВИЦА",
    "A1144": "НОВА ВРАЖДЕБНА",
    "A6601": "УЛ. ХИСАРЯ",
    "A6646": "УЛ. 2 - ВРЕМЕННА",
    "TB2379": "ЮЖЕН ПАРК",
    "A0056": "АВТОСТАНЦИЯ ИЗТОК",
    "TM0376": "БУЛ. СЛИВНИЦА",
    "A0937": "КМЕТСТВО ЖЕЛЕЗНИЦА",
    "A2191": "УЛ. СТОЯН МИХАЙЛОВСКИ",
    "TM1050": "МЕТРОСТАНЦИЯ К. ВЕЛИЧКОВ",
    "A0442": "ВОЕННА АКАДЕМИЯ",
    "A0642": "Ж.К. КРЕМИКОВЦИ",
    "A6141": "УЛ. ОМАЯ",
    "A6590": "ГАРА ВЛАДАЯ",
    "A1109": "СТЦ ИНТЕРПРЕД",
    "A1126": "НАЧАЛО КВ. КЛИСУРА",
    "A2234": "УЛ. ХЕМУС",
    "A2755": "МЕТРОСТАНЦИЯ ВИТОША",
    "A0508": "ГРОБИЩЕН ПАРК НЕГОВАН",
    "A2093": "УЛ. ОРИОН",
    "A0777": "УЛ. ИНЖ. ГЕОРГИ БЕЛОВ",
    "A0062": "АВТОСТАНЦИЯ ОРЛАНДОВЦИ",
    "A2279": "64-ТО ОУ",
    "A1081": "МОСТА НА СЕЛСКА РЕКА",
    "A1766": "УЛ. ТРАКИЙСКА МОГИЛА",
    "A0117": "БИОМЕТ ЕООД - ПО ЖЕЛАНИЕ",
    "A0496": "ГОРСКО СТОПАНСТВО",
    "A1206": "ОПИТНА СТАНЦИЯ",
    "TB2409": "ХИПЕРМАРКЕТ ПРАКТИКЕР",
    "A2798": "ЛЕТИЩЕН ЦЕНТЪР СОФИЯ",
    "A6197": "Ж.К.ХАДЖИ ДИМИТЪР",
    "A1628": "БУЛ. КРЪСТЬО ПАСТУХОВ",
    "TB0006": "12-ТИ ДКЦ",
    "A0049": "АВТОРЕМОНТНА БАЗА",
    "A2173": "УЛ. СИНЧЕЦ",
    "A6361": "78-МО СОУ БАНКЯ - ВРЕМЕННА",
    "OM_N22": "",
    "A6167": "УЛ. ОПЪЛЧЕНСКА",
    "TB1406": "28-МИ ПОЩЕНСКИ КЛОН",
    "A2699": "ЗАСЛОН БАЙ КРЪСТЬО - ПО ЖЕЛАНИЕ",
    "A6161": "КВ. МАНАСТИРСКИ ЛИВАДИ",
    "A1771": "ТРД",
    "TB2092": "УЛ. ОРИОН",
    "A0803": "Ж.К. БАНИШОРА",
    "A1558": "СЕЛО СВЕТОВРАЧАНЕ",
    "A0125": "БЛ. 1 Ж.К. ГОРУБЛЯНЕ",
    "TB0357": "БУЛ. ПЕНЧО СЛАВЕЙКОВ",
    "A0206": "МБАЛ ТОКУДА БОЛНИЦА",
    "TB0019": "24-ТИ ДКЦ",
    "A2529": "83-ТО ОУ ПАНЧАРЕВО",
    "A6509": "УЛ. 5006",
    "TB0328": "БУЛ. К. ВЕЛИЧКОВ",
    "A0820": "КВ. ВИДНИТЕ",
    "A1267": "ПЛ. БИРИМИРЦИ",
    "TM1491": "РУМЪНСКО ПОСОЛСТВО",
    "TM1821": "УЛ. АЛЕКО КОНСТАНТИНОВ",
    "A1654": "ПРОМИШЛЕНА ЗОНА",
    "TB2666": "УЛ. ДЖЕРМАН - ПО ЖЕЛАНИЕ",
    "TM1273": "ПЛ. ЖУРНАЛИСТ",
    "A6498": "ПЛОЩАД НЕЗАВИСИМОСТ",
    "A1018": "МАГАЗИН МЕТРО 1",
    "A1522": "СЕЛО ГОРНИ БОГРОВ",
    "A2763": "БУЛ. ЧЕРНИ ВРЪХ",
    "TM2000": "УЛ. КОЗЛОДУЙ",
    "A1397": "ВИСШЕ УЧИЛИЩЕ ПО ТЕЛЕКОМУНИКАЦИИ И ПОЩИ",
    "A6252": "РСП БЗН ВРЕМЕННА",
    "TB0031": "36-ТО СОУ",
    "A0915": "ВОЕННОРЕМОНТЕН ЗАВОД",
    "A2239": "УЛ. ХРИСТО БОТЕВ",
    "A2277": "90-ТО СОУ",
    "TB0302": "БУЛ. ВИТОША",
    "A0310": "БУЛ. ГОЦЕ ДЕЛЧЕВ",
    "A1392": "ПОЛЕТО",
    "TB0022": "ОНКОЛОГИЧЕН ДИСПАНСЕР",
    "A0151": "БЛ. 20 Ж.К. СВОБОДА",
    "A2464": "УЛ. МАК",
    "TB1764": "ПГХМБТ ПРОФ. П. РАЙКОВ",
    "A1798": "КУЛТУРЕН ЦЕНТЪР ЛЮЛИН",
    "TB1994": "УЛ. КЛОКОТНИЦА",
    "A0681": "Ж.К. ОБЕЛЯ-2",
    "A1028": "УЛ. ЛЕСКОВЕЦ",
    "A1047": "МЕТРОСТАНЦИЯ ВАРДАР",
    "A2748": "УЛ. АЛ. СТАМБОЛИЙСКИ",
    "A2386": "УЛ. ОБРАДОВСКИ ПЪТ",
    "A0412": "В.З. КИНОЦЕНТЪР",
    "A2278": "61-ВО ОУ",
    "A2666": "УЛ. ДЖЕРМАН - ПО ЖЕЛАНИЕ",
    "A2693": "СЕЛО ДОЛНИ ЛОЗЕН",
    "A2839": "РЕЗИДЕНШЪЛ ПАРК СОФИЯ",
    "A1019": "МАГАЗИН МЕТРО 2",
    "TM1113": "НАДЛЕЗ НАДЕЖДА",
    "A2740": "МЕТРОСТАНЦИЯ ДРУЖБА",
    "A0721": "УЛ. 5007",
    "A6624": "МЕТРОСТАНЦИЯ ГОРНА БАНЯ",
    "A1875": "УЛ. ВИХРЕН",
    "TM2076": "УЛ. ОДРИН",
    "TM6593": "УЛ. ПОЗИТАНО",
    "A6515": "ПЛ. СТОЧНА ГАРА",
    "A0887": "КВ. ОРЛАНДОВЦИ",
    "TB2739": "МЕТРОСТАНЦИЯ ДРУЖБА",
    "A2499": "УЛ. КАЛО",
    "A0747": "ЗООПАРКА",
    "A2437": "ГРАД БУХОВО",
    "TB0236": "МЕТРОСТАНЦИЯ ДРУЖБА",
    "A0440": "ВОЕННА АКАДЕМИЯ",
    "A0524": "ДЕТСКИ ДОМ",
    "A1976": "УЛ. КАМЕН АНДРЕЕВ",
    "TB2066": "УЛ. ОБИКОЛНА",
    "A0729": "ЗАЛА ФЕСТИВАЛНА",
    "A1885": "УЛ. ВЪЗРОЖДЕНСКА",
    "A2674": "БУЛ. РОЖЕН",
    "A0010": "14-ТИ ДКЦ",
    "A1989": "УЛ. КИРИЛ ХРИСТОВ",
    "A1751": "ПП УСПЕХ",
    "A2100": "УЛ. ОСТРОВО",
    "A2332": "ЦЕНТРАЛЕН ГРОБИЩЕН ПАРК",
    "A0990": "КУЛИНАРЕН К-Т ПЕЙФИЛ",
    "A2686": "ОКОЛОВРЪСТЕН ПЪТ",
    "TM1593": "СОФИЯ МЕД АД",
    "A2651": "УЛ. РАЧО ПЕТКОВ КАЗАНДЖИЯТА",
    "A2667": "УЛ. ДЖЕРМАН - ПО ЖЕЛАНИЕ",
    "TM0878": "КВ. ОВЧА КУПЕЛ",
    "A0489": "ПРОМИШЛЕНА ЗОНА",
    "A1345": "ПК СПАРТАК",
    "A6444": "ПЛ. ОРЛОВ МОСТ",
    "TM0518": "ДЕПО ИСКЪР",
    "A1147": "ГРОБИЩЕН ПАРК МАЛАШЕВЦИ",
    "A6278": "УЛ. КОСТЕНСКИ ВОДОПАД",
    "TM0313": "БУЛ. ГОЦЕ ДЕЛЧЕВ",
    "A6188": "Ул.Костенски водопад",
    "A0121": "БКС СРЕДЕЦ",
    "A6616": "Ж.К. МЛАДОСТ-1",
    "A0549": "СТИНД",
    "A6107": "НДК - ВРЕМЕННА",
    "TM2113": "УЛ. ПИРОТСКА",
    "A1006": "ЛЕТИЩЕ СОФИЯ ТЕРМИНАЛ 1",
    "A0792": "КАНТОНА",
    "A6679": "УЛ. КАЛИМАНЦИ - ВРЕМЕННА",
    "TM1926": "УЛ. ДИМИТЪР ПЕТКОВ",
    "TM2580": "УЛ. КОРАБ ПЛАНИНА",
    "A0107": "ГРОБИЩЕН ПАРК БАНКЯ",
    "A2500": "УЛ. ВИТОША",
    "TB0385": "БУЛ. ХРИСТО БОТЕВ",
    "A0463": "ГАРА ПИОНЕР",
    "A1170": "КВ. СУХОДОЛ",
    "A0064": "АВТОСТАНЦИЯ ХЛАДИЛНИКА",
    "A2795": "СОФИЯ ТЕХ ПАРК",
    "A0718": "КОТЛОСТРОЕНЕ АД",
    "A2402": "УЛ. ВАСИЛ ЛЕВСКИ",
    "A2188": "СЕЛО ПОДГУМЕР",
    "A1201": "УЛ. ВРЪХ МАНЧО",
    "A0382": "БУЛ. ХРИСТО БОТЕВ",
    "A1877": "ЯЗ. СУХОДОЛ",
    "A0919": "КВ. КРЪСТОВА ВАДА",
    "TB0173": "БЛ. 245 Ж.К. ОБЕЛЯ-2",
    "A2310": "ХЛАДИЛЕН ЗАВОД",
    "A2727": "ЦЕНТЪР КВ. ВИДНИТЕ",
    "A1167": "ОВЧАРСКИ КОШАРИ",
    "A1999": "УЛ. КНЯЗ БОРИС I",
    "A6353": "НАДЛЕЗ - ФИЛИПОВСКО ШОСЕ",
    "A1244": "КВ. ПАНЧАРЕВО - ПО ЖЕЛАНИЕ",
    "M13": "МЛАДОСТ 1",
    "TM0011": "14-ТИ ДКЦ",
    "A6592": "УЛ. ИВАН КРАЛИЧА",
    "TM0297": "БУЛ. ВАСИЛ ЛЕВСКИ",
    "A1498": "САМОКОВСКО ШОСЕ",
    "A1601": "СПИ ПРОФ. П. МУТАФЧИЕВ",
    "A0016": "20-ТИ ДКЦ",
    "A0258": "БОЛНИЦАТА С. ПОДГУМЕР",
    "A6149": "ВРЕМЕННА А123",
    "A2345": "ЦЕНТЪР С. ПОДГУМЕР",
    "TM0672": "Ж.К. МОТОПИСТА",
    "A1211": "МОСТА НА БАНИШКА РЕКА - ПО ЖЕЛАНИЕ",
    "A2379": "ЮЖЕН ПАРК",
    "TB0419": "ВМА",
    "A2697": "РАЗКЛОНА ЗА ДРАГАЛЕВСКИ МАНАСТИР - ПО ЖЕЛАНИЕ",
    "A1781": "ТРИЪГЪЛНИКА Ж.К. ЛЮЛИН",
    "A1071": "МИТНИЦА СУХОДОЛ",
    "A1526": "ЦЕНТЪР ДОЛНИ ЛОЗЕН",
    "A6476": "БУЛ.БРЮКСЕЛ",
    "TM2333": "ЦЕНТРАЛЕН ГРОБИЩЕН ПАРК",
    "A1036": "УМБАЛ АЛЕКСАНДРОВСКА",
    "A2776": "ПЪТЕН ВЪЗЕЛ ФИЛИПОВЦИ",
    "A1400": "ПОСОЛСТВАТА",
    "TB0400": "ХОТЕЛ ХИЛТЪН",
    "A6486": "ПГВАД",
    "A6586": "УЛ. ПРОФ. АТАНАС ИШИРКОВ",
    "A6657": "УЛ. ОБЕЛСКИ МОГИЛИ - ВРЕМЕННА",
    "A1334": "ПЛАЖА ПАНЧАРЕВО - ПО ЖЕЛАНИЕ",
    "A2445": "МЕТРОСТАНЦИЯ ОБЕЛЯ",
    "A0044": "8-МИ ДКЦ",
    "TM1837": "УЛ. БЕЛИ ДУНАВ",
    "A6579": "УЛ. ДАМЯНИЦА",
    "A2319": "ХМС",
    "TM0872": "КВ. ОБЕЛЯ",
    "A0861": "ЮЖЕН ПАРК",
    "A6504": "РАЗКЛОНА ЗА КВ. СЕСЛАВЦИ",
    "TB6300": "УМБАЛСМ Пирогов",
    "A1398": "ПОПОВА МАХАЛА",
    "A1846": "УЛ. БОЯН МАГЕСНИК",
    "TM1928": "УЛ. ДИМИТЪР ПЕТКОВ",
    "A0973": "КОРИЯТА",
    "A1913": "УЛ. ГРАФ ИГНАТИЕВ",
    "A2341": "ЦЕНТЪР С. ВОЙНЕГОВЦИ",
    "A0952": "КМЕТСТВО ЛОЗЕН",
    "A1100": "МОСТА НА Р. ИСКЪР",
    "A2407": "ПЪТЯ ЗА ЛЕТИЩЕ ВРАЖДЕБНА",
    "TM2099": "УЛ. ОСВОБОЖДЕНИЕ",
    "TB1878": "УЛ. ВИХРЕН",
    "A0092": "31-ВО СУЧЕМ",
    "A2714": "КОШАРИТЕ",
    "A6232": "Ж.К. ИВАН ВАЗОВ",
    "A0455": "ГАРА КАЗИЧЕНЕ",
    "A1310": "ПЛ. СРЕД СЕЛО",
    "A6157": "МЕТРОСТАНЦИЯ ВАРДАР",
    "A1373": "ПО ЖЕЛАНИЕ",
    "A6384": "КВ. РЕПУБЛИКА",
    "TB0284": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "A1556": "СЕЛО ПЛАНА",
    "TM1720": "СЪДЕБНА ПАЛАТА",
    "TM2160": "УЛ. РИЛСКА ОБИТЕЛ",
    "A2365": "ЧИТАЛИЩЕ Д. ЧИНТУЛОВ",
    "A1794": "УЛ. 3",
    "A2840": "ЗЛАТНИ МОСТОВЕ",
    "TB2353": "НЦ ПО РАДИОБИОЛОГИЯ - ПО ЖЕЛАНИЕ",
    "A0285": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "TM2083": "УЛ. ОПЪЛЧЕНСКА",
    "A6654": "УЛ. 4 - ВРЕМЕННА",
    "A2070": "УЛ. ОБОРИЩЕ",
    "A2272": "90-ТО СОУ",
    "A0745": "СОФАРМА АД",
    "A1553": "КМЕТСТВО МЪРЧАЕВО",
    "A0976": "УЛ. СРЕБЪРНА",
    "A6355": "КВ. ИЗГРЕВ",
    "A0804": "Ж.К. БАНИШОРА",
    "A2009": "УЛ. КОСТЕНСКИ ВОДОПАД",
    "TM1283": "ПЛ. МАКЕДОНИЯ",
    "A1597": "БАЛКАН СТАР",
    "A6095": "УЛ. НИШАВА",
    "TB1939": "УЛ. ДУНАВ",
    "TB0358": "БУЛ. ПЕНЧО СЛАВЕЙКОВ",
    "TB0010": "14-ТИ ДКЦ",
    "A2431": "ЦЕНТЪР КВ.ФИЛИПОВЦИ",
    "A2361": "УЛ. ЛЮЛЯК",
    "A2428": "АВТОСТАНЦИЯ ИЗТОК",
    "A2721": "БУЛ. КОПЕНХАГЕН",
    "TM0581": "КВ. МАНАСТИРСКИ ЛИВАДИ-ЗАПАД",
    "A1702": "ПРОМИШЛЕНА ЗОНА КАЗИЧЕНЕ",
    "A1929": "УЛ. ДИМИТЪР ПЕШЕВ",
    "TB1296": "ПЛ. РУСКИ ПАМЕТНИК",
    "A1540": "СЕЛО КОКАЛЯНЕ",
    "A1716": "ОБЩИНА ЛЮЛИН",
    "TM2254": "УЛ. ЦАРЕВЕЦ",
    "A1541": "СЕЛО КРИВИНА",
    "A1978": "УЛ. КАМЕНОДЕЛСКА",
    "TB2205": "УЛ. ТИМОК",
    "A0638": "Ж.К. КРАСНО СЕЛО",
    "TM1134": "НДК",
    "A6308": "КЕМПИНСКИ ХОТЕЛ ЗОГРАФСКИ",
    "TM0568": "БАЛКАНКАР АД",
    "A2181": "УЛ. СТ. ДОСПЕВСКИ",
    "A1668": "УЛ. ВЕСКО ВЕСЕЛИНОВ",
    "TM2250": "УЛ. ЦАР СИМЕОН",
    "A1145": "НОВА ВРАЖДЕБНА",
    "A0401": "БУЛ. ЧЕРНИ ВРЪХ",
    "A2675": "АВТОСТАНЦИЯ БАНКЯ",
    "A0042": "66-ТО СОУ",
    "A0509": "ГРОБИЩЕН ПАРК ДРАГАЛЕВЦИ",
    "A6116": "РАЗКЛОНА ЗА ЛЕТИЩЕТО",
    "TB1193": "УМБАЛ СВ. АННА",
    "A0322": "БУЛ. ПЕТЪР ДЕРТЛИЕВ",
    "TM0755": "СТОКОВ БАЗАР ИЛИЯНЦИ",
    "A0084": "КОМПЛЕКС НА БАН",
    "TM1298": "Пл. Руски паметник",
    "TB0490": "УЛ. Т. КАБЛЕШКОВ",
    "A1029": "УЛ. ЛЕСКОВЕЦ",
    "TM1305": "ПЛ. РУЧЕЙ",
    "A1627": "СТАРАТА ЧЕШМА - ПО ЖЕЛАНИЕ",
    "A1177": "ОКОЛОВРЪСТЕН ПЪТ",
    "A1066": "МИНАТА - ПО ЖЕЛАНИЕ",
    "A6649": "УЛ. 7 - ВРЕМЕННА",
    "A6656": "УЛ. ОБЕЛСКИ МОГИЛИ - ВРЕМЕННА",
    "TB1324": "ХОТЕЛ ХЕМУС",
    "A6283": "КАТОЛИЧЕСКИ ГРОБИЩЕН ПАРК",
    "TM2241": "УЛ. ХРИСТО СИЛЯНОВ",
    "A1118": "НАЧАЛО С. БИСТРИЦА - ПО ЖЕЛАНИЕ",
    "A0942": "ЧИТАЛИЩЕ ПРОСВЕТА",
    "A1094": "МОСТА НА ВЛАДАЙСКА РЕКА",
    "A1377": "УЛ. БРЕЗИТЕ",
    "A0896": "КВ. РЕПУБЛИКА",
    "A1078": "УЛ. ГРАДИНА",
    "A6393": "ТЕХНИЧЕСКИ УНИВЕРСИТЕТ",
    "A0050": "АВТОСТАНЦИЯ БАНКЯ",
    "A2454": "ЛЕТИЩЕ СОФИЯ ТЕРМИНАЛ 2",
    "A0728": "ЗАГ ДИНАМИК АД",
    "A1848": "УЛ. БУРЯ",
    "A1399": "ПОПОВА МАХАЛА",
    "A1951": "УЛ. ИВАН ЙОСИФОВ",
    "A1200": "УЛ. ВРЪХ МАНЧО",
    "TB0011": "14-ТИ ДКЦ",
    "A1733": "ТЕЛЕВИЗИОННА КУЛА",
    "A0268": "БУЛ. АКАД. ИВ. ГЕШОВ",
    "A2794": "БЛ. 648 Ж.К. ЛЮЛИН 6",
    "A0740": "МЕСТНОСТ КАЛАЙДЖИЙКАТА",
    "A1231": "УЛ. ПАТРИАРХ ДАМЯН",
    "A1850": "УЛ. БЪДЕЩЕ - ПО ЖЕЛАНИЕ",
    "TB2108": "МЕТРОСТАНЦИЯ ХАДЖИ ДИМИТЪР",
    "A1568": "СЕМИНАРИЯТА",
    "A1590": "СКИОРКАТА",
    "A0293": "БУЛ. ВАРДАР",
    "A2791": "РАЗКЛОНА ЗА ЛИФТ СИМЕОНОВО",
    "A6234": "КВ. МАНАСТИРСКИ ЛИВАДИ",
    "A1182": "ОКОЛОВРЪСТЕН ПЪТ",
    "A2702": "ХОТЕЛ ПРОСТОР",
    "A0120": "БКС СРЕДЕЦ",
    "A1779": "ТРИЪГЪЛНИКА КВ. ГОРУБЛЯНЕ",
    "A2430": "Ж.К. ЛЕВСКИ Г",
    "A0603": "Ж.К. ДИАНАБАД",
    "A2209": "УЛ. БОРЯНА",
    "TB0289": "Бул. Братя Бъкстон",
    "A2085": "УЛ. ОПЪЛЧЕНСКА",
    "A0658": "ХОТЕЛ АМБАСАДОР",
    "A0787": "КАНТОНА",
    "A0506": "ГРЕБНА БАЗА - ПО ЖЕЛАНИЕ",
    "TM0380": "УЛ. ХАН КУБРАТ",
    "A2176": "УЛ. СНЕЖНИК",
    "A0688": "Ж.К. СЛАТИНА",
    "A1185": "ОКОЛОВРЪСТЕН ПЪТ - ПО ЖЕЛАНИЕ",
    "TB2171": "УЛ. СВ.СВ. КИРИЛ И МЕТОДИЙ",
    "A1103": "МОСТА ЧАВДАР",
    "A0189": "ТЕХНО ПАРК СОФИЯ",
    "A2325": "ХОТЕЛ ПЛИСКА",
    "A0894": "КВ. РЕПУБЛИКА",
    "A2719": "ВИЛИТЕ",
    "TB2408": "ХИПЕРМАРКЕТ ПРАКТИКЕР",
    "A2044": "УЛ. МАЛОМИР",
    "A2679": "МЕТРОСТАНЦИЯ Г. М. ДИМИТРОВ",
    "TM2130": "УЛ. ПОП ГРУЙО",
    "A1227": "КВ. ФАКУЛТЕТА",
    "A0569": "БАЛКАНКАР АД",
    "A1752": "ПП УСПЕХ",
    "A2035": "УЛ. ЛУИ АЙЕР",
    "A6246": "РЕМОНТНА БАЗА",
    "A0079": "АП ЗЕМЛЯНЕ",
    "A0106": "ГРОБИЩЕН ПАРК БАНКЯ",
    "A1510": "СЕЛИМИЦА",
    "TB1325": "ХОТЕЛ ХЕМУС",
    "TB0804": "Ж.К. БАНИШОРА",
    "A0821": "КВ. ВИТОША",
    "A1544": "СЕЛО КУБРАТОВО",
    "A2288": "160-ТО ОУ",
    "A1583": "БУЛ. ВЛАДИМИР ВАЗОВ",
    "A1371": "ПО ЖЕЛАНИЕ",
    "A2180": "УЛ. СТ. ДОСПЕВСКИ",
    "A6373": "УЛ.АРБАНАСИ",
    "TB0235": "МЕТРОСТАНЦИЯ ДРУЖБА",
    "A0735": "МЕСТНОСТ КАЛАЙДЖИЙКАТА",
    "A1051": "МЕТРОСТАНЦИЯ К. ВЕЛИЧКОВ",
    "TB0366": "БУЛ. ПРАГА",
    "A0766": "ИНСТИТУТ ПО ОРТОПЕДИЯ",
    "A1580": "БУЛ. ВЛАДИМИР ВАЗОВ",
    "TM2068": "МЕТРОСТАНЦИЯ ОВЧА КУПЕЛ",
    "A0600": "Ж.К. ГОЦЕ ДЕЛЧЕВ",
    "A2053": "УЛ. НАРОДЕН ГЕРОЙ",
    "A1163": "БЛ. 21 Ж.К. ЛЮЛИН-1",
    "A2094": "УЛ. ОРФЕЙ",
    "A2370": "ЦЪРКВАТА СВ. ГЕОРГИ",
    "A2759": "ПАРАКЛИС СВ. ВМЧК МИНА - ПО ЖЕЛАНИЕ",
    "TB2736": "Ж.К. ХАДЖИ ДИМИТЪР",
    "A1011": "ЛИФТА СИМЕОНОВО",
    "A1530": "СЕЛО ЖЕЛЕЗНИЦА",
    "A1128": "НАЧАЛО С. КЪТИНА - ПО ЖЕЛАНИЕ",
    "TB0373": "БУЛ. ХРИСТОФОР КОЛУМБ",
    "TB2170": "УЛ. СВ.СВ. КИРИЛ И МЕТОДИЙ",
    "A2040": "УЛ. МАКГАХАН",
    "A2668": "ОБЩИНА БАНКЯ",
    "A1643": "УЛ. РУЧЕЙ",
    "M210": "ЕВРОПЕЙСКИ СЪЮЗ",
    "TM1927": "УЛ. ДИМИТЪР ПЕТКОВ",
    "A0266": "БОЯНСКО ХАНЧЕ",
    "TM1909": "ПЛ. СЛАВЕЙКОВ",
    "A2033": "УЛ. ЛУИ АЙЕР",
    "A2216": "УЛ. ТОДОРИНИ КУКЛИ",
    "A0802": "КВ. АБДОВИЦА",
    "A1044": "МЕТРОСТАНЦИЯ ВАРДАР",
    "A2605": "КМЕТСТВО БОЖУРИЩЕ",
    "A2792": "РАЗКЛОНА ЗА ЛИФТ СИМЕОНОВО",
    "A1124": "НАЧАЛО КВ. ГНИЛЯНЕ",
    "A6503": "РАЗКЛОНА ЗА КВ. СЕСЛАВЦИ",
    "A0303": "МЕТРОСТАНЦИЯ ХАДЖИ ДИМИТЪР",
    "A0696": "ГАРА ЧЕПИНЦИ",
    "TB1301": "ПЛ. РУЧЕЙ",
    "A1641": "ПРОМИШЛЕНА ЗОНА БУСМАНЦИ",
    "A2834": "РАЗКЛОНА ЗА С. ГЕРМАН",
    "A2021": "АВТОСТАНЦИЯ ИЗТОК",
    "A2111": "УЛ. ПИРОТСКА",
    "A2542": "НСБАЛ ПО ОНКОЛОГИЯ",
    "A0576": "СОФИЯ ТЕХ ПАРК",
    "A2703": "ХОТЕЛ ПРОСТОР",
    "A6521": "БЛ. 3 Ж.К. БОТУНЕЦ 1",
    "A0958": "ОБЩИНА БАНКЯ",
    "A2669": "ОБЩИНА БАНКЯ",
    "A2183": "СЕЛО ПОДГУМЕР",
    "TB0599": "Ж.К. ГОЦЕ ДЕЛЧЕВ",
    "TM0306": "БУЛ. ВЪЗКРЕСЕНИЕ",
    "A1023": "МАХАЛА ДЖЮЛНИЦИ",
    "TM1636": "ХЮНДАЙ БЪЛГАРИЯ",
    "A1313": "ПЛ. СТОЧНА ГАРА",
    "A2171": "УЛ. СВ.СВ. КИРИЛ И МЕТОДИЙ",
    "A6402": "ПЛ. СВ. НЕДЕЛЯ",
    "A0561": "ЕЛ. КАНТОН",
    "A1651": "НАЧАЛО С. ГОЛЯНОВЦИ",
    "A0188": "БЛ. 302 Ж.К. НАДЕЖДА-3",
    "A0072": "АЛБЕНА - ПО ЖЕЛАНИЕ",
    "A6546": "ПАРК МУЗЕЙ ВРАНА",
    "A1050": "МЕТРОСТАНЦИЯ К. ВЕЛИЧКОВ",
    "TB2784": "МБАЛ Вита",
    "TM0621": "ГАРА ЗАХАРНА ФАБРИКА",
    "A0920": "УЛ. ЧЕПИНСКО ШОСЕ",
    "A2200": "УЛ. ТЕМЕНУГА - ПО ЖЕЛАНИЕ",
    "A6466": "САТИРИЧЕН ТЕАТЪР",
    "A1600": "СПИ ПРОФ. П. МУТАФЧИЕВ",
    "A2419": "БУЛ. ЦАРИЦА ЙОАНА",
    "A0743": "СОФАРМА АД",
    "A1158": "УЛ. ГЮЕШЕВО",
    "A6109": "ХОТЕЛ ХЕМУС",
    "A2343": "ЦЕНТЪР ПО ХИГИЕНА",
    "A0327": "БУЛ. К. ВЕЛИЧКОВ",
    "A1736": "ПГ ПО ДИЗАЙН ЕЛИСАВЕТА ВАЗОВА",
    "M6": "ОПЪЛЧЕНСКА",
    "A2561": "СПЗ МОДЕРНО ПРЕДГРАДИЕ",
    "A1300": "ПЛ. РУСКИ ПАМЕТНИК",
    "A2225": "УЛ. УЧЕНИЧЕСКА",
    "A6096": "Ж.К. ХИПОДРУМА",
    "A0022": "ОНКОЛОГИЧЕН ДИСПАНСЕР",
    "A0055": "АВТОСТАНЦИЯ ДЪРВЕНИЦА",
    "A1820": "УЛ. АЛЕКСАНДЪР ЖЕНДОВ",
    "A2544": "УЛ. ЛЕСНОВСКА РЕКА",
    "TB2063": "УЛ. НИШАВА",
    "A1389": "УЛ. ВЕНЕРА",
    "TM2075": "УЛ. ОДРИН",
    "A1062": "МЕТРОСТАНЦИЯ СЛИВНИЦА",
    "A0641": "Ж.К. КРАСНО СЕЛО",
    "A1847": "УЛ. БОЯН МАГЕСНИК",
    "TM1490": "РУМЪНСКО ПОСОЛСТВО",
    "A0572": "МЕСТНОСТ ЕЛИТЕ",
    "M204": "ХАН КУБРАТ",
    "A6655": "УЛ. 4 - ВРЕМЕННА",
    "A6721": "ОДЗ - временна",
    "A6038": "МЕТРОСТАНЦИЯ ОВЧА КУПЕЛ",
    "TB1320": "ПЛ. СТОЧНА ГАРА",
    "A6658": "УЛ. ОБЕЛСКИ МОГИЛИ - ВРЕМЕННА",
    "TB0444": "УМБАЛ СВ. ЕКАТЕРИНА",
    "A2718": "ВИЛИТЕ",
    "A2167": "УЛ. ЛУИ ПАСТЬОР",
    "A6564": "БУЛ. БЪЛГАРИЯ",
    "TM6100": "ЖП. ПРЕЛЕЗ",
    "TM0359": "НАДЛЕЗ НАДЕЖДА",
    "A1994": "УЛ. КЛОКОТНИЦА",
    "A1762": "ТПК ХЕМУС",
    "A1411": "28-МИ ПОЩЕНСКИ КЛОН",
    "A2323": "ХОТЕЛ ГОРНА БАНЯ",
    "A1907": "УЛ. Г. БЕНКОВСКИ",
    "A6088": "УЛ. Т. КАБЛЕШКОВ",
    "TM0632": "Ж.К. КРАСНА ПОЛЯНА",
    "A0906": "КВ. ТРЕБИЧ - ПО ЖЕЛАНИЕ",
    "TM2582": "УЛ. ИСКЪРСКО ШОСЕ",
    "A1984": "УЛ. КАП. Д. СПИСАРЕВСКИ",
    "A2099": "УЛ. ОСВОБОЖДЕНИЕ",
    "A0424": "УЛ. ГЕРЕНА",
    "A1372": "ПО ЖЕЛАНИЕ",
    "A6121": "ТЕХНИЧЕСКИ УНИВЕРСИТЕТ",
    "A1555": "СЕЛО НЕГОВАН",
    "TB0275": "БУЛ. АКАД. ИВ. ГЕШОВ",
    "A0623": "Ж.К. ИВАН ВАЗОВ",
    "A2347": "ЦЕНТЪР КВ.ФИЛИПОВЦИ",
    "TB1400": "ПОСОЛСТВАТА",
    "A0444": "УМБАЛ СВ. ЕКАТЕРИНА",
    "A1692": "УНСС",
    "A2706": "ГАРА ГОРНА БАНЯ",
    "TB1282": "Пл. Македония",
    "TB2327": "ХОТЕЛ ПЛИСКА",
    "TM0272": "БУЛ. АКАД. ИВ. ГЕШОВ",
    "A6467": "СЪДЕБНА ПАЛАТА",
    "TB1862": "УЛ. ВАСИЛ КЪНЧЕВ",
    "TM0634": "Ж.К. КРАСНА ПОЛЯНА",
    "A2063": "УЛ. НИШАВА",
    "A2838": "КВ. СИМЕОНОВО",
    "A1031": "УЛ. МОНТЕВИДЕО",
    "A1493": "РУСКО ПОСОЛСТВО",
    "A2802": "МАНАСТИР СВ. АПОСТОЛИ ПЕТЪР И ПАВЕЛ",
    "A0626": "Ж.К. ИВАН ВАЗОВ",
    "A1296": "ПЛ. РУСКИ ПАМЕТНИК",
    "A1054": "МЕТРОСТАНЦИЯ ЛЮЛИН",
    "A1585": "СЕРВИЗ ПОЛМО",
    "A1374": "В.З. МАЛИНОВА ДОЛИНА",
    "TM2475": "КВ. ТРИЪГЪЛНИКА",
    "A1286": "ПЛ. МЕГДАНА",
    "A1931": "УЛ. ДОБРОТИЧ",
    "A2738": "В.З. ПОДГУМЕР - ПО ЖЕЛАНИЕ",
    "A1430": "РАЗКЛОНА ЗА С. ГУРМАЗОВО - ПО ЖЕЛАНИЕ",
    "A6219": "УЛ. РЕПУБЛИКА",
    "A1497": "САМОКОВСКО ШОСЕ",
    "A1183": "ОКОЛОВРЪСТЕН ПЪТ",
    "A1551": "СЕЛО МРАМОР",
    "TM0541": "МОДНА КЪЩА АГРЕСИЯ",
    "TM2078": "УЛ. ОДРИН",
    "A2429": "УЛ. ХРИСТО СИЛЯНОВ",
    "A2289": "СЕЛСКОСТОПАНСКА АКАДЕМИЯ",
    "TM2001": "УЛ. КОЗЛОДУЙ",
    "A2172": "УЛ. СИНЧЕЦ",
    "A6321": "УЛ. ЖЕЛЕЗОПЪТНА",
    "TB2096": "УЛ. ОРЯХОВО",
    "A2787": "УЛ. СОЧИ",
    "A0497": "ГОРУБЛЯНСКО ХАНЧЕ",
    "A2754": "БЛ. 458 Ж.К. МЛАДОСТ 4",
    "A6378": "МЕТРОСТАНЦИЯ БИЗНЕС ПАРК",
    "A6227": "Ж.К. ИВАН ВАЗОВ",
    "TB2114": "УЛ. ПИРОТСКА",
    "A0029": "36-ТИ ПОЩЕНСКИ КЛОН",
    "A0361": "МЕТРОСТАНЦИЯ МИЗИЯ",
    "A0635": "Ж.К. КРАСНА ПОЛЯНА",
    "A1106": "МОТЕЛ БОЖУР",
    "A2080": "УЛ. ОПЪЛЧЕНСКА",
    "A1122": "УЛ. МИНЗУХАР",
    "A0791": "КАНТОНА",
    "A1821": "УЛ. АЛЕКО КОНСТАНТИНОВ",
    "A2524": "78-МО СОУ БАНКЯ",
    "A0204": "БЛ. 39 Ж.К. ГОРУБЛЯНЕ",
    "A0902": "УЛ. ЙОСИФ ЩРОСМАЙЕР",
    "A1513": "СЕЛО БИСТРИЦА",
    "A1793": "УЛ. 2",
    "A2105": "УЛ. ОХРИД",
    "A0785": "УЛ. ДРУЖБА",
    "M205": "МАРИЯ ЛУИЗА",
    "A0586": "Ж.К. БЪКСТОН",
    "A0837": "КВ. ДРАГАЛЕВЦИ",
    "A0052": "АВТОСТАНЦИЯ ГЕО МИЛЕВ",
    "A0422": "УЛ. ЗЛАТЕН ВЕК",
    "TB0209": "БЛ. 41 Ж.К. МЛАДОСТ-1",
    "A6212": "Временна Ул. 24-та",
    "A6035": "АВТОСТАНЦИЯ КНЯЖЕВО",
    "TB1936": "УЛ. ДОЙРАН",
    "A2734": "УЛ. 746 - ПО ЖЕЛАНИЕ",
    "A2256": "УЛ. ЦВЕТНА ГРАДИНА",
    "TM0540": "МОДНА КЪЩА АГРЕСИЯ",
    "TM2337": "ЦЕНТРАЛНИ ХАЛИ",
    "A6216": "ОБЩИНА ВРЪБНИЦА",
    "A2230": "УЛ. ХАН АСПАРУХ",
    "TB0871": "КВ. ОБЕЛЯ",
    "A0166": "143-ТО ОУ",
    "TM0848": "КВ. ИЛИЯНЦИ",
    "A1851": "УЛ. БЪДЕЩЕ - ПО ЖЕЛАНИЕ",
    "A2195": "УЛ. СУЛТАН ТЕПЕ",
    "A1116": "НАЦИОНАЛНА ОПЕРА",
    "A1012": "Ж.К. ДЪРВЕНИЦА",
    "A6472": "УЛ. МИТРОПОЛИТ КИРИЛ ВИДИНСКИ",
    "A1972": "УЛ. КАЛИМАНЦИ",
    "TB2100": "УЛ. ОСТРОВО",
    "A0646": "Ж.К. ЛЕВСКИ Г",
    "A0829": "КВ. ГОРНА БАНЯ",
    "A6493": "УЛ. ИНЖ. ГЕОРГИ БЕЛОВ",
    "A1538": "СЕЛО КАЗИЧЕНЕ",
    "A6485": "УЛ. ПРОФ. ИВАН ГЕОРГОВ",
    "A0983": "УЛ. ВРАЧАНСКА",
    "A1436": "РАБОТИЛНИЦАТА",
    "A6407": "МЕТРОСТАНЦИЯ ОПЪЛЧЕНСКА",
    "A0001": "102-РО ОУ",
    "A0057": "АВТОСТАНЦИЯ ИЗТОК",
    "TB2280": "61-ВО ОУ",
    "A6084": "БУЛ. ИНЖ. ИВАН ИВАНОВ",
    "A1169": "ОКОЛОВРЪСТЕН ПЪТ",
    "A2701": "ГОРСКИ ДОМ",
    "A1253": "ЗАПАДЕН ПАРК",
    "A1463": "Р-Т АЛЕКСАНДРИЯ",
    "A0195": "БЛ. 35 Ж.К. ЗАХАРНА ФАБРИКА",
    "A1325": "ХОТЕЛ ХЕМУС",
    "A6277": "УЛ. КОСТЕНСКИ ВОДОПАД",
    "A1914": "УЛ. ГРАФ ИГНАТИЕВ",
    "A0940": "КМЕТСТВО КАЗИЧЕНЕ",
    "A6358": "УЛ.БЪЛГАРСКИ ВОИН",
    "A0819": "КВ. ВИДНИТЕ",
    "A2835": "РАЗКЛОНА ЗА С. ГЕРМАН",
    "TB2313": "ХЛЕБОЗАВОДА",
    "TM2197": "БЛ. 13 Ж.К.ЗАПАДЕН ПАРК",
    "TB0612": "Ж.К. ДРУЖБА-2",
    "A2243": "УЛ. ЦАР БОРИС ІІІ",
    "A0069": "НБУ",
    "TB0773": "УЛ. КАЛОЯНОВО",
    "A1572": "СЕМИНАРИЯТА",
    "A2657": "В.З. БУНКЕРА-ПО ЖЕЛАНИЕ",
    "A2201": "УЛ. ЦАРИБРОДСКА",
    "TB0017": "20-ТИ ДКЦ",
    "TM1838": "УЛ. БЕЛИ ДУНАВ",
    "A0267": "БОЯНСКО ХАНЧЕ",
    "A6496": "ПЛ. НЕЗАВИСИМОСТ",
    "TM2056": "УЛ. НАРОДНО ХОРО",
    "A0748": "ЗООПАРКА",
    "TB1251": "ЗАПАДЕН ПАРК",
    "A2194": "УЛ. СУЛТАН ТЕПЕ",
    "A1098": "МОСТА НА Р. ИСКЪР",
    "A2409": "ХИПЕРМАРКЕТ ПРАКТИКЕР",
    "SUSt": "Софийски Университет Св. Климент Охридски",
    "SU_B1": "",
    "SU_B4": "",
    "SU_B2": "",
    "SU_B5": "",
    "SU_B3": "",
    "SU_B6": "",
    "SU_E1": "бул. Васил Левски",
    "SU_E2": "пл. Народно събрание",
    "SU_E3": "пл. Народно събрание",
    "SU_E4": "Св. Александър Невски",
    "SU_E5": "СУ Св. Климент Охридски",
    "SU_E6": "бул. Цар Освободител",
    "SU_E7": "Княжеска градина",
    "SU_N1": "",
    "SU_N2": "",
    "SU_N3": "",
    "SU_N4": "",
    "SU_N5": "",
    "SU_N6": "",
    "SU_N7": "",
    "SU_N8": "",
    "SU_N9": "",
    "SU_N10": "",
    "SU_N11": "",
    "SU_N12": "",
    "SU_N13": "",
    "SU_N14": "",
    "OMSt": "Орлов мост",
    "OM_B1": "",
    "OM_B2": "",
    "OM_B3": "",
    "OM_B4": "",
    "OM_B5": "",
    "OM_B6": "",
    "OM_E1": "Княжеска градина",
    "OM_E2": "ул. Сан Стефано",
    "OM_E3": "пл. Орлов мост",
    "OM_E4": "пл. Орлов мост",
    "OM_E5": "Царевецъ",
    "OM_E6": "ул. Иван Асен II",
    "OM_E7": "езеро Ариана",
    "OM_N1": "",
    "OM_N2": "",
    "OM_N3": "",
    "OM_N4": "",
    "OM_N5": "",
    "OM_N6": "",
    "OM_N7": "",
    "OM_N8": "",
    "OM_N9": "",
    "OM_N10": "",
    "OM_N11": "",
    "A2102": "УЛ. ОХРИД",
    "A0323": "БУЛ. КНЯГИНЯ МАРИЯ ЛУИЗА",
    "TB1345": "ПК СПАРТАК",
    "TB2102": "УЛ. ОХРИД",
    "TB0323": "БУЛ. КНЯГИНЯ МАРИЯ ЛУИЗА",
    "A0228": "БЛ. 505 Ж.К. ОВЧА КУПЕЛ-1",
    "A0860": "ЮЖЕН ПАРК",
    "TB6008": "УЛ. ПОДП. КАЛИТИН",
    "A1179": "ОКОЛОВРЪСТЕН ПЪТ",
    "A0438": "КРАЯ С. МРАМОР",
    "TB2089": "УЛ. ОРЕХОВА ГОРА",
    "A2165": "УЛ. РОЯК",
    "A6293": "Бул. Ген. Суворов - временна",
    "A1834": "УЛ. БАЕВИЦА - ПО ЖЕЛАНИЕ",
    "OM_N23": "",
    "OM_N24": "",
    "A0719": "НУ ЗА ТАНЦОВО ИЗКУСТВО",
    "TB0719": "НУ ЗА ТАНЦОВО ИЗКУСТВО",
    "TM0758": "УМБАЛСМ ПИРОГОВ",
    "TB1901": "УЛ. ГЕН. НИКОЛА ЖЕКОВ",
    "A2251": "УЛ. ЦАР СИМЕОН",
    "A2081": "УЛ. ОПЪЛЧЕНСКА",
    "TM0625": "Ж.К. ИВАН ВАЗОВ",
    "TB2352": "ТЕХНИЧЕСКА БИБЛИОТЕКА",
    "TB2081": "Ул. Опълченска",
    "TB2623": "В.З. МАЛИНОВА ДОЛИНА",
    "TB0761": "НСБАЛ ПО ОНКОЛОГИЯ",
    "A0184": "БЛ. 3 Ж.К. ДРУЖБА-1",
    "TB1807": "Ул. 6-ти септември",
    "TB0867": "КВ. МАНАСТИРСКИ ЛИВАДИ",
    "A1194": "УМБАЛ СВ. АННА",
    "TB1194": "УМБАЛ СВ. АННА",
    "TM0108": "БЛ. 539 Ж.К.ЛЮЛИН-5",
    "TB6408": "МЕТРОСТАНЦИЯ ОПЪЛЧЕНСКА",
    "A6734": "ВИЛИТЕ - ВРЕМЕННА",
    "A2607": "УЛ. НАРОДЕН ГЕРОЙ",
    "TB0184": "БЛ. 3 Ж.К. ДРУЖБА-1",
    "TB0494": "РЕПУБЛИКАНСКИ НЕВРОДИСПАНСЕР",
    "TM0929": "ЧИТАЛИЩЕ Д-Р П. БЕРОН",
    "A0398": "БУЛ. ЧЕРНИ ВРЪХ",
    "A6408": "МЕТРОСТАНЦИЯ ОПЪЛЧЕНСКА",
    "A0333": "БУЛ. ЛОМСКО ШОСЕ",
    "A2648": "КИНО АРЕНА ІІ",
    "TB0058": "АВТОСТАНЦИЯ ИЛИЯНЦИ",
    "A0761": "НСБАЛ ПО ОНКОЛОГИЯ",
    "TB0392": "БУЛ. ЦАР БОРИС ІІІ",
    "A2352": "ТЕХНИЧЕСКА БИБЛИОТЕКА",
    "A1949": "УЛ. ЗОГРАФСКИ МАНАСТИР",
    "A2777": "МБАЛ ТОКУДА БОЛНИЦА",
    "A0758": "УМБАЛСМ ПИРОГОВ",
    "A1844": "УЛ. БОРЯНА",
    "A1763": "ПГХМБТ ПРОФ. П. РАЙКОВ",
    "TB0938": "ЦЕНТЪРА КВ. ИЛИЯНЦИ - ПО ЖЕЛАНИЕ",
    "A0108": "БЛ. 539 Ж.К.ЛЮЛИН-5",
    "TB0466": "ГАРА ПОДУЯНЕ",
    "TB2059": "УЛ. Н. КОПЕРНИК",
    "TB0720": "НУ ЗА ТАНЦОВО ИЗКУСТВО",
    "TB1895": "УЛ. ГЕН. НИКОЛА ГЕНЕВ",
    "TM0399": "Хотел Хилтън",
    "A2625": "БИЗНЕС ПАРК СОФИЯ",
    "A0291": "БУЛ. БЪЛГАРИЯ",
    "TB2541": "УЛ. ЛЮБЛЯНА",
    "TB2664": "БУЛ. РОЖЕН",
    "A2474": "НОВА ТЕЛЕВИЗИЯ",
    "TB0539": "МОДНА КЪЩА АГРЕСИЯ",
    "A0058": "АВТОСТАНЦИЯ ИЛИЯНЦИ",
    "TB0333": "БУЛ. ЛОМСКО ШОСЕ",
    "A1940": "УЛ. ДУНАВ",
    "TB1272": "ПЛ. ВЪЗРАЖДАНЕ",
    "A0002": "102-РО ОУ",
    "TB1328": "ЦЕНТРАЛНА ГАРА",
    "TB2574": "УЛ. ГЕН. НИКОЛА ЖЕКОВ",
    "TB1935": "УЛ. ДОЙРАН",
    "TB0608": "Ж.К. ДРУЖБА-1",
    "A1328": "ЦЕНТРАЛНА ГАРА",
    "TM2251": "УЛ. ЦАР СИМЕОН",
    "TB1844": "УЛ. БОРЯНА",
    "TB0280": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "TB0291": "БУЛ. БЪЛГАРИЯ",
    "A0392": "БУЛ. ЦАР БОРИС ІІІ",
    "A1073": "МЛАДЕЖКИ ТЕАТЪР",
    "TB0398": "Бул. Черни връх",
    "TB0755": "СТОКОВ БАЗАР ИЛИЯНЦИ",
    "A0280": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "TB2648": "КИНО АРЕНА ІІ",
    "TB1698": "СУ СВ.КЛИМЕНТ ОХРИДСКИ",
    "TB2607": "УЛ. НАРОДЕН ГЕРОЙ",
    "A2623": "В.З. МАЛИНОВА ДОЛИНА",
    "A0499": "ГОРУБЛЯНСКО ХАНЧЕ",
    "A0067": "НБУ",
    "TB1890": "УЛ. ГЕН. АС. НИКОЛОВ",
    "TB1700": "СУ СВ.КЛИМЕНТ ОХРИДСКИ",
    "TB1411": "28-МИ ПОЩЕНСКИ КЛОН",
    "TB0067": "НБУ",
    "A0231": "БЛ. 527 Ж.К. ОВЧА КУПЕЛ-1",
    "TB2041": "ул. Макгахан",
    "TB1464": "НИМ",
    "A0174": "БЛ. 25 Ж.К. ОВЧА КУПЕЛ-2",
    "TB2474": "НОВА ТЕЛЕВИЗИЯ",
    "A0494": "РЕПУБЛИКАНСКИ НЕВРОДИСПАНСЕР",
    "A2578": "102-РО ОУ",
    "TM0002": "102-РО ОУ",
    "TM2578": "102-РО ОУ",
    "TB0002": "102-РО ОУ",
    "M8": "Софийски Университет Св. Климент Охридски",
    "A0608": "Ж.К. ДРУЖБА-1",
    "TB0968": "ПАЗАРА МЛАДОСТ 1",
    "A0968": "ПАЗАРА МЛАДОСТ 1",
    "TM1940": "УЛ. ДУНАВ",
    "TB6226": "Ж.К. МОТОПИСТА",
    "A0098": "НЦ ПО РЕХАБИЛИТАЦИЯ",
    "TB0399": "ХОТЕЛ ХИЛТЪН",
    "A2037": "УЛ. ЛЮБЛЯНА",
    "A0864": "УЛ. БИСТРИШКО ШОСЕ",
    "A2036": "УЛ. ЛУИ АЙЕР",
    "TB0864": "УЛ. БИСТРИШКО ШОСЕ",
    "A2041": "Ул. Макгахан",
    "A2601": "УЛ. ГЕН. ГУРКО",
    "A1934": "УЛ. ДОБРОТИЧ",
    "A1629": "БУЛ. КРЪСТЬО ПАСТУХОВ",
    "A2574": "УЛ. ГЕН. НИКОЛА ЖЕКОВ",
    "TB2037": "УЛ. ЛЮБЛЯНА",
    "A0170": "МЕТРОСТАНЦИЯ АЛ. МАЛИНОВ",
    "TB0222": "БЛ. 458 Ж.К. НАДЕЖДА-4",
    "A0466": "ГАРА ПОДУЯНЕ",
    "TB0174": "БЛ. 25 Ж.К. ОВЧА КУПЕЛ-2",
    "TB2625": "БИЗНЕС ПАРК СОФИЯ",
    "TB2608": "АВТОСТАНЦИЯ ИЛИЯНЦИ",
    "A0604": "Ж.К. ДРУЖБА-1",
    "A2664": "БУЛ. РОЖЕН",
    "A1895": "УЛ. ГЕН. НИКОЛА ГЕНЕВ",
    "A2541": "УЛ. ЛЮБЛЯНА",
    "TB0631": "Ж.К. КРАСНА ПОЛЯНА",
    "TB0170": "МЕТРОСТАНЦИЯ АЛ. МАЛИНОВ",
    "A1567": "ПГСС",
    "A1700": "СУ СВ.КЛИМЕНТ ОХРИДСКИ",
    "TB0053": "АВТОСТАНЦИЯ ГЕО МИЛЕВ",
    "A0631": "Ж.К. КРАСНА ПОЛЯНА",
    "A2227": "УЛ. ХАЙДУТ СИДЕР",
    "A2060": "УЛ. Н. КОПЕРНИК",
    "TB1934": "УЛ. ДОБРОТИЧ",
    "TB2396": "143-ТО ОУ",
    "TM1073": "Младежки театър",
    "TM2227": "УЛ. ХАЙДУТ СИДЕР",
    "A2396": "143-ТО ОУ",
    "TM0648": "Ж.К. ЛОЗЕНЕЦ",
    "A1890": "УЛ. ГЕН. АС. НИКОЛОВ",
    "TB1629": "БУЛ. КРЪСТЬО ПАСТУХОВ",
    "A0801": "Ж.К. ДЪРВЕНИЦА",
    "TM1901": "УЛ. ГЕН. НИКОЛА ЖЕКОВ",
    "A2131": "УЛ. ПОП ГРУЙО",
    "TB1870": "УЛ. ВИТИНЯ",
    "A2354": "НЦ ПО РАДИОБИОЛОГИЯ - ПО ЖЕЛАНИЕ",
    "TB1740": "ТЕХНИЧЕСКИ УНИВЕРСИТЕТ",
    "M308": "Орлов мост",
    "A6620": "МАХАЛА ЛЪКАТА - ПО ЖЕЛАНИЕ",
    "A2842": "УЛ. МИЛЕНИУМ",
    "A1689": "МОСТА НА ЛЕСНОВСКА РЕКА",
    "A0867": "КВ. МАНАСТИРСКИ ЛИВАДИ",
    "TB1135": "НДК",
    "TM1824": "УЛ. АЛЕКСАНДЪР ЖЕНДОВ",
    "A1361": "МАХАЛА ЧЕМЕРНИК - ПО ЖЕЛАНИЕ",
    "A0053": "АВТОСТАНЦИЯ ГЕО МИЛЕВ",
    "TB0191": "БЛ. 32 Ж.К. ОВЧА КУПЕЛ-2",
    "A6736": "Площад Васил Левски-временна",
    "TB1311": "ПЛ. СТОЧНА ГАРА",
    "A1406": "28-МИ ПОЩЕНСКИ КЛОН",
    "A2608": "АВТОСТАНЦИЯ ИЛИЯНЦИ",
    "TM0019": "24-ТИ ДКЦ",
    "TB0414": "ОКОЛОВРЪСТЕН ПЪТ",
    "A1266": "ПЛ. БИРИМИРЦИ",
    "TB0084": "КОМПЛЕКС НА БАН",
    "A1414": "ПРЕЧИСТВАТЕЛНА СТАНЦИЯ - ПО ЖЕЛАНИЕ",
    "TB1259": "ПЛОЩАД НА АВИАЦИЯТА",
    "A0521": "УЛ. 10-ТИ КИЛОМЕТЪР - ПО ЖЕЛАНИЕ",
    "TM0285": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "A1407": "ПОЩА КВ. КУРИЛО - ПО ЖЕЛАНИЕ",
    "A6578": "УЛ.ПОЛК.СТОЙНО БАЧИЙСКИ",
    "A6622": "УЛ. КНЯЖЕВСКА",
    "TB0318": "БУЛ. Д-Р Г. М. ДИМИТРОВ",
    "TB0041": "66-ТО СОУ",
    "A2830": "БУЛ. КН. МАРИЯ ЛУИЗА",
    "A0230": "БЛ. 519 Ж.К. ОВЧА КУПЕЛ-1",
    "A6819": "УЛ. МАЕСТРО АТАНАСОВ - ВРЕМЕННА",
    "TM1322": "Хотел Хемус",
    "A0131": "БЛ. 11 Ж.К. ЛЕВСКИ-В",
    "TB2050": "УЛ. МОНТЕВИДЕО",
    "A6460": "МЕТРОСТАНЦИЯ К. ВЕЛИЧКОВ",
    "A2731": "УЛ. РОДОПСКИ ИЗВОР",
    "A6690": "УЛ. ЛЕШНИКОВА ГОРА - ВРЕМЕННА",
    "A1074": "МЛАДЕЖКИ ТЕАТЪР",
    "A1941": "УЛ. ДУНАВ",
    "A6272": "Ул. Кирил Дрангов-временна",
    "TM0348": "Бул. Никола Петков",
    "A2827": "МЕТРОСТАНЦИЯ СУ СВ. КЛИМЕНТ ОХРИДСКИ",
    "TB6731": "СК ЦСКА - ВРЕМЕННА",
    "TB6225": "ВМА - ВРЕМЕННА",
    "TB0121": "БКС СРЕДЕЦ",
    "TB0047": "АВТОБАЗА ИСКЪР",
    "TB6710": "АВТОСТАНЦИЯ ГЕО МИЛЕВ",
    "TB2074": "УЛ. ОБОРИЩЕ",
    "TB2258": "УЛ. ЧЕРКОВНА",
    "TB2270": "129-ТО ОУ",
    "TB0687": "Ж.К. СЛАТИНА",
    "TB2014": "ВТУ Т. КАБЛЕШКОВ",
    "TB0120": "БКС СРЕДЕЦ",
    "TB2548": "АВТОБАЗА ИСКЪР",
    "TB0092": "31-ВО СУЧЕМ",
    "TB1697": "СУ СВ.КЛИМЕНТ ОХРИДСКИ",
    "TB0442": "ВОЕННА АКАДЕМИЯ",
    "TB6591": "РУМЪНСКО ПОСОЛСТВО",
    "TB2259": "УЛ. ЧЕРКОВНА",
    "TB2271": "129-ТО ОУ",
    "TB1586": "СЛАТИНА БУЛГАРПЛОД",
    "TB0441": "ВОЕННА АКАДЕМИЯ",
    "TB6709": "АВТОСТАНЦИЯ ГЕО МИЛЕВ",
    "TB6194": "ВМА-временна",
    "TB1587": "СЛАТИНА БУЛГАРПЛОД",
    "TB1317": "Пл.Сточна Гара",
    "A2614": "МАНАСТИР СВ. ТЕОДОР СТРАТИЛАТ - ПО ЖЕЛАНИЕ",
    "TB0928": "ЧИТАЛИЩЕ Д-Р П. БЕРОН",
    "TB0690": "Ж.К. СЛАТИНА",
    "A6740": "32 СУ СВ. КЛИМЕНТ ОХРИДСКИ",
    "TB2013": "ВТУ Т. КАБЛЕШКОВ",
    "TM1074": "МЛАДЕЖКИ ТЕАТЪР",
    "A0040": "60-ТО ОУ",
    "A1263": "ПЕТТЕ МОГИЛИ",
    "TB0727": "ЗАГ ДИНАМИК АД",
    "A1607": "СК НА НСА - ПО ЖЕЛАНИЕ",
    "A6091": "ПЛ. ВЪЗРАЖДАНЕ",
    "TM0590": "Ж.к. Бъкстон",
    "A1380": "НАЧАЛО КВ. ИВАНЯНЕ - ПО ЖЕЛАНИЕ",
    "A0436": "УЛ. 12 - ПО ЖЕЛАНИЕ",
    "A2028": "УЛ. ЛИВАДИТЕ - ПО ЖЕЛАНИЕ",
    "TB0255": "БЛ. 89 Ж.К. ДРУЖБА-1",
    "A1067": "МИНАТА - ПО ЖЕЛАНИЕ",
    "TM2240": "УЛ. ХРИСТО СИЛЯНОВ",
    "TM2017": "УЛ. КУБАДИН",
    "A2252": "УЛ. ЦАР СИМЕОН",
    "A1428": "РАЗКЛОНА ЗА С. ГЕРМАН - ПО ЖЕЛАНИЕ",
    "A0253": "БЛ. 715 Ж.К. ЛЮЛИН-7",
    "A6389": "Кв.Вердикал(ул.Окол)",
    "TB2220": "УЛ. ТЪРНОВО",
    "A0855": "КВ. КРЕМИКОВЦИ",
    "A0662": "Ж.К. МЛАДОСТ-1",
    "A0538": "ПРОИЗВОДСТВЕН К-С МОСКВА",
    "TM1280": "Пл. Македония",
    "A1707": "СУПЕРМАРКЕТ",
    "A1925": "УЛ. ДИМИТЪР ПЕТКОВ",
    "A6585": "Читалище Светлина, с.Владая",
    "A6627": "УЛ. 759",
    "A0132": "БЛ. 111 Ж.К. ОБЕЛЯ-1",
    "A1434": "РАЗКЛОНА ЗА ЛЕТИЩЕТО",
    "TB6088": "УЛ. Т. КАБЛЕШКОВ",
    "A1134": "НДК",
    "A6102": "ПЛ. ВАСИЛ ЛЕВСКИ",
    "TM0618": "Ж.К. ЗАПАДЕН ПАРК",
    "TM2212": "УЛ. Т. КАБЛЕШКОВ",
    "A0103": "БАРАЖА ЯЗ. ПАСАРЕЛ - ПО ЖЕЛАНИЕ",
    "TB0499": "ГОРУБЛЯНСКО ХАНЧЕ",
    "A0155": "БЛ. 202 Ж.К. МЛАДОСТ-2",
    "A6732": "ПЛ. СВ. АЛЕКСАНДЪР НЕВСКИ",
    "A0081": "АП МАЛАШЕВЦИ",
    "A0207": "БЛ. 402 Ж.К. ЛЮЛИН-4",
    "A1835": "УЛ. БАЙКАЛ",
    "A1891": "УЛ. ГЕН. АС. НИКОЛОВ",
    "A2627": "ГРОБИЩЕН ПАРК КРЕМИКОВЦИ - ПО ЖЕЛАНИЕ",
    "TB2079": "УЛ. ОПЪЛЧЕНСКА",
    "A0953": "КМЕТСТВО ДОЛНИ БОГРОВ",
    "A1164": "БЛ. 21 Ж.К. ЛЮЛИН-1",
    "A2534": "УЛ. ЛИПА",
    "A0339": "БУЛ. ГЕН.Н.СТОЛЕТОВ",
    "A2059": "УЛ. Н. КОПЕРНИК",
    "A1425": "НАЧАЛО КВ. МИХАЙЛОВО - ПО ЖЕЛАНИЕ",
    "A2175": "УЛ. СИНЯТА СКАЛА - ПО ЖЕЛАНИЕ",
    "A6271": "Ул. Кирил Дрангов-временна",
    "A0244": "БЛ. 630 Ж.К. ЛЮЛИН-6",
    "A6648": "Кв. Обеля",
    "TM0001": "102-РО ОУ",
    "TB2763": "БУЛ. ЧЕРНИ ВРЪХ",
    "A1339": "ПЛОЩАДА С. ВОЛУЯК",
    "A1505": "МОСТА НА СУХОДОЛСКА РЕКА",
    "A1021": "МАНАСТИР СВ. МИНА",
    "A1478": "МОСТА С. ДОЛНИ БОГРОВ",
    "A6257": "Разклона за ж.к.Младост-временна",
    "A1955": "УЛ. ИВАН ЩЕРЕВ",
    "A1445": "РАЗКЛОНА ЗА МАГИСТРАЛА ХЕМУС",
    "A0689": "Ж.К. СЛАТИНА",
    "A6085": "32 СУ СВ. КЛИМЕНТ ОХРИДСКИ",
    "TB2754": "БЛ. 458 Ж.К. МЛАДОСТ 4",
    "A1927": "УЛ. ДИМИТЪР ПЕТКОВ",
    "A6617": "МАХАЛА ЛЪКАТА - ПО ЖЕЛАНИЕ",
    "A6137": "Ж.К. БЪКСТОН",
    "A1141": "НЕСТЛЕ-СОФИЯ",
    "A6650": "Ул. 9 - временна",
    "TB2821": "Метростанция Хаджи Димитър",
    "A0612": "Ж.К. ДРУЖБА-2",
    "A0800": "Ж.К. ДЪРВЕНИЦА",
    "A6350": "ПЛ. ВЕЛЧОВА ЗАВЕРА",
    "A2743": "УЛ. ЗАХАРИ ЗОГРАФ",
    "A1473": "РЕСТОРАНТ ЗЛАТНАТА РИБКА - ПО ЖЕЛАНИЕ",
    "TB2071": "УЛ. ОБОРИЩЕ",
    "TB1326": "УЛ. КЛОКОТНИЦА",
    "A1247": "ПАРК СВ.СВ.ПЕТЪР И ПАВЕЛ",
    "A1350": "МОСТА НА СИМЕОНОВСКА РЕКА",
    "A6193": "Бл.214, ж.к.Борово-временна",
    "TB1265": "ПЛ. АЛ.НЕВСКИ",
    "TB0057": "АВТОСТАНЦИЯ ИЗТОК",
    "TB6037": "МЕТРОСТАНЦИЯ ОВЧА КУПЕЛ",
    "A2486": "БЛ. 323 Ж.К. МЛАДОСТ-3",
    "A0160": "БЛ. 206 Ж.К. ДРУЖБА-2",
    "A6710": "АВТОСТАНЦИЯ ГЕО МИЛЕВ",
    "TB0028": "28-МИ ДКЦ",
    "A0420": "ВЕЦ ПАСАРЕЛ - ПО ЖЕЛАНИЕ",
    "TB0682": "Ж.К. ОВЧА КУПЕЛ-2",
    "A0967": "МАХАЛА КОНЬОВИЦА",
    "A0944": "ОБЩИНА МЛАДОСТ",
    "TM2579": "Ж.К. ВРЪБНИЦА 1",
    "A1803": "УЛ. 5",
    "TB0610": "Ж.К. ЛЮЛИН-3 - УХО",
    "A0243": "БЛ. 630 Ж.К. ЛЮЛИН-6",
    "TB1489": "РУМЪНСКО ПОСОЛСТВО",
    "A1708": "ПАЗАРА ОВЧА КУПЕЛ",
    "TM0012": "140-ТО СОУ",
    "A0194": "БЛ. 327 Ж.К. МЛАДОСТ-3",
    "A6152": "Временна 10 ТМ",
    "A0407": "МАНАСТИР СВ. ЙОАКИМ И АНА - ПО ЖЕЛАНИЕ",
    "TB2797": "ЛЕТИЩЕН ЦЕНТЪР СОФИЯ",
    "TB2848": "НАЦИОНАЛНА ФУТБОЛНА  БАЗА",
    "A1487": "БЛ. 405 Ж.К. ДРУЖБА-2",
    "TB2053": "УЛ. НАРОДЕН ГЕРОЙ",
    "A0038": "53-ТО ОУ",
    "A0414": "ОКОЛОВРЪСТЕН ПЪТ",
    "A2745": "СКЛАДОВА БАЗА ЧЕПИНСКО ШОСЕ",
    "A0260": "УЛ. ЗЕЛЕН БОР - ПО ЖЕЛАНИЕ",
    "TB6670": "46-ТО ОУ КОНСТАНТИН ФОТИНОВ",
    "TB2101": "УЛ. ОХРИД",
    "A1235": "ЦЕНТЪР КВ. БОТУНЕЦ",
    "A2395": "БЛ. 480 Ж.К. МЛАДОСТ-4",
    "A0369": "БУЛ. ПРОФ.ЦВ. ЛАЗАРОВ",
    "TM0574": "Енергопроект",
    "TB2861": "ж.к. Гоце Делчев",
    "A6440": "ПЛ. СВ. АЛЕКСАНДЪР НЕВСКИ",
    "A0083": "УЛ. Н. ХРЕЛКОВ",
    "TM2228": "УЛ. ХАЙДУТ СИДЕР",
    "A1818": "УЛ. 9 - ПО ЖЕЛАНИЕ",
    "A0255": "БЛ. 89 Ж.К. ДРУЖБА-1",
    "TB2755": "МЕТРОСТАНЦИЯ ВИТОША",
    "A0217": "БЛ. 43 Ж.К. МЛАДОСТ-1",
    "A1892": "УЛ. ГЕН. ВЛ. ДИНЧЕВ",
    "TM0622": "Гара Захарна фабрика",
    "TM6238": "ХЮНДАЙ БЪЛГАРИЯ",
    "TB1289": "ПЛ. ОРЛОВ МОСТ",
    "A6401": "ПЛ. ВЪЗРАЖДАНЕ",
    "A6575": "УЛ. ПЪРВА БЪЛГАРСКА АРМИЯ",
    "A0733": "УЛ. 44 - ПО ЖЕЛАНИЕ",
    "A0247": "БЛ. 69 Ж.К. МЛАДОСТ-1",
    "TB2804": "БУЛ. АКАД. ИВ. ЕВ. ГЕШОВ",
    "A2013": "ВТУ Т. КАБЛЕШКОВ",
    "A2747": "НАЧАЛО КВ. ГНИЛЯНЕ",
    "TB0252": "БЛ. 706 Ж.К. ЛЮЛИН-7",
    "A1543": "СЕЛО КУБРАТОВО",
    "A0147": "БЛ. 170 Ж.К. ДРУЖБА-1",
    "A6692": "Ул. Планинец - временна",
    "A0118": "109-ТО ОУ",
    "A1833": "УЛ. БАЕВИЦА - ПО ЖЕЛАНИЕ",
    "A0574": "ЕНЕРГОПРОЕКТ",
    "A2004": "УЛ. КОНСТАНТИН БОДИН",
    "A2045": "УЛ. МИЛИН КАМЪК",
    "A2649": "МЕСТНОСТ КАМБАНИТЕ",
    "A1602": "ВИК - ЕАД - СОФИЯ",
    "TB1608": "СК НА НСА - ПО ЖЕЛАНИЕ",
    "TM1269": "ПЛ. ВЪЗРАЖДАНЕ",
    "TB0094": "НЦ ПО РЕХАБИЛИТАЦИЯ",
    "A1605": "НПМГ АКАД. ЛЮБОМИР ЧАКАЛОВ",
    "A0996": "КЪМПИНГА - ПО ЖЕЛАНИЕ",
    "A1588": "БЛ.49 Ж.К.ДЪРВЕНИЦА",
    "A6670": "46-ТО ОУ КОНСТАНТИН ФОТИНОВ",
    "A1085": "МОСТА НА Р. СТУБЕЛА",
    "TB0745": "СОФАРМА АД",
    "A6381": "Ж.К. МЛАДОСТ-4",
    "TB2798": "ЛЕТИЩЕН ЦЕНТЪР СОФИЯ",
    "A1104": "МОТЕЛ БЕЛИ БРЕЗИ - ПО ЖЕЛАНИЕ",
    "A2684": "УЛ. ЗАХАРИ ПОПОВ",
    "A6739": "УЛ. НАУМ - ВРЕМЕННА",
    "TB1972": "УЛ. КАЛИМАНЦИ",
    "TM0060": "МЕТРОСТАНЦИЯ ОБЕЛЯ",
    "TM1276": "ПЛ. ЛЪВОВ МОСТ",
    "A1117": "НАЧАЛО С. БИСТРИЦА - ПО ЖЕЛАНИЕ",
    "TB1396": "ВИСШЕ УЧИЛИЩЕ ПО ТЕЛЕКОМУНИКАЦИИ И ПОЩИ",
    "A6712": "БУЛ. КОНСТАНТИН ВЕЛИЧКОВ - ВРЕМЕННА",
    "A0311": "БУЛ. ГОЦЕ ДЕЛЧЕВ",
    "A0413": "В.З. КИНОЦЕНТЪР",
    "A0946": "ОБЩИНА МЛАДОСТ",
    "TB2060": "УЛ. Н. КОПЕРНИК",
    "A0845": "КВ. ИЗГРЕВ",
    "A2779": "АДЖИБАДЕМ СИТИ КЛИНИК",
    "TB2616": "СПОРТНА ЗАЛА  АРЕНА АРМЕЕЦ",
    "TM1247": "ПАРК СВ.СВ.ПЕТЪР И ПАВЕЛ",
    "TM6103": "ПЛ. ВАСИЛ ЛЕВСКИ",
    "A1149": "УЛ. ЮРИЙ ГАГАРИН",
    "A6704": "УЛ.ПАПРАТ - ВРЕМЕННА",
    "A1921": "УЛ. ДАМЕ ГРУЕВ - ПО ЖЕЛАНИЕ",
    "A1864": "УЛ. ВАСИЛ ПЕТЛЕШКОВ",
    "TB0890": "Кв. Павлово",
    "TM0575": "Енергопроект",
    "A1439": "РЕЗИДЕНЦИЯ БОЯНА - ПО ЖЕЛАНИЕ",
    "A1644": "ПРОМИШЛЕНА ЗОНА - ПО ЖЕЛАНИЕ",
    "A2637": "156 - ТО ОУ",
    "A6728": "УЛ. БИЛЯНИНИ ИЗВОРИ",
    "TB0426": "КВ. МАЛИНОВА ДОЛИНА - ПО ЖЕЛАНИЕ",
    "A2027": "УЛ. ЛИВАДИТЕ - ПО ЖЕЛАНИЕ",
    "A2844": "УЛ. ПЕРУНИКА",
    "TB2453": "РВД",
    "A2377": "ШУМАКО - ПО ЖЕЛАНИЕ",
    "A6628": "ЧИТАЛИЩЕ ПРОСВЕТА",
    "TB0553": "ДЪРЖАВНА ПЕЧАТНИЦА",
    "A1589": "БЛ.49 Ж.К.ДЪРВЕНИЦА",
    "TB2834": "РАЗКЛОНА ЗА С. ГЕРМАН",
    "TB0486": "86-ти пощенски клон",
    "TB1429": "РАЗКЛОНА ЗА С. ГЕРМАН",
    "TM0059": "МЕТРОСТАНЦИЯ ОБЕЛЯ",
    "TM0319": "Бул. Джавахарлал Неру",
    "TM0374": "Модна къща Агресия",
    "A0698": "СОФИЯ МЕЛ АД",
    "TB1033": "МГУ СВ.ИВ.РИЛСКИ",
    "A1427": "РАЗКЛОНА ЗА С. ГЕРМАН - ПО ЖЕЛАНИЕ",
    "TB2208": "УЛ. Т. КАБЛЕШКОВ",
    "A0144": "БЛ. 162 Ж.К. ДРУЖБА-1",
    "A1790": "УЛ. 10",
    "TB2454": "ЛЕТИЩЕ СОФИЯ ТЕРМИНАЛ 2",
    "A0877": "КВ. ОВЧА КУПЕЛ",
    "A1346": "МОСТА НА СИМЕОНОВСКА РЕКА",
    "A1595": "УЛ. ТИНТЯВА",
    "TB0253": "БЛ. 715 Ж.К. ЛЮЛИН-7",
    "A0795": "КАРИЕРАТА - ПО ЖЕЛАНИЕ",
    "A1615": "СТАДИОН Г. АСПАРУХОВ",
    "A6691": "Ул. Дамяница - временна",
    "TB0974": "София мед АД",
    "A6215": "БУЛ. ЛОМСКО ШОСЕ",
    "A0539": "МОДНА КЪЩА АГРЕСИЯ",
    "A2267": "МАНАСТИР СВ. НИКОЛА - ПО ЖЕЛАНИЕ",
    "TB1090": "МОСТА ЗА РЕЗИДЕНЦИЯ ВРАНА",
    "A2540": "УЛ. БЕСАРАБИЯ",
    "A2375": "ШЛАКОВ ЦЕХ",
    "A2571": "БЛ. 38 Ж.К. ЗАПАДЕН ПАРК",
    "A6453": "МЕТРОСТАНЦИЯ ВАРДАР",
    "A2628": "ГРОБИЩЕН ПАРК КРЕМИКОВЦИ - ПО ЖЕЛАНИЕ",
    "A6217": "УЛ. ЦАРЕВЕЦ",
    "A6730": "ЖК ЗАХАРНА ФАБРИКА",
    "TB0034": "КВ. ГАРА ИСКЪР",
    "TB1725": "ТЕАТЪР СОФИЯ",
    "A2290": "СЕЛСКОСТОПАНСКА АКАДЕМИЯ",
    "TB1963": "УЛ. ИСКЪР",
    "A1471": "ВОДЕНИЧАРСКИ МЕХАНИ-ПО ЖЕЛАНИЕ",
    "A0710": "Ж.П.ПРЕЛЕЗ",
    "A1459": "РАЗКЛОНА ЗА КВ.СИМЕОНОВО",
    "TB2033": "УЛ. ЛУИ АЙЕР",
    "A2387": "УЛ. ОБРАДОВСКИ ПЪТ",
    "TB1491": "РУМЪНСКО ПОСОЛСТВО",
    "TB2373": "МЕТРОСТАНЦИЯ БИЗНЕС ПАРК",
    "A6599": "БЛ. 28 Ж.К. ЛЕВСКИ-Г",
    "A1748": "ТЕЦ СОФИЯ",
    "A1245": "ПАРАКЛИС СВ.ИВ.РИЛСКИ",
    "TB1006": "ЛЕТИЩЕ СОФИЯ ТЕРМИНАЛ 1",
    "A2246": "УЛ. БОР - ПО ЖЕЛАНИЕ",
    "TB2731": "УЛ. РОДОПСКИ ИЗВОР",
    "A6452": "БУЛ. Д-Р ПЕТЪР ДЕРТЛИЕВ",
    "A0134": "БЛ. 116 Ж.К. ОБЕЛЯ-1",
    "TM1154": "СК ЦСКА",
    "A2125": "Ж.К. ЛЕВСКИ-В",
    "A1575": "СЕПАРАЦИЯТА - ПО ЖЕЛАНИЕ",
    "TM1299": "Пл. Руски паметник",
    "A0775": "ИНСТИТУТА",
    "A1887": "115-ТО ОУ",
    "A0753": "М - ТЕЛ",
    "TB0566": "БАЛКАНКАР АД",
    "TB1374": "В.З. МАЛИНОВА ДОЛИНА",
    "A2449": "Ж.К. МАГНОЛИЯ - ПО ЖЕЛАНИЕ",
    "A6820": "УЛ. 243 - ВРЕМЕННА",
    "A1442": "ХАЙДУШКИ ЧУКАР - ПО ЖЕЛАНИЕ",
    "A1587": "СЛАТИНА БУЛГАРПЛОД",
    "A0225": "БЛ. 501 Ж.К. ДРУЖБА-2",
    "TM1325": "Хотел Хемус",
    "A1070": "МИТНИЦА СУХОДОЛ",
    "A6716": "Ул. Стръмна",
    "A6610": "БУЛ. ВАРДАР - ВРЕМЕННА",
    "A2751": "УЛ. АЛ. СТАМБОЛИЙСКИ - ПО ЖЕЛАНИЕ",
    "A0779": "В.З. СИМЕОНОВО-ДРАГАЛЕВЦИ",
    "TB0973": "КОРИЯТА",
    "TM1271": "ПЛ. ВЪЗРАЖДАНЕ",
    "A0667": "Ж.К. МЛАДОСТ-3",
    "TB0970": "Корабчето",
    "TB1632": "БУЛ. КРЪСТЬО ПАСТУХОВ",
    "A0474": "ГАРА РУДНИЧНА - ПО ЖЕЛАНИЕ",
    "A0516": "ДЕПО ИСКЪР",
    "A6290": "Ул.Ген.Суворов-временна",
    "TB2432": "БУЛ. БРЮКСЕЛ",
    "A1823": "УЛ. АЛЕКСАНДЪР ЖЕНДОВ",
    "TM0286": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "TB0123": "БЛ. 22 Ж.К. ДИАНАБАД",
    "A2182": "УЛ. СТ. ДОСПЕВСКИ",
    "A2448": "БЛ. 119 Ж.К. ОБЕЛЯ-1",
    "A1854": "УЛ. СТАР БЕЛОВОДСКИ ПЪТ - ПО ЖЕЛАНИЕ",
    "TB1246": "ПАРАКЛИС СВ.ИВ.РИЛСКИ",
    "A0711": "УЛ. КНЯГИНЯ КОСАРА",
    "A6291": "Бул.Александър Пушкин-временна",
    "TM2060": "УЛ. Н. КОПЕРНИК",
    "TM2045": "Ул. Милин камък",
    "TB6275": "БУЛ. БЪЛГАРИЯ",
    "A1341": "ПЛОЩАДА КВ. ВРАЖДЕБНА",
    "A6436": "ПЛ. КНЯЗ АЛЕКСАНДЪР І",
    "TM0644": "Ж.к. Лагера",
    "A2393": "БЛ. 472 Ж.К. МЛАДОСТ-4",
    "A1917": "МЕТРОСТАНЦИЯ АКАД. А. Т.- БАЛАН",
    "TB1192": "УМБАЛ СВ. АННА",
    "TB0254": "БЛ. 715 Ж.К. ЛЮЛИН-7",
    "TM1284": "ПЛ. МАКЕДОНИЯ",
    "TB0347": "БУЛ. НИКОЛА ПЕТКОВ",
    "A1507": "ЧЕШМАТА - ПО ЖЕЛАНИЕ",
    "A6390": "Кв.Вердикал",
    "A6713": "Ул. Методи Иванов",
    "A0192": "БЛ. 32 Ж.К. ОВЧА КУПЕЛ-2",
    "A0587": "Ж.К. БЪКСТОН",
    "TM0222": "БЛ. 458 Ж.К. НАДЕЖДА-4",
    "A0226": "БЛ. 501 Ж.К. ДРУЖБА-2",
    "A6359": "УЛ.ФИЛИП КУТЕВ",
    "A6660": "ПЪРВА АГ БОЛНИЦА СВ. СОФИЯ",
    "A1986": "УЛ. КАП. Д. СПИСАРЕВСКИ",
    "A2432": "БУЛ. БРЮКСЕЛ",
    "TB6331": "УЛ. ПРОФЕСОР ФРИТЬОФ НАНСЕН",
    "A1977": "УЛ. КАМЕН АНДРЕЕВ",
    "A1608": "СК НА НСА - ПО ЖЕЛАНИЕ",
    "A1699": "СУ СВ.КЛИМЕНТ ОХРИДСКИ",
    "TM0378": "Женски пазар",
    "TB2182": "УЛ. СТ. ДОСПЕВСКИ",
    "A2218": "КЕНДИ ООД - ПО ЖЕЛАНИЕ",
    "A2026": "УЛ. ЛЕШНИКОВА ГОРА",
    "A1893": "УЛ. ГЕН. ВЛ. ДИНЧЕВ",
    "A1811": "УЛ. 61",
    "A0370": "БУЛ. ПРОФ.ЦВ. ЛАЗАРОВ",
    "A0648": "Ж.К. ЛОЗЕНЕЦ",
    "TB2213": "УЛ. Т. КАБЛЕШКОВ",
    "A1822": "УЛ. АЛЕКО КОНСТАНТИНОВ",
    "TB2034": "УЛ. ЛУИ АЙЕР",
    "A0256": "БЛ. 98 Ж.К. ДРУЖБА-1",
    "TM0274": "БУЛ. АКАД. ИВ. ГЕШОВ",
    "A0152": "БЛ. 264 Ж.К. ЛЮЛИН-2",
    "A2401": "УЛ.ВАСИЛ ЛЕВСКИ",
    "A0945": "ОБЩИНА МЛАДОСТ",
    "TM2103": "УЛ. ПЛАНИНЕЦ",
    "A1506": "МОСТА НА СУХОДОЛСКА РЕКА",
    "A2617": "СЕЛО МИРОВЯНЕ - ПО ЖЕЛАНИЕ",
    "TB0969": "ПАЗАРА МЛАДОСТ 1",
    "A0389": "БУЛ. ЦАР БОРИС ІІІ",
    "A1484": "МОСТА НА СУХАТА РЕКА",
    "A1444": "РАЗКЛОНА ЗА МАГИСТРАЛА ХЕМУС",
    "A1910": "УЛ. ГИНЦИ",
    "A6405": "БУЛ. ХРИСТО БОТЕВ",
    "A2658": "УЛ. ХРИСТО СИЛЯНОВ",
    "A1284": "ПЛ. МАКЕДОНИЯ",
    "TB1891": "УЛ. ГЕН. АС. НИКОЛОВ",
    "TM0589": "Ж.к. Бъкстон",
    "A0149": "148-МО СОУ",
    "A0914": "КВ. КРЪСТОВА ВАДА",
    "TB2088": "УЛ. ОРЕХОВА ГОРА",
    "A1664": "НАЧАЛО С. ЖЕЛЯВА",
    "A0176": "БЛ. 255 Ж.К. ДРУЖБА-2",
    "A2389": "БЛ. 442 Ж.К. МЛАДОСТ-4",
    "A1617": "СТАДИОНА БАНКЯ",
    "TB0492": "РЕПУБЛИКАНСКИ НЕВРОДИСПАНСЕР",
    "A6196": "Бул.Владимир Вазов-временна",
    "TM1572": "Семинарията",
    "TM0367": "БУЛ. ПРОФ.ЦВ. ЛАЗАРОВ",
    "TB0309": "Бул. Гоце Делчев",
    "A1852": "УЛ. БЪЛГАРСКИ ВОИН",
    "A2295": "ФОХАР АД",
    "A1902": "УЛ. ГЕН. ГУРКО",
    "TB0146": "БЛ. 170 Ж.К. ДРУЖБА-1",
    "TB0346": "БУЛ. НИКОЛА ПЕТКОВ",
    "TB1075": "ДАНОН Сердика АД",
    "A6254": "Ул.Ив.Димитров-Куклата-временна",
    "TM2039": "Ул. Люботрън",
    "A2775": "ПЪТЕН ВЪЗЕЛ ФИЛИПОВЦИ",
    "A0257": "БЛ. 98 Ж.К. ДРУЖБА-1",
    "A1068": "МИНАТА - ПО ЖЕЛАНИЕ",
    "A6158": "МЕТРОСТАНЦИЯ ВАРДАР",
    "A6203": "Временна бл.18 Студентски град",
    "A6644": "КВ. КАРПУЗИЦА",
    "A2537": "УЛ. СТАРА ПЛАНИНА",
    "TM0693": "Ж.к. Хиподрума",
    "A1747": "ТЕЦ СОФИЯ",
    "A0703": "Ж.П.ПРЕЛЕЗ - ПО ЖЕЛАНИЕ",
    "TB2778": "Аджибадем сити клиник",
    "TB0315": "БУЛ. Д-Р Г. М. ДИМИТРОВ",
    "A6566": "ДЕНДРАРИУМА - ПО ЖЕЛАНИЕ",
    "A0175": "БЛ. 25 Ж.К. ОВЧА КУПЕЛ-2",
    "A1769": "ПЪТЕН ВЪЗЕЛ ЯНА - ПО ЖЕЛАНИЕ",
    "A6696": "УЛ. ЛЕШНИКОВА ГОРА - ВРЕМЕННА",
    "A0985": "КРАЯ С. ЖИТЕН",
    "TB0423": "КВ. МАЛИНОВА ДОЛИНА - ПО ЖЕЛАНИЕ",
    "A2433": "БУЛ. БРЮКСЕЛ",
    "TB2542": "НСБАЛ ПО ОНКОЛОГИЯ",
    "A0082": "УЛ. Н. ХРЕЛКОВ",
    "TB0137": "БЛ. 115 Ж.К. ЛЮЛИН-10",
    "A0751": "ЗСК КРЕМИКОВЦИ АД",
    "A2463": "Ж.К. МЛАДОСТ-1",
    "A0318": "БУЛ. Д-Р Г. М. ДИМИТРОВ",
    "A6019": "БУЛ. ШИПЧЕНСКИ ПРОХОД",
    "TB0245": "БЛ. 66 Ж.К. МЛАДОСТ-1",
    "A0376": "БУЛ. СЛИВНИЦА",
    "A0502": "ЦЕНТЪРА БАНКЯ",
    "TB2330": "Хотел Хемус",
    "A1618": "СТАДИОНА БАНКЯ",
    "TB0307": "Бул. Възкресение",
    "A1855": "УЛ. СТАР БЕЛОВОДСКИ ПЪТ - ПО ЖЕЛАНИЕ",
    "A0281": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "TM0027": "28-ми ДКЦ",
    "A1107": "БЛ. 703 Ж.К. ЛЮЛИН-7",
    "TM1270": "ПЛ. ВЪЗРАЖДАНЕ",
    "A0221": "БЛ. 458 Ж.К. НАДЕЖДА-4",
    "A0200": "БЛ. 350 Ж.К. ЛЮЛИН-3",
    "A0594": "Ж.К. ВРЪБНИЦА-2",
    "TB1502": "САТИРИЧЕН ТЕАТЪР",
    "A1338": "ПЛОЩАДА КВ. ВРАЖДЕБНА",
    "TB1845": "УЛ. БОРЯНА",
    "A0212": "ТЪРГОВСКИ Ц-Р ОВЧА КУПЕЛ",
    "A6619": "ПРИРОДЕН ПАРК ВИТОША - ПО ЖЕЛАНИЕ",
    "A1576": "СЕПАРАЦИЯТА - ПО ЖЕЛАНИЕ",
    "TB0598": "УЛ. ТОДОР КАБЛЕШКОВ",
    "TM1712": "Магазин Витоша",
    "TM0674": "Ж.К. НАДЕЖДА-3",
    "A1008": "ЛИФТА - ПО ЖЕЛАНИЕ",
    "TM0336": "УЛ. ЛУИ ПАСТЬОР",
    "A1479": "МОСТА С. ДОЛНИ БОГРОВ",
    "A0445": "УЛ. А. КОНСТАНТИНОВ - ПО ЖЕЛАНИЕ",
    "A2566": "УЛ. АНДРЕЙ ГЕРМАНОВ",
    "A6382": "УЛ. ПЪРВА БЪЛГАРСКА АРМИЯ",
    "A1040": "ПРОМИШЛЕНА ЗОНА - ПО ЖЕЛАНИЕ",
    "A2497": "ПО ЖЕЛАНИЕ (ВЕРДИКАЛ)",
    "A7001": "Ул.Манастирска",
    "A0161": "БЛ. 206 Ж.К. МЛАДОСТ-2",
    "TB6564": "БУЛ. БЪЛГАРИЯ",
    "A0240": "БЛ. 603 Ж.К. ВРЪБНИЦА-2",
    "TB0182": "БЛ. 3 Ж.К. ДРУЖБА-1",
    "TB1605": "НПМГ АКАД. ЛЮБОМИР ЧАКАЛОВ",
    "A0237": "БЛ. 6 Ж.К. МЛАДОСТ-1",
    "A2481": "УЛ. МАКГАХАН",
    "TM6181": "УЛ. АЛАБИН",
    "A2139": "УЛ. ПРОБУДА",
    "A2523": "ПО ЖЕЛАНИЕ - НОВА МАХАЛА",
    "A6368": "Летище София Враждебна-временна",
    "A1210": "МОСТА НА БАНИШКА РЕКА - ПО ЖЕЛАНИЕ",
    "A6715": "Ул. Стръмна",
    "TM1308": "ПЛ. СВ. НЕДЕЛЯ",
    "A1676": "НАЧАЛО С. МРАМОР - ПО ЖЕЛАНИЕ",
    "A1080": "МОСТА НА ЛЕСНОВСКА РЕКА",
    "TB0565": "БАЛКАНКАР АД",
    "A2443": "БЛ. 119 Ж.К. ОБЕЛЯ-1",
    "TM2581": "Ул. Кораб планина",
    "A0843": "КВ. ИЗГРЕВ - ПО ЖЕЛАНИЕ",
    "A2598": "Ж.К. ВОДОЛЕЙ",
    "A2750": "УЛ. АЛ. СТАМБОЛИЙСКИ - ПО ЖЕЛАНИЕ",
    "A1841": "УЛ. БЕСАРАБИЯ",
    "TB0232": "БЛ. 527 Ж.К. ОВЧА КУПЕЛ-1",
    "TB2795": "СОФИЯ ТЕХ ПАРК",
    "A2841": "УЛ. МИЛЕНИУМ",
    "A2459": "ПЛОЩАДА КВ. ВРАЖДЕБНА",
    "A0356": "БУЛ. ПЕНЧО СЛАВЕЙКОВ",
    "A6218": "УЛ. РЕПУБЛИКА",
    "TB1163": "БЛ. 21 Ж.К. ЛЮЛИН-1",
    "TM2110": "Ул. Нишава",
    "A0340": "БУЛ. ГЕН.Н.СТОЛЕТОВ",
    "A2281": "177-МО ОУ",
    "A1620": "СТАДИОНА БАНКЯ - ПО ЖЕЛАНИЕ",
    "A1926": "УЛ. ДИМИТЪР ПЕТКОВ",
    "TB2161": "УЛ. РИЛСКА ОБИТЕЛ",
    "A0100": "БАНЯТА КВ. ГНИЛЯНЕ",
    "A0610": "Ж.К. ЛЮЛИН-3 - УХО",
    "A6729": "УЛ. БИЛЯНИНИ ИЗВОРИ",
    "TB2805": "БУЛ. АКАД. ИВ. ЕВ. ГЕШОВ",
    "A1356": "ПО ЖЕЛАНИЕ",
    "TB0764": "НСБАЛ ПО ОНКОЛОГИЯ",
    "TB2624": "РЕЗИДЕНШЪЛ ПАРК СОФИЯ",
    "A0812": "МЕТРОСТАНЦИЯ МИЗИЯ",
    "A6333": "ПЪТНОСТРОИТЕЛНА ТЕХНИКА - ВРЕМЕННА",
    "A0154": "БЛ. 202 Ж.К. МЛАДОСТ-2",
    "A6667": "Ул.Бели брод-временна",
    "TB0893": "Кв. Павлово",
    "A0754": "СТОКОВ БАЗАР ИЛИЯНЦИ",
    "A1271": "ПЛ. ВЪЗРАЖДАНЕ",
    "A0251": "БЛ. 706 Ж.К. ЛЮЛИН-7",
    "A2565": "СЕВЕРНА ПРОМИШЛЕНА ЗОНА",
    "TB0664": "Ж.К. МЛАДОСТ-2",
    "A6608": "10-ТИ ДКЦ",
    "TB2300": "ХАНЧЕТО - ПО ЖЕЛАНИЕ",
    "A2513": "УЛ. ГЕРГАНА",
    "TM0400": "Хотел Хилтън",
    "TM0923": "ЧИТАЛИЩЕ ПЕНЧО П.СЛАВЕЙКОВ 1921",
    "A0150": "148-МО СОУ",
    "A2265": "УНИКАЛНИ КОНСТРУКЦИИ - ПО ЖЕЛАНИЕ",
    "A0439": "КРАЯ С. МРАМОР",
    "A1805": "УЛ. 507",
    "A6588": "СЕМИНАРИЯТА",
    "A2266": "МАНАСТИР СВ. НИКОЛА - ПО ЖЕЛАНИЕ",
    "A1258": "ПЛОЩАД НА АВИАЦИЯТА",
    "A6635": "УЛ. БРЕЗИТЕ - ВРЕМЕННА",
    "TB0246": "БЛ. 66 Ж.К. МЛАДОСТ-1",
    "TB2217": "УЛ. ТОДОРИНИ КУКЛИ",
    "A6737": "УЛ.МАЕСТРО АТАНАСОВ - ВРЕМЕННА",
    "A2712": "ХОТЕЛ-РЕСТОРАНТ",
    "TB1692": "УНСС",
    "TB2106": "ул. Панайот Хитов",
    "TB6416": "УЛ. Г. С. РАКОВСКИ",
    "TM2420": "ул. Пловдив",
    "A0183": "БЛ. 3 Ж.К. БОТУНЕЦ-1",
    "TB0605": "Ж.К. ДРУЖБА-1",
    "A1782": "БУЛ. ПАНЧО ВЛАДИГЕРОВ",
    "A6103": "ПЛ. ВАСИЛ ЛЕВСКИ",
    "A0601": "УЛ. ТОДОР КАБЛЕШКОВ",
    "A1788": "ТЪРГОВСКИ Ц-Р ЛЮЛИН",
    "A6673": "Ул. Езерата - временна",
    "TM1048": "Метростанция Западен парк",
    "A2005": "УЛ. КОНСТАНТИН БОДИН",
    "A2263": "УЛ. ЮРИЙ ГАГАРИН",
    "TB0468": "ГАРА ПОДУЯНЕ",
    "TB1962": "УЛ. ИСКЪР",
    "A1722": "ПГВАД",
    "A2076": "УЛ. ОДРИН",
    "A6331": "УЛ. ПРОФЕСОР ФРИТЬОФ НАНСЕН",
    "TB0972": "КОРИЯТА",
    "TB1037": "Стадиона Герман",
    "A1084": "МОСТА",
    "TB1016": "МЕТРОСТАНЦИЯ ЦАРИГРАДСКО ШОСЕ",
    "A0966": "МАХАЛА КОНЬОВИЦА",
    "A1624": "В.З. КИТКАТА - ПО ЖЕЛАНИЕ",
    "A1408": "ПОЩА КВ. КУРИЛО - ПО ЖЕЛАНИЕ",
    "A1447": "РАЗКЛОНА ЗА С. ПЛАНА",
    "A1619": "СТАДИОНА БАНКЯ - ПО ЖЕЛАНИЕ",
    "A2061": "УЛ. Н. КОПЕРНИК",
    "A0875": "КВ. ОВЧА КУПЕЛ",
    "A2730": "УЛ. РОДОПСКИ ИЗВОР",
    "A0182": "БЛ. 3 Ж.К. ДРУЖБА-1",
    "A6138": "Ул.Евлия Челеби",
    "TB2128": "УЛ. ПОП ГРУЙО",
    "A0475": "ГАРА РУДНИЧНА - ПО ЖЕЛАНИЕ",
    "TB0231": "БЛ. 527 Ж.К. ОВЧА КУПЕЛ-1",
    "A1001": "УЛ. ЛОКОМОТИВ",
    "A6668": "134 СУ ДИМЧО ДЕБЕЛЯНОВ",
    "TM2022": "Автостанция Изток",
    "A2672": "ГАРА СОФИЯ СЕВЕР",
    "A6388": "Ул. Акад. Борис Стефанов-временна",
    "A2077": "УЛ. ОДРИН",
    "A6097": "КВ. ЛАГЕРА",
    "TB2829": "БУЛ. КН. МАРИЯ ЛУИЗА",
    "TM1726": "Театър София",
    "A1178": "ОКОЛОВРЪСТЕН ПЪТ",
    "TB0340": "БУЛ. ГЕН.Н.СТОЛЕТОВ",
    "A6461": "БУЛ. Д-Р ПЕТЪР ДЕРТЛИЕВ",
    "A1457": "РАЗКЛОНА ЗА КВ.ДРАГАЛЕВЦИ",
    "A1663": "НАЧАЛО С. ЖЕЛЯВА",
    "A1944": "ЦЕНТРАЛЕН ГРОБИЩЕН ПАРК",
    "A2128": "УЛ. ПОП ГРУЙО",
    "A0363": "БУЛ. ПРАГА",
    "A1632": "БУЛ. КРЪСТЬО ПАСТУХОВ",
    "A6567": "ЗЛАТНИ МОСТОВЕ",
    "TB0546": "БУЛ. РОЖЕН",
    "A0139": "БЛ. 15 Ж.К. ХР. СМИРНЕНСКИ",
    "A0911": "МЕТРОСТАНЦИЯ ВИТОША",
    "A2010": "УЛ. КОСТЕНСКИ ВОДОПАД",
    "TB1801": "УЛ. ИВАН АНДОНОВ",
    "A2520": "ЕКОБУЛСОРТ - ПО ЖЕЛАНИЕ",
    "A6400": "БУЛ. ПРАГА",
    "TB1032": "МГУ СВ.ИВ.РИЛСКИ",
    "TB0237": "БЛ. 6 Ж.К. МЛАДОСТ-1",
    "A2293": "ФОНДОВИ ЖИЛИЩА",
    "A6023": "УЛ. МИТРОПОЛИТ К. ВИДИНСКИ",
    "TM2196": "БЛ. 13 Ж.К.ЗАПАДЕН ПАРК",
    "A0178": "БЛ. 255 Ж.К. ДРУЖБА-2",
    "A1007": "ЛИФТА - ПО ЖЕЛАНИЕ",
    "A1882": "УЛ. ВЛАДАЙСКО ВЪСТАНИЕ - ПО ЖЕЛАНИЕ",
    "A2640": "БЛ. 8 Ж.К. КРЕМИКОВЦИ",
    "A0656": "Ж.К. ЛЮЛИН-5",
    "A1416": "УЛ. МАЛА КОРИЯ - ПО ЖЕЛАНИЕ",
    "TB2782": "УЛ. МОГИЛАТА",
    "TB1708": "ПАЗАРА ОВЧА КУПЕЛ",
    "A0684": "Ж.К. ОВЧА КУПЕЛ-2",
    "A2809": "УЛ. ХАН КУБРАТ",
    "A2636": "УЛ. СВ. ПЕТКА",
    "A1041": "ПРОМИШЛЕНА ЗОНА - ПО ЖЕЛАНИЕ",
    "A0218": "БЛ. 43 Ж.К. МЛАДОСТ-1",
    "A0682": "Ж.К. ОВЧА КУПЕЛ-2",
    "A1883": "УЛ. ВЛАДАЙСКО ВЪСТАНИЕ - ПО ЖЕЛАНИЕ",
    "A1093": "МОСТА КВ.ВРАЖДЕБНА",
    "A2038": "УЛ. ЛЮБОТРЪН",
    "A2639": "БЛ. 8 Ж.К. КРЕМИКОВЦИ",
    "TM0694": "Ж.к. Хиподрума",
    "A2398": "БЛ. 539 Ж.К.ЛЮЛИН-5",
    "A2153": "УЛ. РАДЕЦКИ",
    "A0191": "БЛ. 32 Ж.К. ОВЧА КУПЕЛ-2",
    "A0279": "БУЛ. АЛ. ПУШКИН",
    "A1357": "ПО ЖЕЛАНИЕ",
    "A1279": "ПЛ. МАКЕДОНИЯ",
    "TB1012": "Ж.К. ДЪРВЕНИЦА",
    "A1108": "БЛ. 703 Ж.К. ЛЮЛИН-7",
    "A1462": "Р-Т АЛЕКСАНДРИЯ",
    "A1816": "УЛ. 9 - ПО ЖЕЛАНИЕ",
    "A2536": "УЛ. СТАРА ПЛАНИНА",
    "A2824": "В.З. МАЛИНОВА ДОЛИНА - ПО ЖЕЛАНИЕ",
    "TB1818": "УЛ. 9 - ПО ЖЕЛАНИЕ",
    "A2447": "РАЗКЛОНА-ЛИФТ СИМЕОНОВО",
    "A2581": "УЛ. КОРАБ ПЛАНИНА",
    "A0699": "Ж.П.ПРЕЛЕЗ",
    "TB6103": "ПЛ. ВАСИЛ ЛЕВСКИ",
    "TB0604": "Ж.К. ДРУЖБА-1",
    "A6406": "БУЛ. ХРИСТО БОТЕВ",
    "TB1917": "МЕТРОСТАНЦИЯ АКАД. А. Т.- БАЛАН",
    "TB1198": "УМБАЛ СВ. АННА-УХО",
    "TB1015": "Ж.К. ДЪРВЕНИЦА",
    "A0513": "АВТОСЕРВИЗ",
    "A6264": "УЛ. БИСТРИШКО ШОСЕ",
    "A2632": "УЛ. ВИШНОВСКА - ПО ЖЕЛАНИЕ",
    "A6437": "Пл.Васил Левски",
    "A6142": "Ул.Евлия Челеби",
    "A6225": "ВМА - ВРЕМЕННА",
    "TM0708": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "A1658": "ЕЗЕРОТО",
    "A1603": "ВИК - ЕАД - СОФИЯ",
    "A0550": "СТИНД",
    "A1682": "УЛ. ПРИЛЕП",
    "TB0625": "Ж.К. ИВАН ВАЗОВ",
    "TB0167": "БЛ. 222 Ж.К. МЛАДОСТ-2",
    "A1426": "РАЗКЛОНА ЗА С. ГЕРМАН",
    "TB2424": "Ж.К. ДРУЖБА -1",
    "A2535": "УЛ. ЛИПА",
    "A1767": "УЛ. АКАЦИЯ - ПО ЖЕЛАНИЕ",
    "A1889": "УЛ. ГЕН. АС. НИКОЛОВ",
    "A2641": "УЛ. МАЕСТРО АТАНАСОВ-ПО ЖЕЛАНИЕ",
    "A0717": "КОТЛОСТРОЕНЕ АД",
    "A2773": "УЛ. 24-ТИ МАЙ",
    "A6334": "НОВАТА ГОРА НА СОФИЯ 2",
    "A6086": "18 СУ УИЛЯМ ГЛАДСТОН",
    "A6454": "МЕТРОСТАНЦИЯ СЕРДИКА",
    "A0878": "КВ. ОВЧА КУПЕЛ",
    "TM6570": "Ж.К. БЪКСТОН",
    "A2654": "МЕТРОСТАНЦИЯ ВИТОША",
    "A2843": "УЛ. ПЕРУНИКА",
    "A6455": "ПЛ. КНЯЗ АЛЕКСАНДЪР І",
    "A2466": "ЛИФТА ДРАГАЛЕВЦИ",
    "A2383": "ЯЗ. СУХОДОЛ",
    "A0781": "КАВАЦИТЕ",
    "A2413": "НАЧАЛО ДОЛНИ ЛОЗЕН",
    "A2846": "МЕТРОСТАНЦИЯ ГОРНА БАНЯ",
    "A1431": "РАЗКЛОНА ЗА С. ГУРМАЗОВО - ПО ЖЕЛАНИЕ",
    "A1770": "УЛ. АКАЦИЯ - ПО ЖЕЛАНИЕ",
    "A2691": "ТИР ПАРКИНГ-МРАМОР - ПО ЖЕЛАНИЕ",
    "A1192": "УМБАЛ СВ. АННА",
    "TB0430": "В.З. ГЕРМАН",
    "A6636": "ВАКЛИНОВИ КОШАРИ - ПО ЖЕЛАНИЕ",
    "TB1975": "УЛ. КАЛИМАНЦИ",
    "A6618": "ПРИРОДЕН ПАРК ВИТОША - ПО ЖЕЛАНИЕ",
    "A2860": "УЛ. КИШИНЕВ",
    "A1155": "СК ЦСКА",
    "TB1809": "ХРАМ СВ. КЛИМЕНТ ОХРИДСКИ",
    "A1638": "КРАЯ С. БИСТРИЦА",
    "A2488": "БЛ. 325 Ж.К. МЛАДОСТ-3",
    "A6689": "Гробищен парк Драгалевци-временна",
    "A1657": "КРАЯ С. ДОБРОСЛАВЦИ",
    "A0211": "ТЪРГОВСКИ Ц-Р ОВЧА КУПЕЛ",
    "TM1571": "Семинарията",
    "TM1307": "ПЛ. СВ. НЕДЕЛЯ",
    "A1868": "УЛ. ВИДИНИ КУЛИ",
    "A2676": "БЛ. 245 Ж.К. ОБЕЛЯ-2",
    "A2812": "УЛ.САВА ФИЛАРЕТОВ",
    "A6292": "Ул. Кота 1050 - временна",
    "TB0168": "БЛ. 222 Ж.К. МЛАДОСТ-2",
    "TB1633": "ХЮНДАЙ БЪЛГАРИЯ",
    "TM6239": "ХЮНДАЙ БЪЛГАРИЯ",
    "A0316": "БУЛ. Д-Р Г. М. ДИМИТРОВ",
    "A2307": "ХИПОДРУМА БАНКЯ - ПО ЖЕЛАНИЕ",
    "A2392": "БЛ. 472 Ж.К. МЛАДОСТ-4",
    "TB0042": "66-ТО СОУ",
    "A1488": "БЛ. 405 Ж.К. ДРУЖБА-2",
    "A2480": "БАЛКАН СТАР",
    "TB0876": "Кв. Овча купел",
    "A0248": "БЛ. 69 Ж.К. МЛАДОСТ-1",
    "A0557": "ДЯВОЛСКИ МОСТ - ПО ЖЕЛАНИЕ",
    "TB1607": "СК НА НСА - ПО ЖЕЛАНИЕ",
    "TB1881": "УЛ. ВИШОВГРАД",
    "A2713": "ХОТЕЛ-РЕСТОРАНТ",
    "A6711": "БУЛ. КОНСТАНТИН ВЕЛИЧКОВ - ВРЕМЕННА",
    "TB1501": "САТИРИЧЕН ТЕАТЪР",
    "A0145": "БЛ. 162 Ж.К. ДРУЖБА-1",
    "A0391": "БУЛ. ЦАР БОРИС ІІІ",
    "A1861": "УЛ. ВАСИЛ КЪНЧЕВ",
    "A2522": "ПО ЖЕЛАНИЕ - НОВА МАХАЛА",
    "A6017": "кв.Овча Купел-временна",
    "TB1156": "Ул. Гюешево",
    "A0371": "БУЛ. ПРОФ.ЦВ. ЛАЗАРОВ",
    "A1681": "УЛ. ПРИЛЕП",
    "A1043": "МЕТАЛСНАБ ХОЛДИНГ",
    "A2467": "Ж.К. МЛАДОСТ-1/БЛ. 70/",
    "TB1782": "БУЛ. ПАНЧО ВЛАДИГЕРОВ",
    "A0585": "Ж.К. БОТУНЕЦ -2 - ПО ЖЕЛАНИЕ",
    "A6735": "УЛ.АНГЕЛ БУКУРЕЩЛИЕВ - ВРЕМЕННА",
    "A2047": "УЛ. КОСМОНАВТ - ПО ЖЕЛАНИЕ",
    "A1383": "ПОДЛЕЗА ГАРА ИСКЪР",
    "A2707": "УЛ. ОБЗОР",
    "A2832": "192 - РО СОУ С. БИСТРИЦА",
    "TB1615": "СТАДИОН Г. АСПАРУХОВ",
    "A0796": "КАТОЛИЧЕСКИ ГРОБИЩЕН ПАРК",
    "TB0520": "ДЕПО ИСКЪР",
    "A1242": "ПЛ. ВАСИЛ ЛЕВСКИ",
    "A6202": "Временна бл.18 Студентски град",
    "A0054": "АВТОСТАНЦИЯ ДЪРВЕНИЦА",
    "A2148": "УЛ. ПРОФ.И.ГЕОРГОВ",
    "TB2730": "УЛ. РОДОПСКИ ИЗВОР",
    "A6495": "ПЛОЩАД НЕЗАВИСИМОСТ",
    "A2807": "ЕСТАКАДА Ж.К. МЛАДОСТ 2",
    "A0652": "Ж.К. ЛЮЛИН-3",
    "A1032": "МГУ СВ.ИВ.РИЛСКИ",
    "A6387": "Ул. Акад. Борис Стефанов-временна",
    "A0019": "24-ТИ ДКЦ",
    "TB6276": "УЛ. Г. ИЗМИРЛИЕВ",
    "A0546": "БУЛ. РОЖЕН",
    "TB0269": "БУЛ. АКАД. ИВ. ГЕШОВ",
    "TM2576": "БУЛ. ЛОМСКО ШОСЕ",
    "A1952": "УЛ. ИВАН КРАЛИЧА",
    "A1458": "РАЗКЛОНА ЗА КВ.СИМЕОНОВО",
    "A1662": "КРАЯ НА С. ЖЕЛЕЗНИЦА - ПО ЖЕЛАНИЕ",
    "A0153": "БЛ. 264 Ж.К. ЛЮЛИН-2",
    "A0854": "КВ. КРЕМИКОВЦИ",
    "TM1323": "Хотел Хемус",
    "A1248": "ПАРК СВ.СВ.ПЕТЪР И ПАВЕЛ",
    "A2746": "СКЛАДОВА БАЗА ЧЕПИНСКО ШОСЕ",
    "A6822": "УЛ. ЛАГАДИНА - ВРЕМЕННА",
    "A0159": "БЛ. 206 Ж.К. ДРУЖБА-2",
    "A2638": "156 - ТО ОУ",
    "A1612": "СТАДИОН Г. АСПАРУХОВ",
    "A1614": "СТАДИОН Г. АСПАРУХОВ",
    "A0769": "ИНСТИТУТ ПО ЧЕРНА МЕТАЛУРГИЯ - ПО ЖЕЛАНИЕ",
    "A1355": "ПО ЖЕЛАНИЕ",
    "A6659": "БУЛ. ВАРДАР",
    "A0584": "Ж.К. БОТУНЕЦ -2 - ПО ЖЕЛАНИЕ",
    "A1083": "МОСТА",
    "A6671": "Гробищен парк Драгалевци-временна",
    "TB1422": "ПЪРВА ГРАДСКА БОЛНИЦА",
    "A0794": "КАРИЕРАТА - ПО ЖЕЛАНИЕ",
    "A6363": "Пл. Централна гара-временна",
    "A1433": "УЛ. РОДИНА - ПО ЖЕЛАНИЕ",
    "TB6663": "ж.к. Гоце Делчев-зарядна",
    "TM0322": "БУЛ. ПЕТЪР ДЕРТЛИЕВ",
    "TM0331": "ул. Христо Силянов",
    "TB1455": "Разклона за летището",
    "A0583": "Ж.к.Борово",
    "A0625": "Ж.К. ИВАН ВАЗОВ",
    "TB0098": "НЦ ПО РЕХАБИЛИТАЦИЯ",
    "A2376": "ШЛАКОВ ЦЕХ",
    "A6298": "ЦЕНТЪРА БАНКЯ",
    "TB6116": "Разклона за летището",
    "A0522": "УЛ. 10-ТИ КИЛОМЕТЪР - ПО ЖЕЛАНИЕ",
    "TB0200": "БЛ. 350 Ж.К. ЛЮЛИН-3",
    "A0993": "КУЛТУРЕН ДОМ",
    "A1500": "НАЧАЛО С. ВЛАДАЯ - ПО ЖЕЛАНИЕ",
    "TB1091": "МОСТА ЗА РЕЗИДЕНЦИЯ ВРАНА",
    "A2384": "ЯЗ. МАЛО БУЧИНО",
    "A6430": "МЕТРОСТАНЦИЯ КН. МАРИЯ ЛУИЗА",
    "A0782": "В.З. СИМЕОНОВО-ДРАГАЛЕВЦИ",
    "A1592": "СКИОРКАТА - ПО ЖЕЛАНИЕ",
    "A1065": "УЛ. 503",
    "A0140": "БЛ. 15 Ж.К. ХР. СМИРНЕНСКИ",
    "A0668": "Ж.К. МЛАДОСТ-3",
    "TM1949": "УЛ. ЗОГРАФСКИ МАНАСТИР",
    "A1240": "ЦЕНТЪРА С. КРИВИНА",
    "TB1195": "УМБАЛ СВ. АННА",
    "A0270": "БУЛ. АКАД. ИВ. ГЕШОВ",
    "TB0293": "Бул. Вардар",
    "A6068": "БЯЛАТА ВОДА - ПО ЖЕЛАНИЕ",
    "A6625": "МЕТРОСТАНЦИЯ ГОРНА БАНЯ",
    "A0390": "БУЛ. ЦАР БОРИС ІІІ",
    "A1881": "УЛ. ВИШОВГРАД",
    "A0811": "В.З. БЕЛОВОДСКИ ПЪТ - ПО ЖЕЛАНИЕ",
    "TB1257": "ПЛОЩАД  НА АВИАЦИЯТА",
    "A0208": "БЛ. 402 Ж.К. ЛЮЛИН-4",
    "A2656": "В.З. БУНКЕРА-ПО ЖЕЛАНИЕ",
    "TM1738": "ПГД ЕЛИСАВЕТА ВАЗОВА",
    "A1229": "ЯЗ. ПАСАРЕЛ - ПО ЖЕЛАНИЕ",
    "TB0929": "ЧИТАЛИЩЕ Д-Р П. БЕРОН",
    "TB1039": "Стадиона Герман",
    "TB2433": "БУЛ. БРЮКСЕЛ",
    "A2597": "Ж.К. ВОДОЛЕЙ",
    "TB0197": "БЛ. 350 Ж.К. ЛЮЛИН-3",
    "TM0873": "КВ. ОБЕЛЯ",
    "TB0611": "Ж.К. ДРУЖБА-2",
    "A1446": "РАЗКЛОНА ЗА С. ПЛАНА",
    "A1918": "МЕТРОСТАНЦИЯ АКАД.А. Т.- БАЛАН",
    "A0702": "Ж.П.ПРЕЛЕЗ - ПО ЖЕЛАНИЕ",
    "A0173": "БЛ. 245 Ж.К. ОБЕЛЯ-2",
    "A0317": "БУЛ. Д-Р Г. М. ДИМИТРОВ",
    "TB0425": "В.З. ГЕРМАН",
    "A1159": "МЕТАЛИКА ООД",
    "A6569": "УЛ. БЯЛАТА ЧЕШМА - ПО ЖЕЛАНИЕ",
    "TB1264": "ПЛ. АЛ.НЕВСКИ",
    "A0146": "БЛ. 170 Ж.К. ДРУЖБА-1",
    "A1441": "МАХАЛА ЯПАДЖА",
    "A1622": "СТАДИОНА КАЗИЧЕНЕ",
    "A0742": "ЗИМЕН ДВОРЕЦ НА СПОРТА",
    "A1115": "НАЦИОНАЛНА ОПЕРА",
    "TB0257": "БЛ. 98 Ж.К. ДРУЖБА-1",
    "TB0304": "Бул. Възкресение",
    "A0193": "БЛ. 327 Ж.К. МЛАДОСТ-3",
    "A0343": "БУЛ. НИКОЛА ВАПЦАРОВ",
    "A0342": "БУЛ. НИКОЛА ВАПЦАРОВ",
    "A1105": "МОТЕЛ БЕЛИ БРЕЗИ - ПО ЖЕЛАНИЕ",
    "A2783": "УЛ. МОГИЛАТА",
    "A0947": "ОБЩИНА МЛАДОСТ",
    "TM1454": "РАЗКЛОНА ЗА ЛЕТИЩЕТО",
    "A6229": "БУЛ. ПЕНЧО СЛАВЕЙКОВ",
    "A6615": "УЛ. ПРЕЗВИТЕР КОЗМА",
    "A0263": "ГРОБИЩЕН ПАРК БОЯНА - ПО ЖЕЛАНИЕ",
    "A2626": "ОКОЛОВРЪСТЕН ПЪТ",
    "A0066": "БУЛ. ИСТОРИЯ СЛАВЯНОБЪЛГАРСКА",
    "A2823": "В.З. МАЛИНОВА ДОЛИНА - ПО ЖЕЛАНИЕ",
    "A6823": "НИМ-ВРЕМЕННА",
    "TB0069": "НБУ",
    "TB1475": "БЛ. 6 Ж.К. СУХАТА РЕКА",
    "TM0221": "БЛ. 458 Ж.К. НАДЕЖДА-4",
    "A2744": "УЛ. ЗАХАРИ ЗОГРАФ",
    "A2774": "БЛ. 350 Ж.К. ЛЮЛИН-3",
    "A2404": "МОСТА НА Р.БЛАЗНА БАРА",
    "A2789": "Ж.К. ХАДЖИ ДИМИТЪР",
    "TB0471": "Гара Подуяне",
    "TB6662": "ж.к. Гоце Делчев",
    "TM2169": "Ул. Сан Стефано",
    "A0910": "МЕТРОСТАНЦИЯ ВИТОША",
    "A2118": "УЛ. ПОДУЕНСКА",
    "TB0684": "Ж.К. ОВЧА КУПЕЛ-2",
    "A1735": "ТЕРСНАБ",
    "A0141": "БЛ. 155 Ж.К. ДРУЖБА-1",
    "A0478": "ГАРА ЯНА",
    "A6435": "ПЛ. КНЯЗ АЛЕКСАНДЪР І",
    "TB2351": "ТЕХНИЧЕСКА БИБЛИОТЕКА",
    "A0241": "БЛ. 620 Ж.К. ЛЮЛИН-6",
    "TM0592": "Ж.К. ВРЪБНИЦА-1",
    "A0123": "БЛ. 22 Ж.К. ДИАНАБАД",
    "A2761": "ПАРАКЛИС СВ. ВМЧК МИНА - ПО ЖЕЛАНИЕ",
    "TB0362": "БУЛ. ПРАГА",
    "A1582": "РАЗКЛОНА ЗА КВ. ВРАЖДЕБНА",
    "TB0033": "КВ. ГАРА ИСКЪР",
    "TB6279": "УЛ. Г. ИЗМИРЛИЕВ",
    "A2144": "УЛ. ПРОСВЕТА",
    "A2219": "КЕНДИ ООД - ПО ЖЕЛАНИЕ",
    "A6674": "Ул. Овче поле - временна",
    "A0163": "БЛ. 207 Ж.К. ОБЕЛЯ-2",
    "A0186": "БЛ. 301 Ж.К. ЛЮЛИН-3",
    "A2817": "УЛ. БОТЕВГРАДСКО ШОСЕ",
    "A6614": "УЛ. ПРЕЗВИТЕР КОЗМА",
    "A0129": "БЛ. 108 Ж.К. ДРУЖБА-1",
    "TB0728": "ЗАГ ДИНАМИК АД",
    "A0771": "ИНСТИТУТ ПУШКАРОВ - ПО ЖЕЛАНИЕ",
    "A2446": "РАЗКЛОНА-ЛИФТ СИМЕОНОВО",
    "A1303": "ПЛ. РУЧЕЙ",
    "A0780": "КАВАЦИТЕ",
    "TM1980": "УЛ. КАП. Д. СПИСАРЕВСКИ",
    "A0655": "Ж.К. ЛЮЛИН-5",
    "TB0393": "БУЛ. ЦАР БОРИС ІІІ",
    "A0352": "МЕТРОСТАНЦИЯ ОВЧА КУПЕЛ",
    "A0842": "КВ. ИЗГРЕВ - ПО ЖЕЛАНИЕ",
    "TB0813": "МЕТРОСТАНЦИЯ МИЗИЯ",
    "TB0175": "БЛ. 25 Ж.К. ОВЧА КУПЕЛ-2",
    "A2015": "УЛ. КРУМ СТОЯНОВ",
    "TB2473": "НОВА ТЕЛЕВИЗИЯ",
    "A0351": "МЕТРОСТАНЦИЯ ОВЧА КУПЕЛ",
    "TM0640": "Ж.к. Красно село",
    "A1435": "РАЗКЛОНА ЗА ЛЕТИЩЕТО",
    "A6702": "Метростанция Горна баня",
    "A2492": "КИНО АРЕНА ІІ",
    "A1839": "УЛ. БЯЛАТА ЧЕШМА",
    "A1384": "БУЛ. ИЛИЯНЦИ-ПОДЛЕЗА",
    "A1092": "МОСТА КВ.ВРАЖДЕБНА",
    "A2014": "ВТУ Т. КАБЛЕШКОВ",
    "A1382": "ПОДЛЕЗА ГАРА ИСКЪР",
    "TB2649": "МЕСТНОСТ КАМБАНИТЕ",
    "A2621": "УЛ. КРУШАТА - ПО ЖЕЛАНИЕ",
    "A2647": "БУЛ. ПРОФ.ЦВ. ЛАЗАРОВ",
    "A1560": "КВ. ЧЕЛОПЕЧЕНЕ",
    "A2496": "ПО ЖЕЛАНИЕ (ВЕРДИКАЛ)",
    "A2112": "УЛ. ПИРОТСКА",
    "A2609": "Ж.К. МЛАДОСТ-1/БЛ. 70/",
    "TB0124": "БЛ. 22 Ж.К. ДИАНАБАД БЛ. 1",
    "TM0096": "НЦ по рехабилитация",
    "TB1155": "СК ЦСКА",
    "A1376": "РАЗКЛОНА КВ. КРЕМИКОВЦИ",
    "TB0801": "Ж.к. Дървеница",
    "A6634": "УЛ. БРЕЗИТЕ - ВРЕМЕННА",
    "A2204": "УЛ. ТЕОФИЛ ГАНЕВ",
    "A0275": "БУЛ. АКАД. ИВ. ГЕШОВ",
    "A1025": "МАХАЛА ИВАНЯНЕ - ПО ЖЕЛАНИЕ",
    "A6276": "УЛ. Г. ИЗМИРЛИЕВ",
    "A0315": "БУЛ. Д-Р Г. М. ДИМИТРОВ",
    "TB1488": "БЛ. 405 Ж.К. ДРУЖБА-2",
    "A0447": "УЛ. ЙЕРУСАЛИМ",
    "A0536": "ДЕТСКО ГРАДЧЕ - ПО ЖЕЛАНИЕ",
    "A1571": "СЕМИНАРИЯТА",
    "A6126": "БУЛ. НИКОЛА ПЕТКОВ",
    "A1424": "НАЧАЛО КВ. МИХАЙЛОВО - ПО ЖЕЛАНИЕ",
    "A1485": "РУДНИКА - ПО ЖЕЛАНИЕ",
    "TB0292": "Бул. Вардар",
    "TM0026": "28-ми ДКЦ",
    "A2778": "АДЖИБАДЕМ СИТИ КЛИНИК",
    "A2806": "ЕСТАКАДА Ж.К. МЛАДОСТ 2",
    "TB1634": "ХЮНДАЙ БЪЛГАРИЯ",
    "A1698": "СУ СВ.КЛИМЕНТ ОХРИДСКИ",
    "A1340": "ПЛОЩАДА С. ВОЛУЯК",
    "A6641": "КМЕТСТВО КАЗИЧЕНЕ",
    "TB0169": "МЕТРОСТАНЦИЯ АЛ. МАЛИНОВ",
    "A1302": "ПЛ. РУЧЕЙ",
    "A0362": "БУЛ. ПРАГА",
    "A0700": "Ж.П.ПРЕЛЕЗ",
    "A1246": "ПАРАКЛИС СВ.ИВ.РИЛСКИ",
    "A2596": "ПЕТА ГРАДСКА БОЛНИЦА",
    "A2760": "МЕСТНОСТ ЯРЕМА",
    "A6639": "Ж.К. ЛОЗЕНЕЦ",
    "A0963": "КОКАЛЯНСКО ХАНЧЕ - ПО ЖЕЛАНИЕ",
    "TB0939": "ЦЕНТЪРА КВ. ИЛИЯНЦИ - ПО ЖЕЛАНИЕ",
    "A0409": "В.З. БУНКЕРА - ПО ЖЕЛАНИЕ",
    "TM0368": "БУЛ. ПРОФ.ЦВ. ЛАЗАРОВ",
    "A0136": "БЛ. 135 Ж.К. ЛЮЛИН-10",
    "A2058": "УЛ. Н. КОПЕРНИК",
    "A0517": "ДЕПО ИСКЪР",
    "A1452": "РАЗКЛОНА ЗА ЛЕТИЩЕТО",
    "A2635": "УЛ. СВ. ПЕТКА",
    "A6366": "Кв.Бояна/Кумата/",
    "TM0679": "Ж.К. ОБЕЛЯ-2",
    "A6420": "Ул. Попова шапка",
    "TB6004": "УЛ. ДИМИТЪР ПЕТКОВ",
    "A0650": "Ж.К. ЛЮЛИН-1,2",
    "A0923": "ЧИТАЛИЩЕ ПЕНЧО П.СЛАВЕЙКОВ 1921",
    "TB1931": "УЛ. ДОБРОТИЧ",
    "A2348": "ЦЕНТЪР С. МРАМОР",
    "A0437": "УЛ. 12 - ПО ЖЕЛАНИЕ",
    "A1120": "НАЧАЛО КВ. ГРАДОМАН",
    "A1525": "СЕЛО ДОЛНИ БОГРОВ",
    "A1381": "НАЧАЛО КВ. ИВАНЯНЕ - ПО ЖЕЛАНИЕ",
    "A1863": "УЛ. ВАСИЛ КЪНЧЕВ",
    "A0171": "БЛ. 241 Ж.К. ОБЕЛЯ-2",
    "A0417": "ВСУ Л.КАРАВЕЛОВ",
    "A2514": "УЛ. ГЕРГАНА",
    "A0181": "БЛ. 28 Ж.К. ЛЕВСКИ-Г",
    "A1688": "МОСТА НА ЛЕСНОВСКА РЕКА",
    "A1773": "16-ТИ ДКЦ",
    "A0076": "АНЕВА ЧЕШМА - ПО ЖЕЛАНИЕ",
    "A6672": "Ул. Езерата - временна",
    "TB0377": "БУЛ. СЛИВНИЦА",
    "A0898": "КВ. СЕСЛАВЦИ",
    "TB1017": "МЕТРОСТАНЦИЯ ЦАРИГРАДСКО ШОСЕ",
    "A0435": "В.З. КВ. БОЯНА",
    "A0712": "УЛ. КНЯГИНЯ КОСАРА",
    "TM0639": "Ж.к. Красно село",
    "A0007": "12-ТИ ДКЦ",
    "A0198": "БЛ. 350 Ж.К. ЛЮЛИН-3",
    "A2236": "УЛ. ХОРИЗОНТ",
    "TB1800": "УЛ. ИВАН АНДОНОВ",
    "A0701": "СОФИЯ МЕЛ АД",
    "TB1452": "РАЗКЛОНА ЗА ЛЕТИЩЕТО",
    "TB6005": "УЛ. ДИМИТЪР ПЕТКОВ",
    "A0425": "В.З. ГЕРМАН",
    "TB0095": "НЦ ПО РЕХАБИЛИТАЦИЯ",
    "A0556": "ДЯВОЛСКИ МОСТ - ПО ЖЕЛАНИЕ",
    "A1257": "ПЛОЩАД НА АВИАЦИЯТА",
    "A1874": "УЛ. ВИТИНЯ",
    "TM0852": "КВ. КНЯЖЕВО",
    "A6611": "10-ТИ ДКЦ",
    "A0598": "УЛ. ТОДОР КАБЛЕШКОВ",
    "A6458": "ПЛ. КНЯЗ АЛЕКСАНДЪР І",
    "TB2830": "БУЛ. КН. МАРИЯ ЛУИЗА",
    "A6189": "Ул.Костенски водопад-временна",
    "TB2674": "БУЛ. РОЖЕН",
    "A0981": "УЛ. МЕЧА ПОЛЯНА - ПО ЖЕЛАНИЕ",
    "A0731": "ЗАСЛОН СИНАНИЦА",
    "A0926": "КИНО ОДЕОН",
    "A1961": "УЛ. ИНДИРА ГАНДИ",
    "A2726": "ЕЗЕРОТО - СЕЛО КАЗИЧЕНЕ",
    "TB0227": "БЛ. 505 Ж.К. ОВЧА КУПЕЛ-1",
    "A0813": "МЕТРОСТАНЦИЯ МИЗИЯ",
    "A2487": "БЛ. 323 Ж.К. МЛАДОСТ-3",
    "TB2126": "УЛ. ПОП ГРУЙО",
    "A2292": "ФОНДОВИ ЖИЛИЩА",
    "A2494": "ТЕХНОПОЛИС Ж.К. МЛАДОСТ-4",
    "A2525": "176-ТО ОУ НЕГОВАН",
    "TB0091": "ТЕХНОТЕСТ",
    "TB2035": "УЛ. ЛУИ АЙЕР",
    "TB6178": "УЛ. КАП. Д. СПИСАРЕВСКИ",
    "TB0147": "БЛ. 170 Ж.К. ДРУЖБА-1",
    "TB0152": "БЛ. 264 Ж.К. ЛЮЛИН-2",
    "A6398": "Спортно зала Христо Ботев",
    "TB1976": "УЛ. КАМЕН АНДРЕЕВ",
    "A0384": "БУЛ. ХРИСТО БОТЕВ",
    "TM2577": "УЛ. БЕЛИ ДУНАВ",
    "A0214": "БЛ. 427 Ж.К. МЛАДОСТ-4",
    "A0047": "АВТОБАЗА ИСКЪР",
    "A6221": "МЕТРОСТАНЦИЯ К. ВЕЛИЧКОВ",
    "A0156": "БЛ. 205 Ж.К. МЛАДОСТ-2",
    "A0554": "ДЪРЖАВНА ПЕЧАТНИЦА",
    "A2119": "УЛ. ПОЛК.СТ.БАЧИЙСКИ",
    "A2316": "ХЛЕБОЗАВОДА - ПО ЖЕЛАНИЕ",
    "A0995": "КЪМПИНГ ЧЕРНИЯ КОС - ПО ЖЕЛАНИЕ",
    "A0426": "КВ. МАЛИНОВА ДОЛИНА - ПО ЖЕЛАНИЕ",
    "A1686": "ПРОМИШЛЕНА ЗОНА - ПО ЖЕЛАНИЕ",
    "A2025": "УЛ. ЛЕШНИКОВА ГОРА",
    "A1086": "МОСТА НА Р. СТУБЕЛА",
    "TM6213": "УЛ. ОБИКОЛНА",
    "A6530": "Суходолска река-временна",
    "A6708": "МЕТРОСТАНЦИЯ БЕЛИ ДУНАВ",
    "TB2540": "УЛ. БЕСАРАБИЯ",
    "A1625": "В.З. КИТКАТА - ПО ЖЕЛАНИЕ",
    "A1136": "НДК",
    "A1845": "УЛ. БОРЯНА",
    "A0448": "ГАРА ВЛАДАЯ",
    "A0774": "ИНСТИТУТА",
    "A2403": "МОСТА НА Р.БЛАЗНА БАРА",
    "A6129": "УЛ. РАЙСКА ГРАДИНА",
    "A1119": "НАЧАЛО КВ. ГРАДОМАН",
    "A2299": "ТЪРГОВСКИ К-С НАДЕЖДА",
    "TB2833": "РАЗКЛОНА ЗА С. ГЕРМАН",
    "A0685": "МЕТРОСТАНЦИЯ ГОРНА БАНЯ",
    "A6527": "Ул.Момина сълза-временна",
    "A1960": "УЛ. ИНДИРА ГАНДИ",
    "TB0376": "БУЛ. СЛИВНИЦА",
    "TB2070": "УЛ. ОБОРИЩЕ",
    "TM1709": "ТЪРГОВСКИ К-С ОБЕЛЯ",
    "A1079": "МОСТА НА ЛЕСНОВСКА РЕКА",
    "TM2018": "УЛ. КУБАДИН",
    "A2631": "УЛ. ВИШНОВСКА - ПО ЖЕЛАНИЕ",
    "A2762": "ЦАРЕВА МАХАЛА",
    "A0162": "БЛ. 207 Ж.К. ОБЕЛЯ-2",
    "A0105": "БЛ. 33 Ж.К. СЛАТИНА",
    "TM2109": "Ул. Нишава",
    "A1723": "ПГВАД",
    "A6626": "УЛ. 759",
    "A0870": "УЛ. ТОПЛИКА - ПО ЖЕЛАНИЕ",
    "TB0025": "28-МИ ДКЦ",
    "TB0128": "БЛ. 108 Ж.К. ДРУЖБА-1",
    "A6262": "Кино Арена-временна",
    "A6562": "Ул.Алеко Константинов-временна",
    "TM0473": "Гара Подуяне",
    "TM0877": "Кв. Овча купел",
    "A2338": "ЦЕНТЪР КВ. АБДОВИЦА",
    "A0624": "Ж.К. ИВАН ВАЗОВ",
    "A1375": "РАЗКЛОНА КВ. КРЕМИКОВЦИ",
    "A2848": "НАЦИОНАЛНА ФУТБОЛНА БАЗА",
    "TM2104": "УЛ. ПЛАНИНЕЦ",
    "A2039": "УЛ. ЛЮБОТРЪН",
    "A6301": "Ул.Вишовград-временна",
    "A6731": "СК ЦСКА - ВРЕМЕННА",
    "TB0515": "ДЕПО ИСКЪР",
    "A1804": "УЛ. 507",
    "TM0343": "Бул. Никола Вапцаров",
    "A0950": "КМЕТСТВО СЕЛО ЯНА",
    "A1429": "РАЗКЛОНА ЗА С. ГЕРМАН",
    "TB2304": "ХАНЧЕТО - ПО ЖЕЛАНИЕ",
    "A0087": "БАЗА ГАЗСТРОЙ - ПО ЖЕЛАНИЕ",
    "A1354": "ПО ЖЕЛАНИЕ",
    "A1387": "ПОДСТАНЦИЯ МЕТАЛУРГИЧНА - ПО ЖЕЛАНИЕ",
    "A0101": "БАНЯ ПАНЧАРЕВО - ПО ЖЕЛАНИЕ",
    "A0138": "БЛ. 135 Ж.К. ЛЮЛИН-10",
    "TB0160": "БЛ. 206 Ж.К. ДРУЖБА-2",
    "A1238": "ПАМЕТНИКА С. ДОБРОСЛАВЦИ",
    "TM2235": "Ул. Хемус",
    "A1674": "СТОПАНСТВО МИХАЙЛОВО - ПО ЖЕЛАНИЕ",
    "A0245": "БЛ. 66 Ж.К. МЛАДОСТ-1",
    "A0697": "Ж.П.ПРЕЛЕЗ",
    "A1033": "МГУ СВ.ИВ.РИЛСКИ",
    "A6297": "ЦЕНТЪРА БАНКЯ",
    "A6694": "ГРОБИЩЕН ПАРК ДРАГАЛЕВЦИ - ВРЕМЕННА",
    "TM1324": "Хотел Хемус",
    "A0503": "ЦЕНТЪРА БАНКЯ",
    "A6714": "Ул. Методи Иванов",
    "A2521": "ЕКОБУЛСОРТ - ПО ЖЕЛАНИЕ",
    "TB1245": "ПАРАКЛИС СВ.ИВ.РИЛСКИ",
    "TB0176": "БЛ. 255 Ж.К. ДРУЖБА-2",
    "A2152": "УЛ. РАДЕЦКИ",
    "A6367": "БУЛ. ЧЕРНИ ВРЪХ",
    "A1542": "СЕЛО КРИВИНА",
    "TM0273": "БУЛ. АКАД. ИВ. ГЕШОВ",
    "TB2116": "Надлез Надежда",
    "A1440": "МАХАЛА ЯПАДЖА",
    "A1860": "УЛ. ВАСИЛ КЪНЧЕВ",
    "TB0661": "Ж.К. МЛАДОСТ-1",
    "A0666": "Ж.К. МЛАДОСТ-3",
    "A2155": "УЛ. РЕЗБАРСКА",
    "TB0580": "ж.к. Борово",
    "A2610": "БУЛ. Д-Р ПЕТЪР ДЕРТЛИЕВ",
    "A6438": "Пл.Васил Левски",
    "TB0882": "КВ. ОВЧА КУПЕЛ",
    "A1319": "ПЛ. СТОЧНА ГАРА",
    "A2046": "УЛ. МИЛИН КАМЪК",
    "TB1197": "УМБАЛ СВ. АННА-УХО",
    "A1673": "СТОПАНСТВО МИХАЙЛОВО - ПО ЖЕЛАНИЕ",
    "TB0129": "БЛ. 108 Ж.К. ДРУЖБА-1",
    "A1026": "МАХАЛА ИВАНЯНЕ - ПО ЖЕЛАНИЕ",
    "A0128": "БЛ. 108 Ж.К. ДРУЖБА-1",
    "A0239": "БЛ. 603 Ж.К. ВРЪБНИЦА-2",
    "A0222": "БЛ. 458 Ж.К. НАДЕЖДА-4",
    "A1139": "НАЦИОНАЛЕН ДВОРЕЦ НА КУЛТУРАТА",
    "A6299": "УЛ. АБОБА",
    "A0063": "АВТОСТАНЦИЯ ОРЛАНДОВЦИ",
    "A1236": "ЦЕНТЪР КВ. БОТУНЕЦ",
    "A2353": "НЦ ПО РАДИОБИОЛОГИЯ - ПО ЖЕЛАНИЕ",
    "A6623": "УЛ. КНЯЖЕВСКА",
    "TB6006": "УЛ. ШУМЕН",
    "A2772": "СЕЛО ЖЕЛЯВА",
    "A6065": "ДЕТСКИ СТАЦИОНАР - ПО ЖЕЛАНИЕ",
    "A2206": "УЛ. Т. КАБЛЕШКОВ",
    "TM0619": "Ж.К. ЗАПАДЕН ПАРК",
    "A0577": "БУЛ. ХРИСТОФОР КОЛУМБ",
    "TM1056": "Метростанция Люлин",
    "A0657": "Ж.К. ЛЮЛИН-5",
    "A6073": "ОКОЛОВРЪСТЕН ПЪТ-временна",
    "A0954": "КМЕТСТВО ДОЛНИ БОГРОВ",
    "TB1462": "Р-Т АЛЕКСАНДРИЯ",
    "A0213": "БЛ. 427 Ж.К. МЛАДОСТ-4",
    "A6630": "ОКОЛОВРЪСТЕН ПЪТ",
    "TB0635": "Ж.К. КРАСНА ПОЛЯНА",
    "A1957": "УЛ. ЛЮЛЯК - ПО ЖЕЛАНИЕ",
    "A2434": "Ж. К. ОБЕЛЯ 1",
    "TB2417": "БЛ. 226 Ж.К. ДРУЖБА-2",
    "TB0178": "БЛ. 255 Ж.К. ДРУЖБА-2",
    "TM0624": "Ж.К. ИВАН ВАЗОВ",
    "TM0719": "НУ ЗА ТАНЦОВО ИЗКУСТВО",
    "A0102": "БАНЯ ПАНЧАРЕВО - ПО ЖЕЛАНИЕ",
    "TB0086": "КОМПЛЕКС НА БАН",
    "TB2767": "Ж.К. БОРОВО",
    "A0755": "СТОКОВ БАЗАР ИЛИЯНЦИ",
    "A1491": "РУМЪНСКО ПОСОЛСТВО",
    "A1391": "БЛ. 11 Ж.К. ЛЮЛИН-1",
    "A2634": "УЛ. РАДИВОЯ",
    "A6666": "Ул.Бели брод-временна",
    "A0487": "86-ТИ ПОЩЕНСКИ КЛОН",
    "TM2178": "УЛ. СОЛУНСКА",
    "A0997": "КЪМПИНГА - ПО ЖЕЛАНИЕ",
    "TB0090": "ТЕХНОТЕСТ",
    "A1690": "МОСТА НА ЛЕСНОВСКА РЕКА",
    "A6595": "НДК временна",
    "A0986": "КРАЯ С. ЖИТЕН",
    "A1946": "УЛ. ЗЛАТИЦА",
    "A1939": "УЛ. ДУНАВ",
    "A6533": "Ул.Градина-временна",
    "A0357": "БУЛ. ПЕНЧО СЛАВЕЙКОВ",
    "TM0928": "ЧИТАЛИЩЕ Д-Р П. БЕРОН",
    "A2450": "Ж.К. МАГНОЛИЯ - ПО ЖЕЛАНИЕ",
    "A0180": "БЛ. 28 Ж.К. ЛЕВСКИ-Г",
    "A2694": "МАГИСТРАЛА ХЕМУС",
    "A1623": "СТАДИОНА КАЗИЧЕНЕ",
    "A1919": "МЕТРОСТАНЦИЯ АКАД. А. Т.- БАЛАН",
    "A2531": "РАЗКЛОНА ЗА КВ. СЕСЛАВЦИ",
    "TM0595": "Ж.К. ГЕО МИЛЕВ",
    "A0402": "БУЛ. ШИПЧЕНСКИ ПРОХОД",
    "A0408": "В.З. БУНКЕРА - ПО ЖЕЛАНИЕ",
    "TM1453": "Разклона за летището",
    "TB0601": "УЛ. ТОДОР КАБЛЕШКОВ",
    "A0219": "БЛ. 437 Ж.К. МЛАДОСТ-4",
    "A6432": "ТИХИЯ КЪТ - ПО ЖЕЛАНИЕ",
    "A2711": "КАНТОНА",
    "TB0487": "86-ти пощенски клон",
    "A2006": "УЛ. КОСТА ЛУЛЧЕВ",
    "A2417": "БЛ. 226 Ж.К. ДРУЖБА-2",
    "A2530": "РАЗКЛОНА ЗА КВ. СЕСЛАВЦИ",
    "TM6102": "ПЛ. ВАСИЛ ЛЕВСКИ",
    "TM2574": "УЛ. ГЕН. НИКОЛА ЖЕКОВ",
    "A0065": "БУЛ. ИСТОРИЯ СЛАВЯНОБЪЛГАРСКА",
    "TM1115": "Национална опера",
    "A0479": "ГАРА ЯНА",
    "A2465": "УЛ. МАК",
    "TB0909": "МЕТРОСТАНЦИЯ ВИТОША",
    "A1753": "УЛ. ВИТИНЯ",
    "TB0180": "БЛ. 28 Ж.К. ЛЕВСКИ-Г",
    "TB0776": "УЛ. КАЛОЯНОВО",
    "A1327": "ЦЕНТРАЛНА ГАРА",
    "TB0542": "МОДНА КЪЩА АГРЕСИЯ",
    "A2493": "КИНО АРЕНА ІІ",
    "A1786": "С. МЪРЧАЕВО-ТОЛУМСКА МАХАЛА",
    "A2247": "УЛ. ЦАР СИМЕОН",
    "A6573": "Новата гора на София (Суходол)",
    "A1508": "ЧЕШМАТА - ПО ЖЕЛАНИЕ",
    "A0421": "ВЕЦ ПАСАРЕЛ - ПО ЖЕЛАНИЕ",
    "A0752": "М - ТЕЛ",
    "A2771": "УЛ. ГЕРЕНА",
    "A0246": "БЛ. 66 Ж.К. МЛАДОСТ-1",
    "A1388": "БЛ. 11 Ж.К. ЛЮЛИН-1",
    "TB2011": "УЛ. КОСТЕНСКИ ВОДОПАД",
    "TB6220": "ЦЕНТРАЛНА ГАРА",
    "A0287": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "A0881": "КВ. ОВЧА КУПЕЛ",
    "TB0159": "БЛ. 206 Ж.К. ДРУЖБА-2",
    "A1661": "КРАЯ НА С. ЖЕЛЕЗНИЦА - ПО ЖЕЛАНИЕ",
    "A1755": "ОБЩИНА КРЕМИКОВЦИ",
    "A1768": "ПЪТЕН ВЪЗЕЛ ЯНА - ПО ЖЕЛАНИЕ",
    "A6066": "ДЕТСКИ СТАЦИОНАР - ПО ЖЕЛАНИЕ",
    "A0678": "Ж.К. ОБЕЛЯ-2",
    "A1640": "ПРОМИШЛЕНА ЗОНА БУСМАНЦИ",
    "A6159": "БУЛ. ЛУИ ПАСТЬОР",
    "A1936": "УЛ. ДОЙРАН",
    "TB1743": "ТЕХНИЧЕСКИ УНИВЕРСИТЕТ",
    "A1096": "МОСТА НА Р. ИСКЪР КВ. АБДОВИЦА",
    "A6224": "Ул.Тодор Каблешков-временна",
    "TB2057": "УЛ. НАРОДНО ХОРО",
    "A1992": "УЛ. КИТКАТА - ПО ЖЕЛАНИЕ",
    "A6226": "Ж.К. МОТОПИСТА",
    "A2690": "ТИР ПАРКИНГ- МРАМОР - ПО ЖЕЛАНИЕ",
    "TB1470": "БЛ. 4  Ж.К. СУХАТА РЕКА",
    "TM0335": "Ул. Луи Пастьор",
    "A0148": "БЛ. 20 Ж.К. СВОБОДА",
    "A0571": "ЕЛЕКТРОТЕРМИЯ - ПО ЖЕЛАНИЕ",
    "A2439": "МОСТА НА Р. ИСКЪР КВ. ВРАЖДЕБНА",
    "TB2753": "БЛ. 458 Ж.К. МЛАДОСТ 4",
    "A2845": "Ж.К. ГОЦЕ ДЕЛЧЕВ",
    "A2113": "УЛ. ПИРОТСКА",
    "TB1196": "УМБАЛ СВ. АННА",
    "A1906": "УЛ. Г. БЕНКОВСКИ",
    "TB1753": "УЛ. ВИТИНЯ",
    "A1520": "СЕЛО ГОРНИ БОГРОВ",
    "TB6277": "УЛ. КОСТЕНСКИ ВОДОПАД",
    "TM2398": "БЛ. 539 Ж.К.ЛЮЛИН-5",
    "A6528": "Ул.Градина-временна",
    "TM0655": "Ж.К. ЛЮЛИН-5",
    "A1483": "МОСТА НА СУХАТА РЕКА",
    "A0090": "ТЕХНОТЕСТ",
    "A0665": "Ж.К. МЛАДОСТ-3",
    "TB2593": "ХЕРМЕС ПАРК",
    "A2831": "192 - РО СОУ С. БИСТРИЦА",
    "A2220": "УЛ. ТЪРНОВО",
    "A0416": "ВСУ Л.КАРАВЕЛОВ",
    "A0677": "Ж.К. ОБЕЛЯ-1",
    "A0994": "КЪМПИНГ ЧЕРНИЯ КОС - ПО ЖЕЛАНИЕ",
    "A6640": "СЕЛО КАЗИЧЕНЕ",
    "A0250": "БЛ. 7 Ж.К. ХР. СМИРНЕНСКИ",
    "TB1011": "ЛИФТА СИМЕОНОВО",
    "A6418": "Пл.Руски паметник",
    "TB0881": "КВ. ОВЧА КУПЕЛ",
    "A2670": "МОДНА КЪЩА АГРЕСИЯ",
    "TB0297": "БУЛ. ВАСИЛ ЛЕВСКИ",
    "TB1242": "ПЛ. ВАСИЛ ЛЕВСКИ",
    "TB0192": "БЛ. 32 Ж.К. ОВЧА КУПЕЛ-2",
    "A6439": "ПЛ. СВ. АЛЕКСАНДЪР НЕВСКИ",
    "A1945": "ЦЕНТРАЛЕН ГРОБИЩЕН ПАРК",
    "A6561": "Ул.Г.С.Раковски-временна",
    "A1656": "КРАЯ С. ДОБРОСЛАВЦИ",
    "A6653": "ГАРА ОБЕЛЯ",
    "A0167": "БЛ. 222 Ж.К. МЛАДОСТ-2",
    "A0661": "Ж.К. МЛАДОСТ-1",
    "A2000": "УЛ. КОЗЛОДУЙ",
    "TB1258": "ПЛОЩАД  НА АВИАЦИЯТА",
    "A1991": "УЛ. КИТКАТА - ПО ЖЕЛАНИЕ",
    "A0897": "КВ. СЕСЛАВЦИ",
    "TM1153": "СК ЦСКА",
    "TM1332": "ЦЕНТРАЛНА ГАРА",
    "A6020": "Ж.К. ГЕО МИЛЕВ",
    "A0003": "БЛ. 411 Ж.К. МЛАДОСТ-4",
    "A0423": "КВ. МАЛИНОВА ДОЛИНА - ПО ЖЕЛАНИЕ",
    "TB2792": "РАЗКЛОНА ЗА ЛИФТ СИМЕОНОВО",
    "A6128": "УЛ. РАЙСКА ГРАДИНА",
    "A2426": "МВР - АКАДЕМИЯ",
    "A2018": "УЛ. КУБАДИН",
    "A2442": "БЛ. 119 Ж.К. ОБЕЛЯ-1",
    "A6319": "НУ за танцово изкуство",
    "A0542": "МОДНА КЪЩА АГРЕСИЯ",
    "A6534": "УЛ. КИРИЛ ХРИСТОВ",
    "TB0754": "СТОКОВ БАЗАР ИЛИЯНЦИ",
    "TB2054": "УЛ. НАРОДНО ХОРО",
    "A6643": "СЕЛО КАЗИЧЕНЕ",
    "A2479": "УЛ. 5010",
    "A0772": "ИНСТИТУТ ПУШКАРОВ - ПО ЖЕЛАНИЕ",
    "TM0645": "Ж.к. Лагера",
    "A1042": "МЕТАЛСНАБ ХОЛДИНГ",
    "TB0648": "Ж.К. ЛОЗЕНЕЦ",
    "A0790": "КАНТОНА - ПО ЖЕЛАНИЕ",
    "A1148": "УЛ. ЮРИЙ ГАГАРИН",
    "TM0891": "Кв. Павлово",
    "A2322": "ХОТЕЛ ГОРНА БАНЯ",
    "A6192": "Бл.214, ж.к.Борово-временна",
    "A6258": "Разклона за ж.к. Младост-временна",
    "A0227": "БЛ. 505 Ж.К. ОВЧА КУПЕЛ-1",
    "A1561": "КВ. ЧЕЛОПЕЧЕНЕ",
    "A1232": "ЦЕНТЪРА С. КРИВИНА",
    "A6576": "МАНАСТИР СВ. ПЕТКА ПАРАСКЕВА",
    "A2174": "УЛ. СИНЯТА СКАЛА - ПО ЖЕЛАНИЕ",
    "A0143": "БЛ. 162 Ж.К. ДРУЖБА-1",
    "A2016": "УЛ. КРУМ СТОЯНОВ",
    "A2563": "МЕТРОДЕПО",
    "A2527": "КВ. РЕПУБЛИКА-2",
    "A1726": "ТЕАТЪР СОФИЯ",
    "A0202": "БЛ. 351Б Ж.К. МЛАДОСТ-3",
    "A2007": "УЛ. КОСТА ЛУЛЧЕВ",
    "A6553": "ПЛ. ВЪЗРАЖДАНЕ",
    "A0130": "БЛ. 11 Ж.К. ЛЕВСКИ-В",
    "A1756": "ОБЩИНА КРЕМИКОВЦИ",
    "TB0127": "БЛ. 10 Ж.К. ЛЕВСКИ-Г",
    "A2116": "НАДЛЕЗ НАДЕЖДА",
    "TM1330": "ЦЕНТРАЛНА ГАРА",
    "A0403": "БУЛ. ШИПЧЕНСКИ ПРОХОД",
    "TM1879": "УЛ. ВИШНЕВА",
    "TM6104": "Бул.Христо Ботев",
    "A6457": "УЛ. 6-ТИ СЕПТЕМВРИ",
    "A1591": "СКИОРКАТА - ПО ЖЕЛАНИЕ",
    "A0705": "Ж.П.ПРЕЛЕЗ",
    "A1745": "УЛ. ПОДПОР.ЙОРДАН ТОДОРОВ",
    "A2302": "ХАНЧЕТО",
    "A2397": "Ж.К. ЛЮЛИН-8",
    "A2816": "УЛ. БОТЕВГРАДСКО ШОСЕ",
    "TB1114": "НАДЛЕЗ НАДЕЖДА",
    "A1761": "ТПК ХЕМУС",
    "TB0251": "БЛ. 706 Ж.К. ЛЮЛИН-7",
    "TM0922": "Читалище Пенчо П. Славейков 1921",
    "TM2059": "УЛ. Н. КОПЕРНИК",
    "A1807": "УЛ. 6-ТИ СЕПТЕМВРИ",
    "TB0339": "БУЛ. ГЕН.Н.СТОЛЕТОВ",
    "A1744": "УЛ. ПОДПОР.ЙОРДАН ТОДОРОВ",
    "A2169": "УЛ. САН СТЕФАНО",
    "A1836": "УЛ. БАЙКАЛ",
    "A6719": "СТУДЕНТСКИ ГРАД",
    "A2633": "УЛ. РАДИВОЯ",
    "A6380": "Ж.К. МЛАДОСТ-4",
    "TB0603": "Ж.К. ДИАНАБАД",
    "TB0860": "ЮЖЕН ПАРК",
    "TB0861": "ЮЖЕН ПАРК",
    "A1706": "ПАЗАРА ОВЧА КУПЕЛ",
    "TB2482": "УЛ. ВИТИНЯ",
    "A2394": "БЛ. 480 Ж.К. МЛАДОСТ-4",
    "A1791": "УЛ. 10",
    "A6431": "ТИХИЯ КЪТ - ПО ЖЕЛАНИЕ",
    "A1687": "МОСТА НА ЛЕСНОВСКА РЕКА",
    "TB0316": "БУЛ. Д-Р Г. М. ДИМИТРОВ",
    "TB2125": "Ж.К. ЛЕВСКИ-В",
    "TB0007": "12-ТИ ДКЦ",
    "A1348": "НАЧАЛО КВ Д. МИЛЕНКОВ",
    "A1604": "НПМГ АКАД. ЛЮБОМИР ЧАКАЛОВ",
    "A1764": "ПГХМБТ ПРОФ. П. РАЙКОВ",
    "A0605": "Ж.К. ДРУЖБА-1",
    "A0611": "Ж.К. ДРУЖБА-2",
    "A0723": "УЛ. ПОРУЧИК ХР.ТОПРАКЧИЕВ",
    "A2819": "ПАМЕТНИКА С. ЯНА",
    "TB0370": "БУЛ. ПРОФ.ЦВ. ЛАЗАРОВ",
    "TB2679": "МЕТРОСТАНЦИЯ Г. М. ДИМИТРОВ",
    "A1954": "УЛ. ИВАН ЩЕРЕВ",
    "A2728": "ЕЗЕРОТО - СЕЛО КАЗИЧЕНЕ",
    "A2147": "УЛ. ПРОФ. ИВ. ШИШМАНОВ",
    "TB1327": "ЦЕНТРАЛНА ГАРА",
    "A1027": "МАХАЛА ИВАНЯНЕ - ПО ЖЕЛАНИЕ",
    "TB2141": "Ул. Промишлена",
    "TM0321": "БУЛ. ПЕТЪР ДЕРТЛИЕВ",
    "A6150": "Временна А 123",
    "TB0651": "Ж.К. ЛЮЛИН-3",
    "TB2140": "УЛ. ПРОМИШЛЕНА",
    "A1734": "ТЕРСНАБ",
    "TB0555": "ДЪРЖАВНА ПЕЧАТНИЦА",
    "TB0287": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "TB0369": "БУЛ. ПРОФ.ЦВ. ЛАЗАРОВ",
    "TM0349": "Бул. Никола Петков",
    "TB1823": "ул. Александър Жендов",
    "TM1997": "ПЛ. ГАРИБАЛДИ",
    "A2385": "МАГАЗИН МОСЮ БРИКОЛАЖ",
    "A0446": "УЛ. А. КОНСТАНТИНОВ - ПО ЖЕЛАНИЕ",
    "A0429": "В.З. КВ. БОЯНА",
    "A6568": "ДЕНДРАРИУМА - ПО ЖЕЛАНИЕ",
    "A0104": "БАРАЖА ЯЗ. ПАСАРЕЛ - ПО ЖЕЛАНИЕ",
    "TB0165": "БЛ. 214 Ж.К. БОРОВО",
    "A1249": "ПАРК СВ.СВ.ПЕТЪР И ПАВЕЛ",
    "A1566": "ПГСС",
    "A2599": "УЛ. КОМ",
    "A6609": "БУЛ. ВАРДАР - ВРЕМЕННА",
    "A0393": "БУЛ. ЦАР БОРИС ІІІ",
    "A6587": "УЛ. АТАНАС МАНЧЕВ",
    "A1639": "КРАЯ С. БИСТРИЦА",
    "TM1304": "ПЛ. РУЧЕЙ",
    "TM1822": "Ул. Алеко Константинов",
    "A2149": "УЛ. ПРОФ.И.ГЕОРГОВ",
    "TB1977": "УЛ. КАМЕН АНДРЕЕВ",
    "TB0143": "БЛ. 162 Ж.К. ДРУЖБА-1",
    "TB1699": "СУ СВ.КЛИМЕНТ ОХРИДСКИ",
    "A2495": "ТЕХНОПОЛИС Ж.К. МЛАДОСТ-4",
    "TM2072": "Ул. Оборище",
    "A0714": "ПАРКА С.КРИВИНА",
    "A0490": "УЛ. Т. КАБЛЕШКОВ",
    "A0114": "БЕТОНОВ ВЪЗЕЛ - ПО ЖЕЛАНИЕ",
    "A0127": "БЛ. 10 Ж.К. ЛЕВСКИ-Г",
    "TB1706": "ПАЗАРА ОВЧА КУПЕЛ",
    "A2543": "БУЛ. ЦАР БОРИС ІІІ",
    "A0252": "БЛ. 706 Ж.К. ЛЮЛИН-7",
    "TM1775": "ТРИЪГЪЛНИКА",
    "TM2335": "Централни хали",
    "A0691": "Ж.К. ХАДЖИ ДИМИТЪР",
    "A1140": "НЕСТЛЕ-СОФИЯ",
    "A0404": "БУЛАВТО",
    "A1417": "УЛ. МАЛА КОРИЯ - ПО ЖЕЛАНИЕ",
    "A2240": "УЛ. ХРИСТО СИЛЯНОВ",
    "A6160": "ОБЩИНА ЛЮЛИН",
    "A6818": "СТОКОВ БАЗАР ИЛИЯНЦИ",
    "A0133": "БЛ. 111 Ж.К. ОБЕЛЯ-1",
    "A2710": "КАНТОНА",
    "A2002": "УЛ. КОЗЛОДУЙ",
    "TM1710": "Търговски к-с Обеля",
    "A2237": "УЛ. ХОРИЗОНТ",
    "TB1164": "БЛ. 21 Ж.К. ЛЮЛИН-1",
    "TB1822": "Ул. Алеко Константинов",
    "A0575": "ЕНЕРГОПРОЕКТ",
    "A2424": "Ж.К. ДРУЖБА -1",
    "TB2455": "ЛЕТИЩЕ СОФИЯ ТЕРМИНАЛ 2",
    "A6092": "ПЛ. МАКЕДОНИЯ",
    "TB2235": "ул. Хемус",
    "TM0709": "БУЛ. АЛ. СТАМБОЛИЙСКИ",
    "A0588": "Ж.К. БЪКСТОН",
    "TB1423": "ПЪРВА ГРАДСКА БОЛНИЦА",
    "A2287": "160 -ТО ОУ",
    "A6349": "УЛ. ЖАН ДЬО ЛАФОНТЕН",
    "A0124": "БЛ. 22 Ж.К. ДИАНАБАД БЛ. 1",
    "A2001": "УЛ. КОЗЛОДУЙ",
    "A2797": "ЛЕТИЩЕН ЦЕНТЪР СОФИЯ",
    "A6063": "БЯЛАТА ВОДА - ПО ЖЕЛАНИЕ",
    "A0223": "БЛ. 465 Ж.К. МЛАДОСТ-4",
    "TM2046": "Ул. Милин камък",
    "A6178": "УЛ. КАП. Д. СПИСАРЕВСКИ",
    "A1385": "БУЛ. ИЛИЯНЦИ-ПОДЛЕЗА",
    "A6669": "НАДЛЕЗ НАДЕЖДА",
    "TM2179": "УЛ. СОЛУНСКА",
    "A0005": "БЛ. 411 Ж.К. МЛАДОСТ-4",
    "TM2575": "бул. Рожен",
    "TB0210": "БЛ. 41 Ж.К. МЛАДОСТ-1",
    "A6577": "Новата гора на София (Суходол)",
    "TB1333": "ЦЕНТРАЛНА ГАРА",
    "A1451": "РАЗКЛОНА ЗА ЛЕТИЩЕТО",
    "TM0606": "Ж.К. ДРУЖБА-1",
    "A0663": "Ж.К. МЛАДОСТ-2",
    "A0683": "МЕТРОСТАНЦИЯ ГОРНА БАНЯ",
    "A1088": "МОСТА ДОЛНИ ПАСАРЕЛ",
    "A0535": "ДЕТСКО ГРАДЧЕ - ПО ЖЕЛАНИЕ",
    "TB0744": "СОФАРМА АД",
    "TM1248": "ПАРК СВ.СВ.ПЕТЪР И ПАВЕЛ",
    "A0039": "60-ТО ОУ",
    "A6531": "Суходолска река-временна",
    "TM0680": "Ж.К. ОБЕЛЯ-2",
    "TM1409": "28-ми пощенски клон",
    "A6518": "МОСТА КВ. ВРАЖДЕБНА",
    "TB2850": "УЛ. ЧАВДАР МУТАФОВ",
    "A2120": "УЛ. ПОЛК.СТ.БАЧИЙСКИ",
    "TM1880": "УЛ. ВИШНЕВА",
    "TM6560": "Автостанция Изток",
    "A1272": "ПЛ. ВЪЗРАЖДАНЕ",
    "A2264": "УНИКАЛНИ КОНСТРУКЦИИ - ПО ЖЕЛАНИЕ",
    "A2766": "БУЛ. Д-Р ПЕТЪР ДЕРТЛИЕВ",
    "A6821": "УЛ. ПАНАЙОТ ПИПКОВ - ВРЕМЕННА",
    "TB2626": "Околовръстен път",
    "TM1292": "Пл. Пирдоп",
    "A1443": "ХАЙДУШКИ ЧУКАР - ПО ЖЕЛАНИЕ",
    "TB0554": "ДЪРЖАВНА ПЕЧАТНИЦА",
    "A1250": "ПАРК СВ.СВ.ПЕТЪР И ПАВЕЛ",
    "TM1970": "УЛ. ЙОСИФ ЩРОСМАЙЕР",
    "A0071": "АЛБЕНА - ПО ЖЕЛАНИЕ",
    "A0810": "В.З. БЕЛОВОДСКИ ПЪТ - ПО ЖЕЛАНИЕ",
    "A1812": "УЛ. 61",
    "A2642": "УЛ. МАЕСТРО АТАНАСОВ-ПО ЖЕЛАНИЕ",
    "A1799": "КУЛТУРЕН ЦЕНТЪР ЛЮЛИН",
    "A2308": "ХИПОДРУМА БАНКЯ - ПО ЖЕЛАНИЕ",
    "TB2606": "УЛ. ЛЮБЛЯНА",
    "TM2129": "Ул. Поп Груйо",
    "A1904": "УЛ. ГЕО МИЛЕВ",
    "TM2159": "Ул. Рилска обител",
    "TB1886": "УЛ. ГЕОРГИ РАКОВСКИ",
    "A1596": "УЛ. ТИНТЯВА",
    "TM0607": "Ж.К. ДРУЖБА-1",
    "A0551": "СТИНД",
    "A6214": "УЛ. ХАН КУБРАТ",
    "A2650": "УЛ. РАЧО ПЕТКОВ КАЗАНДЖИЯТА",
    "A2820": "ПАМЕТНИКА С. ЯНА",
    "A6709": "АВТОСТАНЦИЯ ГЕО МИЛЕВ",
    "A0172": "БЛ. 241 Ж.К. ОБЕЛЯ-2",
    "A0704": "Ж.П.ПРЕЛЕЗ",
    "TB1476": "БЛ. 6 Ж.К. СУХАТА РЕКА",
    "A1935": "УЛ. ДОЙРАН",
    "A6647": "Бл. 202, ж.к. Обеля 2 - временна",
    "A6127": "УЛ. ПЛАНИНЕЦ",
    "A6198": "Ул.Лясковец-временна",
    "TB0206": "МБАЛ ТОКУДА БОЛНИЦА",
    "TM0469": "Гара Подуяне",
    "A0799": "КАУЧУКОВ ЗАВОД ЗЕБРА - ПО ЖЕЛАНИЕ",
    "A2420": "УЛ. ПЛОВДИВ",
    "A2512": "РАЗКЛОНА ЗА С. ДОЛНИ БОГРОВ",
    "TB0812": "МЕТРОСТАНЦИЯ МИЗИЯ",
    "TB1463": "Р-Т АЛЕКСАНДРИЯ",
    "TB1798": "КУЛТУРЕН ЦЕНТЪР ЛЮЛИН",
    "A2032": "УЛ. ЛИЛЯЧЕ",
    "A1475": "БЛ. 6 Ж.К. СУХАТА РЕКА",
    "A1953": "УЛ. ИВАН КРАЛИЧА",
    "A6351": "КРАЯ НА КВ. МИХАЙЛОВО",
    "A1360": "МАХАЛА ЧЕМЕРНИК - ПО ЖЕЛАНИЕ",
    "TB0614": "ЛТУ",
    "TB0971": "Корабчето",
    "A1697": "СУ СВ.КЛИМЕНТ ОХРИДСКИ",
    "A2317": "ХЛЕБОЗАВОДА - ПО ЖЕЛАНИЕ",
    "A0951": "КМЕТСТВО СЕЛО ЯНА",
    "A1472": "ВОДЕНИЧАРСКИ МЕХАНИ-ПО ЖЕЛАНИЕ",
    "A6332": "УЛ. ВИТИНЯ - ВРЕМЕННА",
    "A0099": "БАНЯТА КВ. ГНИЛЯНЕ",
    "TB1136": "НДК",
    "TM1277": "ПЛ. ЛЪВОВ МОСТ",
    "TM6180": "УЛ. АЛАБИН",
    "A6335": "НОВАТА ГОРА НА СОФИЯ 2",
    "TB0500": "ГОРУБЛЯНСКО ХАНЧЕ",
    "TB2451": "Р-Т АЛЕКСАНДРИЯ",
    "TB6038": "МЕТРОСТАНЦИЯ ОВЧА КУПЕЛ",
    "A0210": "БЛ. 41 Ж.К. МЛАДОСТ-1",
    "A2803": "ЖП ГАРА ВЕРИЛА",
    "A0922": "ЧИТАЛИЩЕ ПЕНЧО П. СЛАВЕЙКОВ 1921",
    "A2235": "УЛ. ХЕМУС",
    "A2304": "ХАНЧЕТО - ПО ЖЕЛАНИЕ",
    "A0220": "БЛ. 437 Ж.К. МЛАДОСТ-4",
    "A0341": "БУЛ. НИКОЛА ВАПЦАРОВ",
    "A2207": "УЛ. Т. КАБЛЕШКОВ",
    "A6738": "УЛ.МАЕСТРО АТАНАСОВ - ВРЕМЕННА",
    "A1645": "ПРОМИШЛЕНА ЗОНА - ПО ЖЕЛАНИЕ",
    "A6064": "МУЗЕЙ НА МЕЧКАТА - ПО ЖЕЛАНИЕ",
    "A1469": "БЛ. 4 Ж.К. СУХАТА РЕКА",
    "TM0759": "УМБАЛСМ ПИРОГОВ",
    "TB2791": "РАЗКЛОНА ЗА ЛИФТ СИМЕОНОВО",
    "TB0692": "ж.к. Хаджи Димитър",
    "A2440": "МОСТА НА Р. ИСКЪР КВ. ВРАЖДЕБНА",
    "A0653": "Ж.К. ЛЮЛИН-3",
    "A0187": "БЛ. 302 Ж.К. НАДЕЖДА-3",
    "TB1517": "село Герман",
    "A0135": "БЛ. 116 Ж.К. ОБЕЛЯ-1",
    "TB0652": "Ж.К. ЛЮЛИН-3",
    "A1789": "ТЪРГОВСКИ Ц-Р ЛЮЛИН",
    "A6275": "БУЛ. БЪЛГАРИЯ",
    "A0075": "АНЕВА ЧЕШМА - ПО ЖЕЛАНИЕ",
    "A0088": "БАЗА ГАЗСТРОЙ - ПО ЖЕЛАНИЕ",
    "A0259": "УЛ. ЗЕЛЕН БОР - ПО ЖЕЛАНИЕ",
    "A6478": "бул. П.Евтимий вр.",
    "TM0673": "Ж.К. НАДЕЖДА-3",
    "A1456": "РАЗКЛОНА ЗА КВ.ДРАГАЛЕВЦИ",
    "A2245": "УЛ. БОР - ПО ЖЕЛАНИЕ",
    "A0553": "ДЪРЖАВНА ПЕЧАТНИЦА",
    "A1476": "БЛ. 6 Ж.К. СУХАТА РЕКА",
    "A2268": "КАЛФИН ДОЛ - ПО ЖЕЛАНИЕ",
    "TM1166": "10-ТИ ДКЦ",
    "A2084": "УЛ. ОПЪЛЧЕНСКА",
    "TB1451": "Разклона за летището",
    "A1795": "УЛ. 3",
    "A2208": "УЛ. Т. КАБЛЕШКОВ",
    "A6022": "УЛ. МИТРОПОЛИТ К. ВИДИНСКИ",
    "A1052": "МЕТРОСТАНЦИЯ К.ВЕЛИЧКОВ",
    "A1230": "ЯЗ. ПАСАРЕЛ - ПО ЖЕЛАНИЕ",
    "A1239": "ПАМЕТНИКА С. ДОБРОСЛАВЦИ",
    "A1598": "БАЛКАН СТАР",
    "TB6007": "УЛ. ШУМЕН",
    "TB0308": "БУЛ. ГОЦЕ ДЕЛЧЕВ",
    "A2695": "МАГИСТРАЛА ХЕМУС",
    "A7002": "Ул.Манастирска",
    "TB6268": "Ул.Скайлер-временна",
    "A0209": "БЛ. 41 Ж.К. МЛАДОСТ-1",
    "A6374": "Ул.Топлика-временна",
    "A6733": "УЛ. СКОПИЕ - ВРЕМЕННА",
    "A1901": "УЛ. ГЕН. НИКОЛА ЖЕКОВ",
    "A0888": "КВ. ПАВЛОВО",
    "A6621": "УЛ. БУРЯ",
    "A0142": "БЛ. 155 Ж.К. ДРУЖБА-1",
    "A1212": "РАЗКЛОНА ЗА Д.БОГРОВ",
    "A1322": "ХОТЕЛ ХЕМУС",
    "A2484": "СОФГЕОПРОУЧВАНЕ",
    "A2262": "УЛ. ЮРИЙ ГАГАРИН",
    "A6375": "Ул.Топлика-временна",
    "A1386": "ПОДСТАНЦИЯ МЕТАЛУРГИЧНА - ПО ЖЕЛАНИЕ",
    "TB1297": "ПЛ. РУСКИ ПАМЕТНИК",
    "A0913": "КВ. ХРИСТО БОТЕВ",
    "A0232": "БЛ. 527 Ж.К. ОВЧА КУПЕЛ-1",
    "TB0056": "АВТОСТАНЦИЯ ИЗТОК",
    "A0168": "БЛ. 222 Ж.К. МЛАДОСТ-2",
    "A0789": "КАНТОНА - ПО ЖЕЛАНИЕ",
    "TB1566": "ПГСС",
    "TB2113": "УЛ. ПИРОТСКА",
    "TB0975": "СОФИЯ МЕД АД",
    "TB0256": "БЛ. 98 Ж.К. ДРУЖБА-1",
    "A1448": "РЕЗИДЕНЦИЯ БОЯНА - ПО ЖЕЛАНИЕ",
    "TB2490": "МЕТРОСТАНЦИЯ БИЗНЕС ПАРК",
    "TB0900": "УЛ. ЛЮБЛЯНА",
    "A0734": "УЛ. 44 - ПО ЖЕЛАНИЕ",
    "A2185": "УЛ. СТАРА ПЛАНИНА",
    "A0570": "ЕЛЕКТРОТЕРМИЯ - ПО ЖЕЛАНИЕ",
    "A1464": "НИМ",
    "A6459": "МЕТРОСТАНЦИЯ СЕРДИКА",
    "TM6105": "Бул.Константин Величков",
    "A2847": "ПЛ. СРЕД СЕЛО",
    "A2451": "Р-Т АЛЕКСАНДРИЯ",
    "A1213": "РАЗКЛОНА ЗА Д.БОГРОВ",
    "A1455": "РАЗКЛОНА ЗА ЛЕТИЩЕТО",
    "A0938": "ЦЕНТЪРА КВ. ИЛИЯНЦИ - ПО ЖЕЛАНИЕ",
    "A6695": "ГРОБИЩЕН ПАРК ДРАГАЛЕВЦИ - ВРЕМЕННА",
    "TM2334": "Централни хали",
    "A2142": "УЛ. ПРОСВЕТА",
    "TB6278": "УЛ. КОСТЕНСКИ ВОДОПАД",
    "TM2038": "Ул. Люботрън",
    "A0137": "БЛ. 115 Ж.К. ЛЮЛИН-10",
    "A1432": "УЛ. РОДИНА - ПО ЖЕЛАНИЕ",
    "TM1049": "Метростанция Западен парк",
    "A6125": "УЛ. ПЛАНИНЕЦ",
    "A0686": "Ж.К. СВОБОДА",
    "A1160": "МЕТАЛИКА ООД",
    "A0713": "ПАРК С.КРИВИНА",
    "TB2781": "ЗООПАРКА",
    "A1774": "16-ТИ ДКЦ",
    "A2489": "БЛ. 325 Ж.К. МЛАДОСТ-3",
    "A2244": "УЛ. ЦАР БОРИС ІІІ",
    "A2388": "БЛ. 442 Ж.К. МЛАДОСТ-4",
    "TB0228": "БЛ. 505 Ж.К. ОВЧА КУПЕЛ-1",
    "A2859": "ЛОВЕН ПАРК",
    "A1843": "УЛ. БЕСАРАБИЯ",
    "TB2206": "УЛ. Т. КАБЛЕШКОВ",
    "TB2777": "МБАЛ ТОКУДА БОЛНИЦА",
    "A0197": "БЛ. 350 Ж.К. ЛЮЛИН-3",
    "A0199": "БЛ. 350 Ж.К. ЛЮЛИН-3",
    "A2177": "УЛ. СНЕЖНИК",
    "A1888": "115-ТО ОУ",
    "A2808": "УЛ. РЕПУБЛИКА",
    "A1413": "ПРЕЧИСТВАТЕЛНА СТАНЦИЯ - ПО ЖЕЛАНИЕ",
    "TM0593": "Ж.К. ВРЪБНИЦА-1",
    "A0254": "БЛ. 715 Ж.К. ЛЮЛИН-7",
    "A1298": "ПЛ. РУСКИ ПАМЕТНИК",
    "A1087": "МОСТА КВ. ВИТОША",
    "A0869": "УЛ. ТОПЛИКА - ПО ЖЕЛАНИЕ",
    "A0201": "БЛ. 351Б Ж.К. МЛАДОСТ-3",
    "A0242": "БЛ. 620 Ж.К. ЛЮЛИН-6",
    "A1053": "МЕТРОСТАНЦИЯ К.ВЕЛИЧКОВ",
    "TB0142": "БЛ. 155 Ж.К. ДРУЖБА-1",
    "A0115": "БЕТОНОВ ВЪЗЕЛ",
    "A0185": "БЛ. 301 Ж.К. ЛЮЛИН-3",
    "A2184": "УЛ. СТАРА ПЛАНИНА",
    "TB1604": "НПМГ АКАД. ЛЮБОМИР ЧАКАЛОВ",
    "TB1816": "УЛ. 9 - ПО ЖЕЛАНИЕ",
    "A0783": "УЛ. АСЕН ЙОРДАНОВ",
    "TB0153": "БЛ. 264 Ж.К. ЛЮЛИН-2",
    "TB1487": "БЛ. 405 Ж.К. ДРУЖБА-2",
    "A1237": "ПАМЕТНИКА С. ДОБРОСЛАВЦИ",
    "A2048": "УЛ. КОСМОНАВТ - ПО ЖЕЛАНИЕ",
    "A6261": "Кино Арена-временна",
    "TB1426": "РАЗКЛОНА ЗА С. ГЕРМАН",
    "A0929": "ЧИТАЛИЩЕ Д-Р П. БЕРОН",
    "TB0577": "БУЛ. ХРИСТОФОР КОЛУМБ",
    "A1009": "МАНАСТИР СВ. ЙОАКИМ И АНА - ПО ЖЕЛАНИЕ",
    "TM2002": "УЛ. КОЗЛОДУЙ"
}

    # Start input loop

    # Start input loop
# Obtain Occuapancy
occupancy_dict = {
    0: "Almost empty",
    1: "A lot of seats available",
    2: "Some seats available",
    3: "Mainly standing spaces available",
    4: "Some standing space available",
    5: "Full"
}
# Direction (Hopefully)
trip_dict = {}
with open("trips.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        route = row["route_id"]
        shape = row["shape_id"]
        headsign = row["trip_headsign"]
        
        
        if route not in trip_dict:
            trip_dict[route] = {}
        trip_dict[route][shape] = headsign
#Full Routes
routes={}
with open("shapes.csv", encoding="utf-8") as e:
    reader = csv.DictReader(e)
    for row in reader:
        route_id = row.get("shape_id")
        seq = row.get("shape_pt_sequence")
        lon = row.get("shape_pt_lon")
        lat = row.get("shape_pt_lat")
        if route_id not in routes:
            routes[route_id] = {}   
        routes[route_id][seq]=( lon, lat)

Code

In [ ]:

data = {
    "av": {"lon": [], "lat": [], "ke": [], "stop": [], "model": [], "occ":[], "dir":[], "ind": []},
    "tb": {"lon": [], "lat": [], "ke": [], "stop": [], "model": [], "occ":[], "dir":[], "ind": []},
    "tm": {"lon": [], "lat": [], "ke": [], "stop": [], "model": [], "occ":[], "dir":[], "ind": []},
}
d_av = {}
d_tb = {}
d_tm = {}
coords_av = []
coords_tb = []
coords_tm =[]
map = leafmap.Map()
while(2>0):
    response = requests.get('https://gtfs.sofiatraffic.bg/api/v1/vehicle-positions')
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.content)
    vehicles = {}

    for entity in feed.entity:        
        latitude = entity.vehicle.position.latitude
        longitude = entity.vehicle.position.longitude
        occupancy = entity.vehicle.occupancy_status
        route_id = entity.vehicle.trip.route_id
        route_real_num = lines[route_id][0]
        vehicle_reg = entity.vehicle.vehicle.id
        stop_id = entity.vehicle.stop_id
        trip_id_str = entity.vehicle.trip.trip_id
        trip_id = trip_id_str.split('-')
        trip_shape = trip_id[1]
        if route_real_num not in vehicles:
            vehicles[route_real_num] = {}
        route_id_type = ''.join([x for x in route_id if x.isalpha()])
        if route_id_type not in vehicles[route_real_num]:
            vehicles[route_real_num][route_id_type] = []
        vehicles[route_real_num][route_id_type].append((vehicle_reg, stop_id, latitude, longitude, occupancy, route_id, trip_shape))
    
    print("Reloaded\n")
    while(1>0):
        clear_output(wait=True)
        inp = input("Enter the desired line number")
        if inp == "e":
            break  
        elif inp == "r":
            break
        if inp not in vehicles:
            print(f"No such line: {inp} \n")
            continue
        [data[t][f].clear() for t in data for f in data[t]]
        map.clear_layers()                  
        num_vehicle_reg = vehicle_reg[-4:]
        for vehicle_type in vehicles[inp].keys():
            vehicle_model_id = reg_to_model[vehicle_type].get(num_vehicle_reg, -1) - 1
            if vehicle_model_id == -1:
                    vehicle_model = "Not found"
            else:
                    vehicle_model = models_ref[vehicle_type][vehicle_model_id]
            for vehicle in vehicles[inp][vehicle_type]:
                vehicle_reg = vehicle[0]
                stop_id = vehicle[1]
                latitude = vehicle[2]
                longitude = vehicle[3]
                occupancy = vehicle [4]
                occupancy = occupancy_dict.get(vehicle[4], "Unknown")
                destination = trip_dict.get(vehicle[5], {}).get(vehicle[6])
                index = vehicle[6]
                if (vehicle_reg.startswith("A")):
                    tr = "av"
                    data[tr]["ke"].append(vehicle_reg)
                    data[tr]["lon"].append(longitude)
                    data[tr]["lat"].append(latitude)
                    data[tr]["stop"].append(stops[stop_id])
                    data[tr]["model"].append(vehicle_model)
                    data[tr]["occ"].append(occupancy)
                    data[tr]["dir"].append(destination)
                    data[tr]["ind"].append(index)

                elif (vehicle_reg.startswith("TB")):
                    tr = "tb"
                    data[tr]["ke"].append(vehicle_reg)
                    data[tr]["lon"].append(longitude)
                    data[tr]["lat"].append(latitude)
                    data[tr]["stop"].append(stops[stop_id])
                    data[tr]["model"].append(vehicle_model)
                    data[tr]["occ"].append(occupancy)
                    data[tr]["dir"].append(destination)
                    data[tr]["ind"].append(index)                
                elif (vehicle_reg.startswith("TM")):
                    tr = "tm"
                    data[tr]["ke"].append(vehicle_reg)
                    data[tr]["lon"].append(longitude)
                    data[tr]["lat"].append(latitude)
                    data[tr]["stop"].append(stops[stop_id])
                    data[tr]["model"].append(vehicle_model)
                    data[tr]["occ"].append(occupancy)
                    data[tr]["dir"].append(destination)
                    data[tr]["ind"].append(index)
                
            
                stop = stops[stop_id]

        map = leafmap.Map(center=[42.6977, 23.3219], zoom=12,
                            zoom_control=False,
                            draw_control=False,
                            scale_control=True,
                            fullscreen_control=False,
                            attribution_control=False,
                            toolbar_control=False,
                            title = f"Lines on the {inp} lines"
                            )
        d_av= {
            "Registration": data["av"]["ke"],
            "Destination": data["av"]["dir"],
            "longitude": data["av"]["lon"],
            "latitude": data["av"]["lat"],
            "Last stop": data["av"]["stop"],
            "Model": data["av"]["model"],
            "Occupancy": data["av"]["occ"],
            }
        df_av = pd.DataFrame(d_av)
        geometry_av = [Point(xy) for xy in zip(df_av['longitude'], df_av['latitude'])]
        df_av = df_av.drop(columns=["longitude","latitude"])
        gdf_av = gpd.GeoDataFrame(df_av, geometry=geometry_av)
        gdf_av.crs = "EPSG:4326"
        if(df_av.empty!=1):
            style_av = {"color": "red", "shape":"circle"}
            map.add_gdf(gdf_av, point_style=style_av, layer_name="Buses", zoom_to_layer=False, info_mode='None')
            
            for e, f in routes[data["av"]["ind"][1]].items():
                coords_av.append(routes[data["av"]["ind"][1]][e])
            line_av = LineString(coords_av)
            gdf_bus_line = gpd.GeoDataFrame([{"Type":"Bus"}], geometry = [line_av], crs="EPSG:4326")
            map.add_gdf(gdf_bus_line, layer_name="Bus_Line", style={"color":"red", "weight": 5}, zoom_to_layer=False, info_mode='None')
            

        d_tm= {
            "Registration": data["tm"]["ke"],
            "Destination": data["tm"]["dir"],
            "longitude": data["tm"]["lon"],
            "latitude": data["tm"]["lat"],
            "Last stop": data["tm"]["stop"],
            "Model": data["tm"]["model"],
            "Occupancy": data["tm"]["occ"] 
            }
        df_tm = pd.DataFrame(d_tm)
        geometry_tm = [Point(xy) for xy in zip(df_tm['longitude'], df_tm['latitude'])]
        df_tm = df_tm.drop(columns=["longitude","latitude"])
        gdf_tm = gpd.GeoDataFrame(df_tm, geometry=geometry_tm)
        gdf_tm.crs = "EPSG:4326"
        if(df_tm.empty!=1):
            style_tm = {"color": "green", "shape":"circle"}
            map.add_gdf(gdf_tm, point_style=style_tm, layer_name="Trams", zoom_to_layer=False, info_mode='None')
            for e, f in routes[data["tm"]["ind"][1]].items():
                coords_tm.append(routes[data["tm"]["ind"][1]][e])
            line_tm = LineString(coords_tm)
            gdf_tram_line = gpd.GeoDataFrame([{"Type":"Tram"}], geometry = [line_tm], crs="EPSG:4326")
            map.add_gdf(gdf_tram_line, layer_name="Tram_Line", style={"color":"green", "weight": 5}, zoom_to_layer=False, info_mode='None')

        d_tb= {
            "Registration": data["tb"]["ke"],
            "Destination": data["tb"]["dir"],
            "longitude": data["tb"]["lon"],
            "latitude": data["tb"]["lat"],
            "Last stop": data["tb"]["stop"],
            "Model": data["tb"]["model"],
            "Occupancy": data["tb"]["occ"] 
            }
        df_tb = pd.DataFrame(d_tb)
        geometry_tb = [Point(xy) for xy in zip(df_tb['longitude'], df_tb['latitude'])]
        df_tb = df_tb.drop(columns=["longitude","latitude"])
        gdf_tb = gpd.GeoDataFrame(df_tb, geometry=geometry_tb)
        gdf_tb.crs = "EPSG:4326"
        if(df_tb.empty!=1):
            style_tb = {"color": "blue", "shape":"circle"}
            map.add_gdf(gdf_tb, point_style=style_tb, layer_name="Trolleys", zoom_to_layer=False, info_mode='None')
            for e, f in routes[data["tb"]["ind"][1]].items():
                coords_tb.append(routes[data["tb"]["ind"][1]][e])
            line_tb = LineString(coords_tb)
            gdf_trb_line = gpd.GeoDataFrame([{"Type":"Trolley"}], geometry = [line_tb], crs="EPSG:4326")
            map.add_gdf(gdf_trb_line, layer_name="Trolley_Line", style={"color":"blue", "weight": 5}, zoom_to_layer=False, info_mode='None')

        gdf_invis = pd.concat([gdf_av, gdf_tm, gdf_tb], ignore_index=True)
        gdf_invis.crs = "EPSG:4326"
        style_invis = {"color": "transparent", "fillOpacity": 0}
        map.add_gdf(gdf_invis, point_style=style_invis, layer_name="Lines", zoom_to_layer=False, info_mode='on_click')

        legend_dict = {
            f"Busses from the {inp} line": "#ff0000",
            f"Trolleys from the {inp} line": "#2600ff",
            f"Trams from the {inp} line": "#00eb17"
            }  
        map.add_legend(f"Lines \"{inp}\"", legend_dict=legend_dict, position="topright",)
        display(map)
        inp = "e"
        break

    if inp == "e":
        break 

Map(center=[42.6977, 23.3219], controls=(ScaleControl(options=['imperial', 'max_width', 'metric', 'position', …